In [2]:
import pandas as pd
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import statsmodels.api as sm
from scipy import stats
from scipy.stats import shapiro # Shapiro-Wilk 검정(정규성 검정. 표본이 작을 때 적절함)
from scipy.stats import kruskal
from scipy.stats import pearsonr
from scipy.stats import spearmanr # 비모수 데이터 상관계수 계산
from scipy.stats import chi2_contingency
from scipy.stats import kstest # Kolmogorov-Smirnov 검정(정규성 검정. 표본이 클 때 적절함)
from scipy.stats import anderson # Anderson-Darling 검정(정규성 검정. 지수, 로그정규 등도 비교 가능)
from sklearn.preprocessing import FunctionTransformer # 정규화를 위한 로그변환, symlog 변환 등에 사용
from sklearn.preprocessing import PowerTransformer # 정규화를 위한 Yeo-Jhonson 변환
import seaborn as sns
import pingouin as pg
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# 수정 테스트 다시

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
stup_origin = pd.read_excel('C:/플랫폼 SME 연구센터/투자여부예측/스타트업기준_분석데이터_250226.xlsx')
stup_origin

,Case_num,id,name,inv_round,est_date,inv_date,event,inv_amounts,KSIC,nts_cd,kostat_cd,cls_cd,tech,o2o,inv_dur,Patent_Class_C,Patent_Class_B_minus,Patent_Class_B0,Patent_Class_B_plus,Patent_Class_A_minus,Patent_Class_A0,Patent_Class_A_plus,Patent_Class_AA,cofounder_yn,edu_lvl,prev_exp_relev_yn,prev_job_tenure_5y,edu_bg,startup_exp,prev_mgr_yn,prev_c_lvl_yn,ROE,ROA,ratio_of_netincome_to_sales,equity_ratio,Debt_ratio,pre_2y_sales,pre_1y_sales,rate_of_sales_increase,pre_2y_opincome,pre_1y_opincome,rate_of_opincome_increase,pre_2y_netincome,pre_1y_netincome,rate_of_netincome_increase,pre_2y_assets,pre_1y_assets,rate_of_assets_increase,pre_1y_liabilities,pre_1y_equity
0,2,10,고미코퍼레이션,Series A,2019-02-12,2021-06-21,1,55.0,전자상거래 소매 중개업,152094,47911,G,Non-Tech,2,16,0,1,0,0,0,0,0,0,1,3,0,2,1,1,1,1,0.067707,0.015323,0.009139,0.226318,3.418554,5.99137,56.97477,8.509473,-1.07856,-4.03019,-2.736640,-0.63027,0.52070,1.826154,7.79213,33.98093,3.360930,26.29042,7.69051
1,3,10,고미코퍼레이션,Series B,2019-02-12,2021-12-21,1,140.0,전자상거래 소매 중개업,152094,47911,G,Non-Tech,2,6,0,1,0,0,0,0,0,0,1,3,0,2,1,1,1,1,0.067707,0.015323,0.009139,0.226318,3.418554,5.99137,56.97477,8.509473,-1.07856,-4.03019,-2.736640,-0.63027,0.52070,1.826154,7.79213,33.98093,3.360930,26.29042,7.69051
2,4,10,고미코퍼레이션,Series B,2019-02-12,2022-01-28,1,140.0,전자상거래 소매 중개업,152094,47911,G,Non-Tech,2,1,0,1,0,0,0,0,0,0,1,3,0,2,1,1,1,1,-0.460074,-0.316364,-0.330277,0.687636,0.454258,56.97477,107.43747,0.885703,-4.03019,-38.33597,-8.512199,0.52070,-35.48415,-69.147014,33.98093,112.16258,2.300751,35.03557,77.12701
3,5,13,고피자,Series B,2017-08-25,2021-05-25,1,110.0,"피자, 햄버거, 샌드위치 및 유사 음식점업",152221,56192,I,Non-Tech,3,7,2,0,0,0,0,0,0,0,0,4,0,2,1,0,0,0,-0.686378,-0.301642,-0.270323,0.439469,1.275471,44.38586,72.82602,0.640748,-14.83540,-27.93240,-0.882821,-11.88098,-19.68655,-0.656980,45.95514,65.26454,0.420179,36.58277,28.68177
4,6,13,고피자,Series C,2017-08-25,2022-10-21,1,250.0,"피자, 햄버거, 샌드위치 및 유사 음식점업",152221,56192,I,Non-Tech,3,16,2,0,0,0,0,0,0,0,0,4,0,2,1,0,0,0,-0.965694,-0.599446,-0.660491,0.620741,0.610977,72.82602,107.26536,0.472899,-27.93240,-50.75008,-0.816889,-19.68655,-70.84780,-2.598792,65.26454,118.18878,0.810919,44.82411,73.36467
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
832,911,3873,퓨쳐메디신,Series C,2015-06-04,2020-03-01,1,65.0,의학 및 약학 연구개발업,152359,70113,M,Bio,3,11,2,6,7,0,0,0,0,0,1,5,1,2,1,0,1,1,1.408385,-2.209068,-1.000000,-1.568512,-1.637547,0.22916,0.00000,-1.000000,-42.26703,-71.62066,-0.694481,-45.98936,-107.02011,-1.327062,44.57182,48.44581,0.086916,124.43365,-75.98784
833,912,3873,퓨쳐메디신,Pre-IPO,2015-06-04,2020-12-23,1,72.0,의학 및 약학 연구개발업,152359,70113,M,Bio,3,9,2,6,7,0,0,0,0,0,1,5,1,2,1,0,1,1,1.408385,-2.209068,-1.000000,-1.568512,-1.637547,0.22916,0.00000,-1.000000,-42.26703,-71.62066,-0.694481,-45.98936,-107.02011,-1.327062,44.57182,48.44581,0.086916,124.43365,-75.98784
834,913,3873,퓨쳐메디신,Pre-IPO,2015-06-04,2021-02-01,1,72.0,의학 및 약학 연구개발업,152359,70113,M,Bio,3,1,4,6,7,0,0,0,0,0,1,5,1,2,1,0,1,1,0.812943,-1.153496,-1.000000,-1.418915,-1.704764,0.00000,0.00000,0.000000,-71.62066,-59.00817,0.176101,-107.02011,-105.02185,0.018672,48.44581,91.04655,0.879348,220.23385,-129.18730
835,914,3873,퓨쳐메디신,Pre-IPO,2015-06-04,2023-03-01,1,22.0,의학 및 약학 연구개발업,152359,70113,M,Bio,3,25,6,6,7,0,0,0,0,0,1,5,1,2,1,0,1,1,-4.588304,-1.159582,-1.000000,0.252726,2.956862,10.00000,0.00000,-1.000000,-64.99483,-75.78444,-0.166007,-128.91379,-69.99527,0.457038,84.48898,60.36252,-0.285557,45.10737,15.25515


In [4]:
stup_origin.columns

Index(['Case_num', 'id', 'name', 'inv_round', 'est_date', 'inv_date', 'event',
       'inv_amounts', 'KSIC', 'nts_cd', 'kostat_cd', 'cls_cd', 'tech', 'o2o',
       'inv_dur', 'Patent_Class_C', 'Patent_Class_B_minus', 'Patent_Class_B0',
       'Patent_Class_B_plus', 'Patent_Class_A_minus', 'Patent_Class_A0',
       'Patent_Class_A_plus', 'Patent_Class_AA', 'cofounder_yn', 'edu_lvl',
       'prev_exp_relev_yn', 'prev_job_tenure_5y', 'edu_bg', 'startup_exp',
       'prev_mgr_yn', 'prev_c_lvl_yn', 'ROE', 'ROA',
       'ratio_of_netincome_to_sales', 'equity_ratio', 'Debt_ratio',
       'pre_2y_sales', 'pre_1y_sales', 'rate_of_sales_increase',
       'pre_2y_opincome', 'pre_1y_opincome', 'rate_of_opincome_increase',
       'pre_2y_netincome', 'pre_1y_netincome', 'rate_of_netincome_increase',
       'pre_2y_assets', 'pre_1y_assets', 'rate_of_assets_increase',
       'pre_1y_liabilities', 'pre_1y_equity'],
      dtype='object')

In [4]:
stup = stup_origin[['inv_round', 'inv_amounts', 'cls_cd', 'tech',
       'inv_dur', 'Patent_Class_C', 'Patent_Class_B_minus', 'Patent_Class_B0',
       'Patent_Class_B_plus', 'Patent_Class_A_minus', 'Patent_Class_A0',
       'Patent_Class_A_plus', 'Patent_Class_AA', 'cofounder_yn', 'edu_lvl',
       'prev_exp_relev_yn', 'prev_job_tenure_5y', 'edu_bg', 'startup_exp',
       'ROE', 'ROA', 'ratio_of_netincome_to_sales', 'equity_ratio', 'Debt_ratio',
       'pre_1y_sales', 'rate_of_sales_increase', 'pre_1y_opincome', 'rate_of_opincome_increase',
       'pre_1y_netincome', 'rate_of_netincome_increase',
       'pre_1y_assets', 'rate_of_assets_increase',
       'pre_1y_liabilities', 'pre_1y_equity']]

In [21]:
invst_origin = pd.read_excel('C:/플랫폼 SME 연구센터/투자여부예측/투자자기준_분석데이터_250317.xlsx')
invst_origin

,id,Case_num,investor,name,investor_type,inv_round,est_date,inv_date,inv_dur,inv_amounts,...,pre_1y_netincome,rate_of_netincome_increase,pre_2y_assets,pre_1y_assets,rate_of_assets_increase,pre_1y_liabilities,pre_1y_equity,이사회취임일,BoD_early_late,sequantial_inv
0,10,4,에프앤에프파트너스,고미코퍼레이션,기업벤처캐피탈,Series A,2019-02-12,2021-06-21,NaN,55.0,...,0.52070,1.826154,7.79213,33.98093,3.360930,26.29042,7.69051,NaT,NaN,x
1,10,7,유비쿼스인베스트먼트,고미코퍼레이션,벤처캐피탈,Series A,2019-02-12,2021-06-21,NaN,55.0,...,0.52070,1.826154,7.79213,33.98093,3.360930,26.29042,7.69051,NaT,NaN,x
2,10,9,중소기업은행,고미코퍼레이션,은행,Series A,2019-02-12,2021-06-21,NaN,55.0,...,0.52070,1.826154,7.79213,33.98093,3.360930,26.29042,7.69051,NaT,NaN,x
3,10,12,케이비인베스트먼트,고미코퍼레이션,벤처캐피탈,Series A,2019-02-12,2021-06-21,16.0,55.0,...,0.52070,1.826154,7.79213,33.98093,3.360930,26.29042,7.69051,2021-05-03,1.0,1
4,36,39,아주아이비투자,꾸까,벤처캐피탈,Series A,2014-04-02,2020-04-13,19.0,30.0,...,-21.31349,-1.801900,29.77215,8.56532,-0.712304,3.89093,4.67439,NaT,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3741,3087,3797,쎄트렉아이,페리지에어로스페이스,NaN,Series C,2018-06-29,NaT,NaN,NaN,...,-208.61948,-0.269214,154.24841,279.34359,0.810998,631.88669,-352.54310,2022-03-31,2.0,0
3742,3404,3913,인터베스트,디앤디파마텍,NaN,Pre-IPO,2014-11-28,NaT,NaN,NaN,...,91.50052,1.038099,979.85587,950.47829,-0.029982,51.45970,899.01859,2019-11-01,2.0,x
3743,3404,3916,프랙시스캐피탈파트너스,디앤디파마텍,NaN,Pre-IPO,2014-11-28,NaT,NaN,NaN,...,91.50052,1.038099,979.85587,950.47829,-0.029982,51.45970,899.01859,2021-11-19,2.0,0
3744,3744,3990,마그나인베스트먼트,인핸스드바이오,NaN,Series B,2016-11-07,NaT,NaN,NaN,...,-150.48040,0.039251,65.49411,49.40845,-0.245605,298.50152,-249.09307,2019-05-27,2.0,0


In [7]:
invst_origin.columns

Index(['id', 'Case_num', 'investor', 'name', 'investor_type', 'inv_round',
       'est_date', 'inv_date', 'inv_dur', 'inv_amounts', 'BoD_yn', 'Case',
       'address', 'KSIC', 'nts_cd', 'kostat_cd', 'cls_cd', 'tech', 'o2o',
       'listed', 'Patent_Class_C', 'Patent_Class_B_minus', 'Patent_Class_B0',
       'Patent_Class_B_plus', 'Patent_Class_A_minus', 'Patent_Class_A0',
       'Patent_Class_A_plus', 'Patent_Class_AA', 'cofounder_yn', 'edu_lvl',
       'prev_exp_relev_yn', 'prev_job_tenure_5y', 'edu_bg', 'startup_exp',
       'prev_mgr_yn', 'prev_c_lvl_yn', 'ROE', 'ROA',
       'ratio_of_netincome_to_sales', 'equity_ratio', 'Debt_ratio',
       'pre_2y_sales', 'pre_1y_sales', 'rate_of_sales_increase',
       'pre_2y_opincome', 'pre_1y_opincome', 'rate_of_opincome_increase',
       'pre_2y_netincome', 'pre_1y_netincome', 'rate_of_netincome_increase',
       'pre_2y_assets', 'pre_1y_assets', 'rate_of_assets_increase',
       'pre_1y_liabilities', 'pre_1y_equity', '이사회취임일', 'BoD_early_la

In [8]:
invst = invst_origin[['sequantial_inv', 'inv_amounts', 'inv_dur', 'inv_round', 'cls_cd', 'tech',
       'BoD_yn', 'Case', 'Patent_Class_C', 'Patent_Class_B_minus', 'Patent_Class_B0',
       'Patent_Class_B_plus', 'Patent_Class_A_minus', 'Patent_Class_A0',
       'Patent_Class_A_plus', 'Patent_Class_AA', 'cofounder_yn', 'edu_lvl',
       'prev_exp_relev_yn', 'prev_job_tenure_5y', 'edu_bg', 'startup_exp',
       'ROE', 'ROA', 'ratio_of_netincome_to_sales', 'equity_ratio', 'Debt_ratio',
       'pre_1y_sales', 'rate_of_sales_increase',
       'pre_1y_opincome', 'rate_of_opincome_increase',
       'pre_1y_netincome', 'rate_of_netincome_increase',
       'pre_1y_assets', 'rate_of_assets_increase',
       'pre_1y_liabilities', 'pre_1y_equity']]

### 스타트업

In [5]:
stup['Patent_total'] = stup['Patent_Class_C']+stup['Patent_Class_B_minus']+stup['Patent_Class_B0']+stup['Patent_Class_B_plus']+stup['Patent_Class_A_minus']+stup['Patent_Class_A0']+stup['Patent_Class_A_plus']+stup['Patent_Class_AA']
stup['Patent_above_B_minus'] = stup['Patent_Class_B_minus']+stup['Patent_Class_B0']+stup['Patent_Class_B_plus']+stup['Patent_Class_A_minus']+stup['Patent_Class_A0']+stup['Patent_Class_A_plus']+stup['Patent_Class_AA']

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_23408\1076159296.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stup['Patent_total'] = stup['Patent_Class_C']+stup['Patent_Class_B_minus']+stup['Patent_Class_B0']+stup['Patent_Class_B_plus']+stup['Patent_Class_A_minus']+stup['Patent_Class_A0']+stup['Patent_Class_A_plus']+stup['Patent_Class_AA']
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_23408\1076159296.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stup['Patent_above_B_minus'] =

In [6]:
#stup['inv_round_1'] = stup['inv_round'].replace('지원금',1).replace('Seed',1).replace('Pre-A',1).replace('Series A',2).replace('Series B',2).replace('Series C',2).replace('Series D',3).replace('Series E',3).replace('Series F',3).replace('Series G',3).replace('Pre-IPO',3).replace('M&A',3)
stup['early_late'] = stup['inv_round'].replace('지원금',0).replace('Seed',0).replace('Pre-A',0).replace('Series A',0).replace('Series B',1).replace('Series C',1).replace('Series D',1).replace('Series E',1).replace('Series F',1).replace('Series G',1).replace('Pre-IPO',1).replace('M&A',1)

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_23408\4017669143.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  stup['early_late'] = stup['inv_round'].replace('지원금',0).replace('Seed',0).replace('Pre-A',0).replace('Series A',0).replace('Series B',1).replace('Series C',1).replace('Series D',1).replace('Series E',1).replace('Series F',1).replace('Series G',1).replace('Pre-IPO',1).replace('M&A',1)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_23408\4017669143.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versu

In [7]:
stup['tech_1'] = stup['tech'].replace('Non-Tech',0).replace('Tech',1).replace('Bio',1)

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_23408\3245258193.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  stup['tech_1'] = stup['tech'].replace('Non-Tech',0).replace('Tech',1).replace('Bio',1)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_23408\3245258193.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stup['tech_1'] = stup['tech'].replace('Non-Tech',0).replace('Tech',1).replace('Bio',1)


In [8]:
stup['cls_cd'] = stup['cls_cd'].replace('G',1).replace('I',2).replace('J',3).replace('K',4).replace('M',5).replace('H',6).replace('C',7).replace('N',8).replace('D',9).replace('P',10).replace('S',11).replace('L',12).replace('E',13)

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_23408\2162619682.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  stup['cls_cd'] = stup['cls_cd'].replace('G',1).replace('I',2).replace('J',3).replace('K',4).replace('M',5).replace('H',6).replace('C',7).replace('N',8).replace('D',9).replace('P',10).replace('S',11).replace('L',12).replace('E',13)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_23408\2162619682.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stup['cls_cd'] = stup['cls_cd'].replace('G'

In [9]:
stup['prev_exp_relev_yn'] = stup['prev_exp_relev_yn'].replace(2,0)
stup['edu_bg'] = stup['edu_bg'].replace(2,0)

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_23408\3260559428.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stup['prev_exp_relev_yn'] = stup['prev_exp_relev_yn'].replace(2,0)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_23408\3260559428.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stup['edu_bg'] = stup['edu_bg'].replace(2,0)


In [10]:
conti_var = ['inv_amounts','inv_dur', 'Patent_total','Patent_above_B_minus', 'ROE', 'ROA',
       'ratio_of_netincome_to_sales', 'equity_ratio', 'Debt_ratio',
       'pre_1y_sales', 'rate_of_sales_increase',
       'pre_1y_opincome', 'rate_of_opincome_increase',
       'pre_1y_netincome', 'rate_of_netincome_increase',
       'pre_1y_assets', 'rate_of_assets_increase',
       'pre_1y_liabilities', 'pre_1y_equity']

In [11]:
stup_nmnl_var = ['early_late','tech_1', 'cls_cd','cofounder_yn', 'edu_lvl', 'prev_exp_relev_yn', 'prev_job_tenure_5y', 'edu_bg', 'startup_exp']

In [13]:
# 상관분석
stup_1 = stup[conti_var]
stup_1

,inv_amounts,inv_dur,Patent_total,Patent_above_B_minus,ROE,ROA,ratio_of_netincome_to_sales,equity_ratio,Debt_ratio,pre_1y_sales,rate_of_sales_increase,pre_1y_opincome,rate_of_opincome_increase,pre_1y_netincome,rate_of_netincome_increase,pre_1y_assets,rate_of_assets_increase,pre_1y_liabilities,pre_1y_equity
0,55.0,16,1,1,0.067707,0.015323,0.009139,0.226318,3.418554,56.97477,8.509473,-4.03019,-2.736640,0.52070,1.826154,33.98093,3.360930,26.29042,7.69051
1,140.0,6,1,1,0.067707,0.015323,0.009139,0.226318,3.418554,56.97477,8.509473,-4.03019,-2.736640,0.52070,1.826154,33.98093,3.360930,26.29042,7.69051
2,140.0,1,1,1,-0.460074,-0.316364,-0.330277,0.687636,0.454258,107.43747,0.885703,-38.33597,-8.512199,-35.48415,-69.147014,112.16258,2.300751,35.03557,77.12701
3,110.0,7,2,0,-0.686378,-0.301642,-0.270323,0.439469,1.275471,72.82602,0.640748,-27.93240,-0.882821,-19.68655,-0.656980,65.26454,0.420179,36.58277,28.68177
4,250.0,16,2,0,-0.965694,-0.599446,-0.660491,0.620741,0.610977,107.26536,0.472899,-50.75008,-0.816889,-70.84780,-2.598792,118.18878,0.810919,44.82411,73.36467
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
832,65.0,11,15,13,1.408385,-2.209068,-1.000000,-1.568512,-1.637547,0.00000,-1.000000,-71.62066,-0.694481,-107.02011,-1.327062,48.44581,0.086916,124.43365,-75.98784
833,72.0,9,15,13,1.408385,-2.209068,-1.000000,-1.568512,-1.637547,0.00000,-1.000000,-71.62066,-0.694481,-107.02011,-1.327062,48.44581,0.086916,124.43365,-75.98784
834,72.0,1,17,13,0.812943,-1.153496,-1.000000,-1.418915,-1.704764,0.00000,0.000000,-59.00817,0.176101,-105.02185,0.018672,91.04655,0.879348,220.23385,-129.18730
835,22.0,25,19,13,-4.588304,-1.159582,-1.000000,0.252726,2.956862,0.00000,-1.000000,-75.78444,-0.166007,-69.99527,0.457038,60.36252,-0.285557,45.10737,15.25515


In [18]:
# T-test, Anova
stup_1 = stup[stup_nmnl_var]
stup_1['inv_amounts'] = stup['inv_amounts']
stup_1['inv_dur'] = stup['inv_dur']
stup_1

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_52296\552374363.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stup_1['inv_amounts'] = stup['inv_amounts']
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_52296\552374363.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stup_1['inv_dur'] = stup['inv_dur']


,early_late,tech_1,cls_cd,cofounder_yn,edu_lvl,prev_exp_relev_yn,prev_job_tenure_5y,edu_bg,startup_exp,inv_amounts,inv_dur
0,0,0,1,1,3,0,2,1,1,55.0,16
1,1,0,1,1,3,0,2,1,1,140.0,6
2,1,0,1,1,3,0,2,1,1,140.0,1
3,1,0,2,0,4,0,2,1,0,110.0,7
4,1,0,2,0,4,0,2,1,0,250.0,16
...,...,...,...,...,...,...,...,...,...,...,...
832,1,1,5,1,5,1,2,1,0,65.0,11
833,1,1,5,1,5,1,2,1,0,72.0,9
834,1,1,5,1,5,1,2,1,0,72.0,1
835,1,1,5,1,5,1,2,1,0,22.0,25


### 투자자

In [176]:
invst['Patent_total'] = invst['Patent_Class_C']+invst['Patent_Class_B_minus']+invst['Patent_Class_B0']+invst['Patent_Class_B_plus']+invst['Patent_Class_A_minus']+invst['Patent_Class_A0']+invst['Patent_Class_A_plus']+invst['Patent_Class_AA']
invst['Patent_above_B_minus'] = invst['Patent_Class_B_minus']+invst['Patent_Class_B0']+invst['Patent_Class_B_plus']+invst['Patent_Class_A_minus']+invst['Patent_Class_A0']+invst['Patent_Class_A_plus']+invst['Patent_Class_AA']

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_35456\3991312295.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  invst['Patent_total'] = invst['Patent_Class_C']+invst['Patent_Class_B_minus']+invst['Patent_Class_B0']+invst['Patent_Class_B_plus']+invst['Patent_Class_A_minus']+invst['Patent_Class_A0']+invst['Patent_Class_A_plus']+invst['Patent_Class_AA']
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_35456\3991312295.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  invst['Patent_above_B

In [177]:
invst['early_late'] = invst['inv_round'].replace('지원금',0).replace('Seed',0).replace('Pre-A',0).replace('Series A',0).replace('Series B',1).replace('Series C',1).replace('Series D',1).replace('Series E',1).replace('Series F',1).replace('Series G',1).replace('Pre-IPO',1).replace('M&A',1)

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_35456\3631998284.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  invst['early_late'] = invst['inv_round'].replace('지원금',0).replace('Seed',0).replace('Pre-A',0).replace('Series A',0).replace('Series B',1).replace('Series C',1).replace('Series D',1).replace('Series E',1).replace('Series F',1).replace('Series G',1).replace('Pre-IPO',1).replace('M&A',1)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_35456\3631998284.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ver

In [178]:
invst['tech_1'] = invst['tech'].replace('Non-Tech',0).replace('Tech',1).replace('Bio',1)

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_35456\2197009007.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  invst['tech_1'] = invst['tech'].replace('Non-Tech',0).replace('Tech',1).replace('Bio',1)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_35456\2197009007.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  invst['tech_1'] = invst['tech'].replace('Non-Tech',0).replace('Tech',1).replace('Bio',1)


In [324]:
invst['cls_cd'] = invst['cls_cd'].replace('G',1).replace('I',2).replace('J',3).replace('K',4).replace('M',5).replace('H',6).replace('C',7).replace('N',8).replace('D',9).replace('P',10).replace('S',11).replace('L',12).replace('E',13)

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_35456\2073223084.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  invst['cls_cd'] = invst['cls_cd'].replace('G',1).replace('I',2).replace('J',3).replace('K',4).replace('M',5).replace('H',6).replace('C',7).replace('N',8).replace('D',9).replace('P',10).replace('S',11).replace('L',12).replace('E',13)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_35456\2073223084.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  invst['cls_cd'] = invst['cls_cd'].replace

In [272]:
invst['prev_exp_relev_yn'] = invst['prev_exp_relev_yn'].replace(2,0)
invst['edu_bg'] = invst['edu_bg'].replace(2,0)

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_35456\2346360814.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  invst['prev_exp_relev_yn'] = invst['prev_exp_relev_yn'].replace(2,0)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_35456\2346360814.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  invst['edu_bg'] = invst['edu_bg'].replace(2,0)


In [179]:
invst['Case_1'] = invst['Case'].replace('First',0).replace('A',1).replace('B',2).replace('C',3).replace('D',4)

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_35456\2763549874.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  invst['Case_1'] = invst['Case'].replace('First',0).replace('A',1).replace('B',2).replace('C',3).replace('D',4)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_35456\2763549874.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  invst['Case_1'] = invst['Case'].replace('First',0).replace('A',1).replace('B',2).replace('C',3).replace('D',4)


In [180]:
invst.columns

Index(['sequantial_inv', 'inv_amounts', 'inv_dur', 'inv_round', 'cls_cd',
       'tech', 'BoD_yn', 'Case', 'Patent_Class_C', 'Patent_Class_B_minus',
       'Patent_Class_B0', 'Patent_Class_B_plus', 'Patent_Class_A_minus',
       'Patent_Class_A0', 'Patent_Class_A_plus', 'Patent_Class_AA',
       'cofounder_yn', 'edu_lvl', 'prev_exp_relev_yn', 'prev_job_tenure_5y',
       'edu_bg', 'startup_exp', 'ROE', 'ROA', 'ratio_of_netincome_to_sales',
       'equity_ratio', 'Debt_ratio', 'pre_1y_sales', 'rate_of_sales_increase',
       'pre_1y_opincome', 'rate_of_opincome_increase', 'pre_1y_netincome',
       'rate_of_netincome_increase', 'pre_1y_assets',
       'rate_of_assets_increase', 'pre_1y_liabilities', 'pre_1y_equity',
       'Patent_total', 'Patent_above_B_minus', 'early_late', 'tech_1',
       'Case_1'],
      dtype='object')

In [325]:
nmnl_var = ['early_late', 'tech_1', 'BoD_yn', 'cls_cd','cofounder_yn', 'edu_lvl', 'prev_exp_relev_yn', 'prev_job_tenure_5y', 'edu_bg', 'startup_exp']

In [326]:
invst_h4_1 = invst[conti_var + nmnl_var]
invst_h4_1

,inv_amounts,inv_dur,Patent_total,Patent_above_B_minus,ROE,ROA,ratio_of_netincome_to_sales,equity_ratio,Debt_ratio,pre_1y_sales,rate_of_sales_increase,pre_1y_opincome,rate_of_opincome_increase,pre_1y_netincome,rate_of_netincome_increase,pre_1y_assets,rate_of_assets_increase,pre_1y_liabilities,pre_1y_equity,early_late,tech_1,BoD_yn,cls_cd,cofounder_yn,edu_lvl,prev_exp_relev_yn,prev_job_tenure_5y,edu_bg,startup_exp
0,55.0,NaN,1,1,0.067707,0.015323,0.009139,0.226318,3.418554,56.97477,8.509473,-4.03019,-2.736640,0.52070,1.826154,33.98093,3.360930,26.29042,7.69051,0,0,0,1,1,3,0,2,1,1
1,55.0,NaN,1,1,0.067707,0.015323,0.009139,0.226318,3.418554,56.97477,8.509473,-4.03019,-2.736640,0.52070,1.826154,33.98093,3.360930,26.29042,7.69051,0,0,0,1,1,3,0,2,1,1
2,55.0,NaN,1,1,0.067707,0.015323,0.009139,0.226318,3.418554,56.97477,8.509473,-4.03019,-2.736640,0.52070,1.826154,33.98093,3.360930,26.29042,7.69051,0,0,0,1,1,3,0,2,1,1
3,55.0,16.0,1,1,0.067707,0.015323,0.009139,0.226318,3.418554,56.97477,8.509473,-4.03019,-2.736640,0.52070,1.826154,33.98093,3.360930,26.29042,7.69051,0,0,1,1,1,3,0,2,1,1
4,30.0,19.0,0,0,-4.559630,-2.488347,-0.473917,0.545734,0.832393,44.97305,0.153301,-21.31652,-1.779929,-21.31349,-1.801900,8.56532,-0.712304,3.89093,4.67439,0,0,0,1,0,3,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3741,NaN,NaN,9,1,0.591756,-0.746820,-50.258491,-1.262041,-1.792367,4.15093,1.000000,-131.61266,-0.508506,-208.61948,-0.269214,279.34359,0.810998,631.88669,-352.54310,1,1,1,5,0,4,1,1,1,0
3742,NaN,NaN,14,7,0.101778,0.096268,0.567899,0.945859,0.057240,161.12110,30.279091,32.92719,1.208316,91.50052,1.038099,950.47829,-0.029982,51.45970,899.01859,1,1,1,5,0,5,0,0,1,0
3743,NaN,NaN,14,7,0.101778,0.096268,0.567899,0.945859,0.057240,161.12110,30.279091,32.92719,1.208316,91.50052,1.038099,950.47829,-0.029982,51.45970,899.01859,1,1,1,5,0,5,0,0,1,0
3744,NaN,NaN,3,1,0.604113,-3.045641,-1104.848752,-5.041507,-1.198353,0.13620,1.270000,-107.60543,-0.575996,-150.48040,0.039251,49.40845,-0.245605,298.50152,-249.09307,1,1,1,7,0,5,0,0,1,0


In [327]:
invst_h4_1['Case_1'] = invst['Case_1']
invst_h4_1['sequantial_inv'] = invst['sequantial_inv']
invst_h4_1

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_35456\3916798222.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  invst_h4_1['Case_1'] = invst['Case_1']
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_35456\3916798222.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  invst_h4_1['sequantial_inv'] = invst['sequantial_inv']


,inv_amounts,inv_dur,Patent_total,Patent_above_B_minus,ROE,ROA,ratio_of_netincome_to_sales,equity_ratio,Debt_ratio,pre_1y_sales,rate_of_sales_increase,pre_1y_opincome,rate_of_opincome_increase,pre_1y_netincome,rate_of_netincome_increase,pre_1y_assets,rate_of_assets_increase,pre_1y_liabilities,pre_1y_equity,early_late,tech_1,BoD_yn,cls_cd,cofounder_yn,edu_lvl,prev_exp_relev_yn,prev_job_tenure_5y,edu_bg,startup_exp,Case_1,sequantial_inv
0,55.0,NaN,1,1,0.067707,0.015323,0.009139,0.226318,3.418554,56.97477,8.509473,-4.03019,-2.736640,0.52070,1.826154,33.98093,3.360930,26.29042,7.69051,0,0,0,1,1,3,0,2,1,1,0,x
1,55.0,NaN,1,1,0.067707,0.015323,0.009139,0.226318,3.418554,56.97477,8.509473,-4.03019,-2.736640,0.52070,1.826154,33.98093,3.360930,26.29042,7.69051,0,0,0,1,1,3,0,2,1,1,0,x
2,55.0,NaN,1,1,0.067707,0.015323,0.009139,0.226318,3.418554,56.97477,8.509473,-4.03019,-2.736640,0.52070,1.826154,33.98093,3.360930,26.29042,7.69051,0,0,0,1,1,3,0,2,1,1,0,x
3,55.0,16.0,1,1,0.067707,0.015323,0.009139,0.226318,3.418554,56.97477,8.509473,-4.03019,-2.736640,0.52070,1.826154,33.98093,3.360930,26.29042,7.69051,0,0,1,1,1,3,0,2,1,1,1,1
4,30.0,19.0,0,0,-4.559630,-2.488347,-0.473917,0.545734,0.832393,44.97305,0.153301,-21.31652,-1.779929,-21.31349,-1.801900,8.56532,-0.712304,3.89093,4.67439,0,0,0,1,0,3,1,1,1,1,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3741,NaN,NaN,9,1,0.591756,-0.746820,-50.258491,-1.262041,-1.792367,4.15093,1.000000,-131.61266,-0.508506,-208.61948,-0.269214,279.34359,0.810998,631.88669,-352.54310,1,1,1,5,0,4,1,1,1,0,2,0
3742,NaN,NaN,14,7,0.101778,0.096268,0.567899,0.945859,0.057240,161.12110,30.279091,32.92719,1.208316,91.50052,1.038099,950.47829,-0.029982,51.45970,899.01859,1,1,1,5,0,5,0,0,1,0,2,x
3743,NaN,NaN,14,7,0.101778,0.096268,0.567899,0.945859,0.057240,161.12110,30.279091,32.92719,1.208316,91.50052,1.038099,950.47829,-0.029982,51.45970,899.01859,1,1,1,5,0,5,0,0,1,0,2,0
3744,NaN,NaN,3,1,0.604113,-3.045641,-1104.848752,-5.041507,-1.198353,0.13620,1.270000,-107.60543,-0.575996,-150.48040,0.039251,49.40845,-0.245605,298.50152,-249.09307,1,1,1,7,0,5,0,0,1,0,2,0


In [328]:
invst_h4_1_a = invst_h4_1[invst_h4_1['sequantial_inv'].isin([0,1])]
invst_h4_1_a = invst_h4_1_a.drop(['inv_amounts','inv_dur','Case_1'],axis=1)
invst_h4_1_a['sequantial_inv'] = invst_h4_1_a['sequantial_inv'].astype(int)
invst_h4_1_a

,Patent_total,Patent_above_B_minus,ROE,ROA,ratio_of_netincome_to_sales,equity_ratio,Debt_ratio,pre_1y_sales,rate_of_sales_increase,pre_1y_opincome,rate_of_opincome_increase,pre_1y_netincome,rate_of_netincome_increase,pre_1y_assets,rate_of_assets_increase,pre_1y_liabilities,pre_1y_equity,early_late,tech_1,BoD_yn,cls_cd,cofounder_yn,edu_lvl,prev_exp_relev_yn,prev_job_tenure_5y,edu_bg,startup_exp,sequantial_inv
3,1,1,0.067707,0.015323,0.009139,0.226318,3.418554,56.97477,8.509473,-4.03019,-2.736640,0.52070,1.826154,33.98093,3.360930,26.29042,7.69051,0,0,1,1,1,3,0,2,1,1,1
4,0,0,-4.559630,-2.488347,-0.473917,0.545734,0.832393,44.97305,0.153301,-21.31652,-1.779929,-21.31349,-1.801900,8.56532,-0.712304,3.89093,4.67439,0,0,0,1,0,3,1,1,1,1,1
5,0,0,-4.559630,-2.488347,-0.473917,0.545734,0.832393,44.97305,0.153301,-21.31652,-1.779929,-21.31349,-1.801900,8.56532,-0.712304,3.89093,4.67439,0,0,0,1,0,3,1,1,1,1,1
7,2,1,-0.380925,-0.367148,-1.000000,0.963833,0.037524,0.00000,-1.000000,-23.13272,-12.697240,-23.29834,-17.741978,63.45761,18.468092,2.29505,61.16256,0,0,0,1,0,5,0,2,0,0,1
8,2,1,-0.380925,-0.367148,-1.000000,0.963833,0.037524,0.00000,-1.000000,-23.13272,-12.697240,-23.29834,-17.741978,63.45761,18.468092,2.29505,61.16256,0,0,0,1,0,5,0,2,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3739,3,2,-0.382241,-0.373907,-9.927318,0.978197,0.022289,11.59130,-0.726749,-117.14166,-3.798941,-115.07052,-3.823727,307.75206,2.195085,6.70987,301.04219,1,1,1,5,0,4,0,0,0,0,0
3741,9,1,0.591756,-0.746820,-50.258491,-1.262041,-1.792367,4.15093,1.000000,-131.61266,-0.508506,-208.61948,-0.269214,279.34359,0.810998,631.88669,-352.54310,1,1,1,5,0,4,1,1,1,0,0
3743,14,7,0.101778,0.096268,0.567899,0.945859,0.057240,161.12110,30.279091,32.92719,1.208316,91.50052,1.038099,950.47829,-0.029982,51.45970,899.01859,1,1,1,5,0,5,0,0,1,0,0
3744,3,1,0.604113,-3.045641,-1104.848752,-5.041507,-1.198353,0.13620,1.270000,-107.60543,-0.575996,-150.48040,0.039251,49.40845,-0.245605,298.50152,-249.09307,1,1,1,7,0,5,0,0,1,0,0


In [329]:
invst_h4_1_bc = invst_h4_1[invst_h4_1['Case_1'].isin([1,3])]
invst_h4_1_bc = invst_h4_1_bc.drop(['sequantial_inv','Case_1'], axis=1)
invst_h4_1_bc

,inv_amounts,inv_dur,Patent_total,Patent_above_B_minus,ROE,ROA,ratio_of_netincome_to_sales,equity_ratio,Debt_ratio,pre_1y_sales,rate_of_sales_increase,pre_1y_opincome,rate_of_opincome_increase,pre_1y_netincome,rate_of_netincome_increase,pre_1y_assets,rate_of_assets_increase,pre_1y_liabilities,pre_1y_equity,early_late,tech_1,BoD_yn,cls_cd,cofounder_yn,edu_lvl,prev_exp_relev_yn,prev_job_tenure_5y,edu_bg,startup_exp
3,55.0,16.0,1,1,0.067707,0.015323,0.009139,0.226318,3.418554,56.97477,8.509473,-4.03019,-2.736640,0.52070,1.826154,33.98093,3.360930,26.29042,7.69051,0,0,1,1,1,3,0,2,1,1
4,30.0,19.0,0,0,-4.559630,-2.488347,-0.473917,0.545734,0.832393,44.97305,0.153301,-21.31652,-1.779929,-21.31349,-1.801900,8.56532,-0.712304,3.89093,4.67439,0,0,0,1,0,3,1,1,1,1
5,30.0,19.0,0,0,-4.559630,-2.488347,-0.473917,0.545734,0.832393,44.97305,0.153301,-21.31652,-1.779929,-21.31349,-1.801900,8.56532,-0.712304,3.89093,4.67439,0,0,0,1,0,3,1,1,1,1
7,100.0,8.0,2,1,-0.380925,-0.367148,-1.000000,0.963833,0.037524,0.00000,-1.000000,-23.13272,-12.697240,-23.29834,-17.741978,63.45761,18.468092,2.29505,61.16256,0,0,0,1,0,5,0,2,0,0
23,80.0,10.0,4,4,-0.690718,-0.624422,-3.632792,0.904019,0.106172,1.85086,0.290247,-12.74299,-5.863505,-6.72379,-5.911080,10.76803,5.448424,1.03353,9.73450,0,1,1,3,0,5,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3605,247.0,50.0,4,4,-11.028401,-0.729112,-3.116183,0.066112,14.125789,108.05086,0.473225,-206.47927,-0.727271,-336.70624,-0.429617,461.80288,-0.098923,431.27205,30.53083,1,0,0,4,0,3,0,1,1,0
3606,247.0,1.0,4,4,-11.028401,-0.729112,-3.116183,0.066112,14.125789,108.05086,0.473225,-206.47927,-0.727271,-336.70624,-0.429617,461.80288,-0.098923,431.27205,30.53083,1,0,0,4,0,3,0,1,1,0
3607,247.0,13.0,4,4,-11.028401,-0.729112,-3.116183,0.066112,14.125789,108.05086,0.473225,-206.47927,-0.727271,-336.70624,-0.429617,461.80288,-0.098923,431.27205,30.53083,1,0,0,4,0,3,0,1,1,0
3608,247.0,7.0,4,4,-11.028401,-0.729112,-3.116183,0.066112,14.125789,108.05086,0.473225,-206.47927,-0.727271,-336.70624,-0.429617,461.80288,-0.098923,431.27205,30.53083,1,0,0,4,0,3,0,1,1,0


In [330]:
invst_h4_1_a_conti = invst_h4_1_a[conti_var[2:]]
invst_h4_1_a_conti['sequantial_inv'] = invst_h4_1_a['sequantial_inv']
invst_h4_1_a_conti

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_35456\1339158226.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  invst_h4_1_a_conti['sequantial_inv'] = invst_h4_1_a['sequantial_inv']


,Patent_total,Patent_above_B_minus,ROE,ROA,ratio_of_netincome_to_sales,equity_ratio,Debt_ratio,pre_1y_sales,rate_of_sales_increase,pre_1y_opincome,rate_of_opincome_increase,pre_1y_netincome,rate_of_netincome_increase,pre_1y_assets,rate_of_assets_increase,pre_1y_liabilities,pre_1y_equity,sequantial_inv
3,1,1,0.067707,0.015323,0.009139,0.226318,3.418554,56.97477,8.509473,-4.03019,-2.736640,0.52070,1.826154,33.98093,3.360930,26.29042,7.69051,1
4,0,0,-4.559630,-2.488347,-0.473917,0.545734,0.832393,44.97305,0.153301,-21.31652,-1.779929,-21.31349,-1.801900,8.56532,-0.712304,3.89093,4.67439,1
5,0,0,-4.559630,-2.488347,-0.473917,0.545734,0.832393,44.97305,0.153301,-21.31652,-1.779929,-21.31349,-1.801900,8.56532,-0.712304,3.89093,4.67439,1
7,2,1,-0.380925,-0.367148,-1.000000,0.963833,0.037524,0.00000,-1.000000,-23.13272,-12.697240,-23.29834,-17.741978,63.45761,18.468092,2.29505,61.16256,1
8,2,1,-0.380925,-0.367148,-1.000000,0.963833,0.037524,0.00000,-1.000000,-23.13272,-12.697240,-23.29834,-17.741978,63.45761,18.468092,2.29505,61.16256,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3739,3,2,-0.382241,-0.373907,-9.927318,0.978197,0.022289,11.59130,-0.726749,-117.14166,-3.798941,-115.07052,-3.823727,307.75206,2.195085,6.70987,301.04219,0
3741,9,1,0.591756,-0.746820,-50.258491,-1.262041,-1.792367,4.15093,1.000000,-131.61266,-0.508506,-208.61948,-0.269214,279.34359,0.810998,631.88669,-352.54310,0
3743,14,7,0.101778,0.096268,0.567899,0.945859,0.057240,161.12110,30.279091,32.92719,1.208316,91.50052,1.038099,950.47829,-0.029982,51.45970,899.01859,0
3744,3,1,0.604113,-3.045641,-1104.848752,-5.041507,-1.198353,0.13620,1.270000,-107.60543,-0.575996,-150.48040,0.039251,49.40845,-0.245605,298.50152,-249.09307,0


In [331]:
invst_h4_1_a_nmnl = invst_h4_1_a[nmnl_var]
invst_h4_1_a_nmnl['sequantial_inv'] = invst_h4_1_a['sequantial_inv']
invst_h4_1_a_nmnl

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_35456\527930580.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  invst_h4_1_a_nmnl['sequantial_inv'] = invst_h4_1_a['sequantial_inv']


,early_late,tech_1,BoD_yn,cls_cd,cofounder_yn,edu_lvl,prev_exp_relev_yn,prev_job_tenure_5y,edu_bg,startup_exp,sequantial_inv
3,0,0,1,1,1,3,0,2,1,1,1
4,0,0,0,1,0,3,1,1,1,1,1
5,0,0,0,1,0,3,1,1,1,1,1
7,0,0,0,1,0,5,0,2,0,0,1
8,0,0,0,1,0,5,0,2,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
3739,1,1,1,5,0,4,0,0,0,0,0
3741,1,1,1,5,0,4,1,1,1,0,0
3743,1,1,1,5,0,5,0,0,1,0,0
3744,1,1,1,7,0,5,0,0,1,0,0


In [356]:
for col in invst_h4_1_bc_nmnl.columns[:-2]:
    print(invst_h4_1_bc_nmnl[col].value_counts().sort_index())

early_late
0     201
1    1021
Name: count, dtype: int64
tech_1
0    244
1    978
Name: count, dtype: int64
BoD_yn
0    1012
1     210
Name: count, dtype: int64
cls_cd
1     144
2       6
3     465
4      39
5     304
6      17
7     181
8      22
9       4
10      6
11     14
12     14
13      6
Name: count, dtype: int64
cofounder_yn
0    1086
1     136
Name: count, dtype: int64
edu_lvl
0     41
1     18
2      5
3    478
4    261
5    419
Name: count, dtype: int64
prev_exp_relev_yn
0    826
1    396
Name: count, dtype: int64
prev_job_tenure_5y
0    133
1    775
2    314
Name: count, dtype: int64
edu_bg
0    294
1    928
Name: count, dtype: int64
startup_exp
0    833
1    378
2     11
Name: count, dtype: int64


In [333]:
invst_h4_1_bc_conti = invst_h4_1_bc[conti_var]
invst_h4_1_bc_conti

,inv_amounts,inv_dur,Patent_total,Patent_above_B_minus,ROE,ROA,ratio_of_netincome_to_sales,equity_ratio,Debt_ratio,pre_1y_sales,rate_of_sales_increase,pre_1y_opincome,rate_of_opincome_increase,pre_1y_netincome,rate_of_netincome_increase,pre_1y_assets,rate_of_assets_increase,pre_1y_liabilities,pre_1y_equity
3,55.0,16.0,1,1,0.067707,0.015323,0.009139,0.226318,3.418554,56.97477,8.509473,-4.03019,-2.736640,0.52070,1.826154,33.98093,3.360930,26.29042,7.69051
4,30.0,19.0,0,0,-4.559630,-2.488347,-0.473917,0.545734,0.832393,44.97305,0.153301,-21.31652,-1.779929,-21.31349,-1.801900,8.56532,-0.712304,3.89093,4.67439
5,30.0,19.0,0,0,-4.559630,-2.488347,-0.473917,0.545734,0.832393,44.97305,0.153301,-21.31652,-1.779929,-21.31349,-1.801900,8.56532,-0.712304,3.89093,4.67439
7,100.0,8.0,2,1,-0.380925,-0.367148,-1.000000,0.963833,0.037524,0.00000,-1.000000,-23.13272,-12.697240,-23.29834,-17.741978,63.45761,18.468092,2.29505,61.16256
23,80.0,10.0,4,4,-0.690718,-0.624422,-3.632792,0.904019,0.106172,1.85086,0.290247,-12.74299,-5.863505,-6.72379,-5.911080,10.76803,5.448424,1.03353,9.73450
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3605,247.0,50.0,4,4,-11.028401,-0.729112,-3.116183,0.066112,14.125789,108.05086,0.473225,-206.47927,-0.727271,-336.70624,-0.429617,461.80288,-0.098923,431.27205,30.53083
3606,247.0,1.0,4,4,-11.028401,-0.729112,-3.116183,0.066112,14.125789,108.05086,0.473225,-206.47927,-0.727271,-336.70624,-0.429617,461.80288,-0.098923,431.27205,30.53083
3607,247.0,13.0,4,4,-11.028401,-0.729112,-3.116183,0.066112,14.125789,108.05086,0.473225,-206.47927,-0.727271,-336.70624,-0.429617,461.80288,-0.098923,431.27205,30.53083
3608,247.0,7.0,4,4,-11.028401,-0.729112,-3.116183,0.066112,14.125789,108.05086,0.473225,-206.47927,-0.727271,-336.70624,-0.429617,461.80288,-0.098923,431.27205,30.53083


In [334]:
invst_h4_1_bc_nmnl = invst_h4_1_bc[nmnl_var]
invst_h4_1_bc_nmnl['inv_amounts'] = invst_h4_1_bc['inv_amounts']
invst_h4_1_bc_nmnl['inv_dur'] = invst_h4_1_bc['inv_dur']
invst_h4_1_bc_nmnl

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_35456\2282705380.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  invst_h4_1_bc_nmnl['inv_amounts'] = invst_h4_1_bc['inv_amounts']
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_35456\2282705380.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  invst_h4_1_bc_nmnl['inv_dur'] = invst_h4_1_bc['inv_dur']


,early_late,tech_1,BoD_yn,cls_cd,cofounder_yn,edu_lvl,prev_exp_relev_yn,prev_job_tenure_5y,edu_bg,startup_exp,inv_amounts,inv_dur
3,0,0,1,1,1,3,0,2,1,1,55.0,16.0
4,0,0,0,1,0,3,1,1,1,1,30.0,19.0
5,0,0,0,1,0,3,1,1,1,1,30.0,19.0
7,0,0,0,1,0,5,0,2,0,0,100.0,8.0
23,0,1,1,3,0,5,0,1,1,0,80.0,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...
3605,1,0,0,4,0,3,0,1,1,0,247.0,50.0
3606,1,0,0,4,0,3,0,1,1,0,247.0,1.0
3607,1,0,0,4,0,3,0,1,1,0,247.0,13.0
3608,1,0,0,4,0,3,0,1,1,0,247.0,7.0


In [69]:
invst_h4_2 = invst[invst['tech_1'].isin([1])] # 기술기반 기업만

In [70]:
invst_h4_2_a = invst_h4_2.drop(['inv_amounts','inv_dur'],axis=1)
invst_h4_2_a

,Case_num,id,investor,name,investor_type,inv_round,est_date,inv_date,BoD_yn,Case,address,KSIC,nts_cd,kostat_cd,cls_cd,tech,o2o,listed,Patent_Class_C,Patent_Class_B_minus,Patent_Class_B0,Patent_Class_B_plus,Patent_Class_A_minus,Patent_Class_A0,Patent_Class_A_plus,Patent_Class_AA,cofounder_yn,edu_lvl,prev_exp_relev_yn,prev_job_tenure_5y,edu_bg,startup_exp,prev_mgr_yn,prev_c_lvl_yn,ROE,ROA,ratio_of_netincome_to_sales,equity_ratio,Debt_ratio,pre_2y_sales,pre_1y_sales,rate_of_sales_increase,pre_2y_opincome,pre_1y_opincome,rate_of_opincome_increase,pre_2y_netincome,pre_1y_netincome,rate_of_netincome_increase,pre_2y_assets,pre_1y_assets,rate_of_assets_increase,pre_1y_liabilities,pre_1y_equity,Patent_total,Patent_above_B_minus,inv_round_1,tech_1,Case_1
22,97,65,삼성벤처투자,노타,기업벤처캐피탈,Series A,2015-04-24,2020-08-18,0,D,"대전 유성구 문지로 193, 창조관 2219호 (문지동)",응용 소프트웨어 개발 및 공급업,152260,58222,J,Tech,2,2,0,3,0,1,0,0,0,0,0,5,0,1,1,0,0,0,-0.690718,-0.624422,-3.632792,0.904019,0.106172,1.434500,1.850860,0.290247,-1.856630,-12.742990,-5.863505,-0.972900,-6.723790,-5.911080,1.669870,10.768030,5.448424,1.033530,9.73450,4,4,4,1,4
23,98,65,스톤브릿지벤처스,노타,벤처캐피탈,Series A,2015-04-24,2020-08-18,1,A,"대전 유성구 문지로 193, 창조관 2219호 (문지동)",응용 소프트웨어 개발 및 공급업,152260,58222,J,Tech,2,2,0,3,0,1,0,0,0,0,0,5,0,1,1,0,0,0,-0.690718,-0.624422,-3.632792,0.904019,0.106172,1.434500,1.850860,0.290247,-1.856630,-12.742990,-5.863505,-0.972900,-6.723790,-5.911080,1.669870,10.768030,5.448424,1.033530,9.73450,4,4,4,1,1
24,102,65,엘비인베스트먼트,노타,벤처캐피탈,Series A,2015-04-24,2020-08-18,0,First,"대전 유성구 문지로 193, 창조관 2219호 (문지동)",응용 소프트웨어 개발 및 공급업,152260,58222,J,Tech,2,2,0,3,0,1,0,0,0,0,0,5,0,1,1,0,0,0,-0.690718,-0.624422,-3.632792,0.904019,0.106172,1.434500,1.850860,0.290247,-1.856630,-12.742990,-5.863505,-0.972900,-6.723790,-5.911080,1.669870,10.768030,5.448424,1.033530,9.73450,4,4,4,1,0
25,105,65,엘지씨엔에스,노타,대기업/중견기업,Series A,2015-04-24,2020-08-18,0,D,"대전 유성구 문지로 193, 창조관 2219호 (문지동)",응용 소프트웨어 개발 및 공급업,152260,58222,J,Tech,2,2,0,3,0,1,0,0,0,0,0,5,0,1,1,0,0,0,-0.690718,-0.624422,-3.632792,0.904019,0.106172,1.434500,1.850860,0.290247,-1.856630,-12.742990,-5.863505,-0.972900,-6.723790,-5.911080,1.669870,10.768030,5.448424,1.033530,9.73450,4,4,4,1,4
26,125,74,에스비아이인베스트먼트,뉴로핏,벤처캐피탈,Series A,2016-03-14,2020-02-04,0,First,"서울 강남구 테헤란로 124, 12층 (역삼동)",응용 소프트웨어 개발 및 공급업,152260,58222,J,Tech,2,2,0,1,1,0,0,0,2,0,0,3,0,1,0,0,0,0,-0.238809,-0.219628,-30.645164,0.919683,0.087332,0.390230,0.448630,0.149655,-3.145270,-13.668930,-3.345869,0.111990,-13.748340,-123.763997,17.954780,62.598270,2.486440,5.027730,57.57054,4,4,4,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3706,2290,1122,아주아이비투자,트리플,NaN,M&A,2016-01-11,NaT,1,B,"경기 성남시 분당구 판교역로14번길 16, 2층, 3층 (백현동)",자료 처리업,152266,63111,J,Tech,2,2,0,0,0,0,0,0,0,0,0,3,1,1,0,0,1,0,-1.102418,-0.408000,-0.557608,0.370096,1.702003,432.358205,516.512065,0.194639,-117.559067,-177.718151,-0.511735,-206.147353,-288.011283,-0.397114,545.777668,705.909399,0.293401,444.655308,261.25409,0,0,11,1,2
3707,2401,1207,앵커에퀴티파트너스코리아,프레시지,NaN,M&A,2016-01-26,NaT,1,B,경기 용인시 처인구 이동읍 삼배울로 23 (덕성리),기타 식사용 가공처리 조리식품 제조업,151131,10759,C,Tech,2,2,0,1,0,0,0,0,0,0,0,3,1,1,1,1,1,1,-2.720058,-1.441134,-1.547467,0.529817,0.887443,2148.537120,1498.849590,-0.302386,-439.931270,-616.201150,-0.400676,-1869.117820,-2319.419950,-0.240917,4044.660490,1609.440960,-0.602083,756.731200,852.70976,1,1,11,1,2
3708,2402,1207,에스비브이에이,프레시지,NaN,M&A,2016-01-26,NaT,1,B,경기 용인시 처인구 이동읍 삼배울로 23 (덕성리),기타 식사용 가공처리 조리식품 제조업,151131,10759,C,Tech,2,2,0,0,0,0,0,0,0,0,0,3,1,1,1,1,1,1,-3.792637,-0.577379,-0.527130,0.152237,5.568716,711.781430,1271.368000,0.786178,-148.963040,-460.563280,-2.091796,-256.646850,-670.176470,-1.611279,970.057370,1160.722670,0.196551,984.018030,176.70464,0,0,11,1,2
3711,2693,1472,에이티넘인베스트먼트,엑소코바이오,NaN,M&A,2017-01-03,NaT,1,B,충북 청주

In [75]:
invst_h4_2_a_conti = invst_h4_2_a[conti_var[2:]]
#invst_h4_2_a_conti['Case_1'] = invst_h4_2_a['Case_1']
invst_h4_2_a_conti

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_37532\1087010585.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  invst_h4_2_a_conti['Case_1'] = invst_h4_2_a['Case_1']


,Patent_total,Patent_above_B_minus,Patent_Class_C,Patent_Class_B_minus,Patent_Class_B0,Patent_Class_B_plus,Patent_Class_A_minus,Patent_Class_A0,Patent_Class_A_plus,Patent_Class_AA,ROE,ROA,ratio_of_netincome_to_sales,equity_ratio,Debt_ratio,pre_2y_sales,pre_1y_sales,rate_of_sales_increase,pre_2y_opincome,pre_1y_opincome,rate_of_opincome_increase,pre_2y_netincome,pre_1y_netincome,rate_of_netincome_increase,pre_2y_assets,pre_1y_assets,rate_of_assets_increase,pre_1y_liabilities,pre_1y_equity,Case_1
22,4,4,0,3,0,1,0,0,0,0,-0.690718,-0.624422,-3.632792,0.904019,0.106172,1.434500,1.850860,0.290247,-1.856630,-12.742990,-5.863505,-0.972900,-6.723790,-5.911080,1.669870,10.768030,5.448424,1.033530,9.73450,4
23,4,4,0,3,0,1,0,0,0,0,-0.690718,-0.624422,-3.632792,0.904019,0.106172,1.434500,1.850860,0.290247,-1.856630,-12.742990,-5.863505,-0.972900,-6.723790,-5.911080,1.669870,10.768030,5.448424,1.033530,9.73450,1
24,4,4,0,3,0,1,0,0,0,0,-0.690718,-0.624422,-3.632792,0.904019,0.106172,1.434500,1.850860,0.290247,-1.856630,-12.742990,-5.863505,-0.972900,-6.723790,-5.911080,1.669870,10.768030,5.448424,1.033530,9.73450,0
25,4,4,0,3,0,1,0,0,0,0,-0.690718,-0.624422,-3.632792,0.904019,0.106172,1.434500,1.850860,0.290247,-1.856630,-12.742990,-5.863505,-0.972900,-6.723790,-5.911080,1.669870,10.768030,5.448424,1.033530,9.73450,4
26,4,4,0,1,1,0,0,0,2,0,-0.238809,-0.219628,-30.645164,0.919683,0.087332,0.390230,0.448630,0.149655,-3.145270,-13.668930,-3.345869,0.111990,-13.748340,-123.763997,17.954780,62.598270,2.486440,5.027730,57.57054,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3706,0,0,0,0,0,0,0,0,0,0,-1.102418,-0.408000,-0.557608,0.370096,1.702003,432.358205,516.512065,0.194639,-117.559067,-177.718151,-0.511735,-206.147353,-288.011283,-0.397114,545.777668,705.909399,0.293401,444.655308,261.25409,2
3707,1,1,0,1,0,0,0,0,0,0,-2.720058,-1.441134,-1.547467,0.529817,0.887443,2148.537120,1498.849590,-0.302386,-439.931270,-616.201150,-0.400676,-1869.117820,-2319.419950,-0.240917,4044.660490,1609.440960,-0.602083,756.731200,852.70976,2
3708,0,0,0,0,0,0,0,0,0,0,-3.792637,-0.577379,-0.527130,0.152237,5.568716,711.781430,1271.368000,0.786178,-148.963040,-460.563280,-2.091796,-256.646850,-670.176470,-1.611279,970.057370,1160.722670,0.196551,984.018030,176.70464,2
3711,40,38,2,13,14,3,3,0,4,1,0.356666,0.222820,0.572833,0.624731,0.600690,243.652210,411.668270,0.689573,89.992730,203.206340,1.258031,86.599970,235.816980,1.723061,677.856370,1058.329210,0.561288,397.158530,661.17068,2


In [74]:
invst_h4_2_a_nmnl = invst_h4_2_a[nmnl_var]
invst_h4_2_a_nmnl

,inv_round_1,tech_1,BoD_yn,cofounder_yn,edu_lvl,prev_exp_relev_yn,prev_job_tenure_5y,edu_bg,startup_exp,prev_mgr_yn,prev_c_lvl_yn,Case_1
22,4,1,0,0,5,0,1,1,0,0,0,4
23,4,1,1,0,5,0,1,1,0,0,0,1
24,4,1,0,0,5,0,1,1,0,0,0,0
25,4,1,0,0,5,0,1,1,0,0,0,4
26,4,1,0,0,3,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
3706,11,1,1,0,3,1,1,0,0,1,0,2
3707,11,1,1,0,3,1,1,1,1,1,1,2
3708,11,1,1,0,3,1,1,1,1,1,1,2
3711,11,1,1,0,4,1,2,1,0,1,1,2


In [77]:
invst_h4_2_bc = invst_h4_2[invst_h4_2['Case_1'].isin([1,3])] # 기술기반 기업 중 Case A, C만

In [79]:
invst_h4_2['Case_1'].value_counts()

Case_1
4    1610
3     816
0     331
1     162
2     107
Name: count, dtype: int64

In [48]:
invst_h4_2.shape

(3026, 60)

In [78]:
invst_h4_2_bc.shape

(978, 60)

In [81]:
invst_h4_2_bc_conti = invst_h4_2_bc[conti_var]
invst_h4_2_bc_conti

,inv_amounts,inv_dur,Patent_total,Patent_above_B_minus,Patent_Class_C,Patent_Class_B_minus,Patent_Class_B0,Patent_Class_B_plus,Patent_Class_A_minus,Patent_Class_A0,Patent_Class_A_plus,Patent_Class_AA,ROE,ROA,ratio_of_netincome_to_sales,equity_ratio,Debt_ratio,pre_2y_sales,pre_1y_sales,rate_of_sales_increase,pre_2y_opincome,pre_1y_opincome,rate_of_opincome_increase,pre_2y_netincome,pre_1y_netincome,rate_of_netincome_increase,pre_2y_assets,pre_1y_assets,rate_of_assets_increase,pre_1y_liabilities,pre_1y_equity
23,80.0,10.0,4,4,0,3,0,1,0,0,0,0,-0.690718,-0.624422,-3.632792,0.904019,0.106172,1.43450,1.85086,0.290247,-1.85663,-12.74299,-5.863505,-0.97290,-6.72379,-5.911080,1.66987,10.76803,5.448424,1.03353,9.73450
31,230.0,20.0,8,3,5,1,1,1,0,0,0,0,-1.061627,-0.968026,-9.420626,0.911832,0.096693,2.73405,1.87687,-0.313520,-0.54453,-17.85600,-31.791582,-0.47069,-17.68129,-36.564618,14.82316,18.26531,0.232214,1.61041,16.65490
34,230.0,20.0,8,3,5,1,1,1,0,0,0,0,-1.061627,-0.968026,-9.420626,0.911832,0.096693,2.73405,1.87687,-0.313520,-0.54453,-17.85600,-31.791582,-0.47069,-17.68129,-36.564618,14.82316,18.26531,0.232214,1.61041,16.65490
37,230.0,20.0,8,3,5,1,1,1,0,0,0,0,-1.061627,-0.968026,-9.420626,0.911832,0.096693,2.73405,1.87687,-0.313520,-0.54453,-17.85600,-31.791582,-0.47069,-17.68129,-36.564618,14.82316,18.26531,0.232214,1.61041,16.65490
51,45.0,25.0,0,0,0,0,0,0,0,0,0,0,0.013892,0.011540,1.000000,0.830692,0.203815,0.00000,0.00000,0.000000,-1.64666,-3.26593,-0.983366,-1.46416,0.55720,1.380560,3.63248,48.28401,12.292299,8.17486,40.10915
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3682,160.0,8.0,1,0,1,0,0,0,0,0,0,0,-9.272905,-1.093379,-6.810842,0.117911,7.480961,7.63636,0.55636,-0.927143,4.63107,-7.69872,-2.662406,4.77792,-3.78928,-1.793082,8.36482,3.46566,-0.585686,3.05702,0.40864
3689,2.7,4.0,1,1,0,0,0,1,0,0,0,0,2.544010,-1.142230,-4.830638,-0.448988,-3.227232,0.00000,0.32776,1.000000,-0.05348,-2.73228,-50.089753,-0.00196,-1.58329,-806.801020,0.04716,1.38614,28.392282,2.00850,-0.62236
3697,4276.7,34.0,25,5,20,4,0,1,0,0,0,0,-0.365972,-0.345572,-85.865839,0.944258,0.059032,0.25062,4.02039,15.041776,-204.82972,-321.08488,-0.567570,-214.72871,-345.21416,-0.607676,284.25966,998.96523,2.514270,55.68399,943.28124
3698,4276.7,35.0,25,5,20,4,0,1,0,0,0,0,-0.365972,-0.345572,-85.865839,0.944258,0.059032,0.25062,4.02039,15.041776,-204.82972,-321.08488,-0.567570,-214.72871,-345.21416,-0.607676,284.25966,998.96523,2.514270,55.68399,943.28124


In [82]:
invst_h4_2_bc_nmnl = invst_h4_2_bc[nmnl_var]
invst_h4_2_bc_nmnl

,inv_round_1,tech_1,BoD_yn,cofounder_yn,edu_lvl,prev_exp_relev_yn,prev_job_tenure_5y,edu_bg,startup_exp,prev_mgr_yn,prev_c_lvl_yn,Case_1
23,4,1,1,0,5,0,1,1,0,0,0,1
31,4,1,0,0,3,0,0,1,0,0,0,3
34,4,1,0,0,3,0,0,1,0,0,0,3
37,4,1,0,0,3,0,0,1,0,0,0,3
51,4,1,0,0,5,0,1,1,0,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...
3682,2,1,0,0,3,2,1,1,0,1,0,3
3689,2,1,0,0,3,0,1,0,1,1,1,3
3697,11,1,0,0,4,1,1,1,0,1,1,3
3698,11,1,0,0,4,1,1,1,0,1,1,3


In [88]:
#conti_var = ['Patent_total','Patent_above_B_minus','Patent_Class_C', 'Patent_Class_B_minus', 'Patent_Class_B0',
#       'Patent_Class_B_plus', 'Patent_Class_A_minus', 'Patent_Class_A0',
#       'Patent_Class_A_plus', 'Patent_Class_AA',  'ROE', 'ROA',
#       'ratio_of_netincome_to_sales', 'equity_ratio', 'Debt_ratio',
#       'pre_2y_sales', 'pre_1y_sales', 'rate_of_sales_increase',
#       'pre_2y_opincome', 'pre_1y_opincome', 'rate_of_opincome_increase',
#       'pre_2y_netincome', 'pre_1y_netincome', 'rate_of_netincome_increase',
#       'pre_2y_assets', 'pre_1y_assets', 'rate_of_assets_increase',
#       'pre_1y_liabilities', 'pre_1y_equity']

## 기술통계량 계산

In [13]:
stup_1 = stup_h2[conti_var]
stup_1

,inv_amounts,inv_dur,Patent_total,Patent_above_B_minus,Patent_Class_C,Patent_Class_B_minus,Patent_Class_B0,Patent_Class_B_plus,Patent_Class_A_minus,Patent_Class_A0,...,pre_1y_opincome,rate_of_opincome_increase,pre_2y_netincome,pre_1y_netincome,rate_of_netincome_increase,pre_2y_assets,pre_1y_assets,rate_of_assets_increase,pre_1y_liabilities,pre_1y_equity
0,55.0,16,1,1,0,1,0,0,0,0,...,-4.03019,-2.736640,-0.63027,0.52070,1.826154,7.79213,33.98093,3.360930,26.29042,7.69051
6,30.0,19,0,0,0,0,0,0,0,0,...,-21.31652,-1.779929,-7.60680,-21.31349,-1.801900,29.77215,8.56532,-0.712304,3.89093,4.67439
13,100.0,7,2,1,1,1,0,0,0,0,...,-23.13272,-12.697240,-1.24311,-23.29834,-17.741978,3.25957,63.45761,18.468092,2.29505,61.16256
14,80.0,13,3,1,2,1,0,0,0,0,...,-44.30315,-0.915173,-23.29834,-63.41895,-1.722037,63.45761,33.55366,-0.471243,6.30418,27.24948
16,80.0,10,4,4,0,3,0,1,0,0,...,-12.74299,-5.863505,-0.97290,-6.72379,-5.911080,1.66987,10.76803,5.448424,1.03353,9.73450
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
817,130.0,12,0,0,0,0,0,0,0,0,...,-10.66728,-1.235493,-4.81665,-10.31685,-1.141914,1.88185,16.22090,7.619656,5.10439,11.11651
823,5.0,21,0,0,0,0,0,0,0,0,...,-0.43887,-19.498365,0.00378,0.50483,132.552910,0.21466,1.13830,4.302804,0.61969,0.51861
824,20.0,6,0,0,0,0,0,0,0,0,...,-0.43887,-19.498365,0.00378,0.50483,132.552910,0.21466,1.13830,4.302804,0.61969,0.51861
825,80.0,16,2,0,2,0,0,0,0,0,...,-19.21652,-2.357817,-2.94332,-14.90306,-4.063350,23.99158,9.11492,-0.620078,1.36277,7.75215


In [11]:
stup_1['inv_round'] = stup['inv_round']
stup_1

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_67660\924880873.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stup_1['inv_round'] = stup['inv_round']


,inv_amounts,inv_dur,Patent_total,Patent_above_B_minus,Patent_Class_C,Patent_Class_B_minus,Patent_Class_B0,Patent_Class_B_plus,Patent_Class_A_minus,Patent_Class_A0,Patent_Class_A_plus,Patent_Class_AA,ROE,ROA,ratio_of_netincome_to_sales,equity_ratio,Debt_ratio,pre_2y_sales,pre_1y_sales,rate_of_sales_increase,pre_2y_opincome,pre_1y_opincome,rate_of_opincome_increase,pre_2y_netincome,pre_1y_netincome,rate_of_netincome_increase,pre_2y_assets,pre_1y_assets,rate_of_assets_increase,pre_1y_liabilities,pre_1y_equity,inv_round
0,55.0,16,1,1,0,1,0,0,0,0,0,0,0.067707,0.015323,0.009139,0.226318,3.418554,5.99137,56.97477,8.509473,-1.07856,-4.03019,-2.736640,-0.63027,0.52070,1.826154,7.79213,33.98093,3.360930,26.29042,7.69051,Series A
1,140.0,6,1,1,0,1,0,0,0,0,0,0,0.067707,0.015323,0.009139,0.226318,3.418554,5.99137,56.97477,8.509473,-1.07856,-4.03019,-2.736640,-0.63027,0.52070,1.826154,7.79213,33.98093,3.360930,26.29042,7.69051,Series B
2,140.0,1,1,1,0,1,0,0,0,0,0,0,-0.460074,-0.316364,-0.330277,0.687636,0.454258,56.97477,107.43747,0.885703,-4.03019,-38.33597,-8.512199,0.52070,-35.48415,-69.147014,33.98093,112.16258,2.300751,35.03557,77.12701,Series B
3,110.0,7,2,0,2,0,0,0,0,0,0,0,-0.686378,-0.301642,-0.270323,0.439469,1.275471,44.38586,72.82602,0.640748,-14.83540,-27.93240,-0.882821,-11.88098,-19.68655,-0.656980,45.95514,65.26454,0.420179,36.58277,28.68177,Series B
4,250.0,16,2,0,2,0,0,0,0,0,0,0,-0.965694,-0.599446,-0.660491,0.620741,0.610977,72.82602,107.26536,0.472899,-27.93240,-50.75008,-0.816889,-19.68655,-70.84780,-2.598792,65.26454,118.18878,0.810919,44.82411,73.36467,Series C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
832,65.0,11,15,13,2,6,7,0,0,0,0,0,1.408385,-2.209068,-1.000000,-1.568512,-1.637547,0.22916,0.00000,-1.000000,-42.26703,-71.62066,-0.694481,-45.98936,-107.02011,-1.327062,44.57182,48.44581,0.086916,124.43365,-75.98784,Series C
833,72.0,9,15,13,2,6,7,0,0,0,0,0,1.408385,-2.209068,-1.000000,-1.568512,-1.637547,0.22916,0.00000,-1.000000,-42.26703,-71.62066,-0.694481,-45.98936,-107.02011,-1.327062,44.57182,48.44581,0.086916,124.43365,-75.98784,Pre-IPO
834,72.0,1,17,13,4,6,7,0,0,0,0,0,0.812943,-1.153496,-1.000000,-1.418915,-1.704764,0.00000,0.00000,0.000000,-71.62066,-59.00817,0.176101,-107.02011,-105.02185,0.018672,48.44581,91.04655,0.879348,220.23385,-129.18730,Pre-IPO
835,22.0,25,19,13,6,6,7,0,0,0,0,0,-4.588304,-1.159582,-1.000000,0.252726,2.956862,10.00000,0.00000,-1.000000,-64.99483,-75.78444,-0.166007,-128.91379,-69.99527,0.457038,84.48898,60.36252,-0.285557,45.10737,15.25515,Pre-IPO


In [12]:
stup_1['inv_round'].value_counts()

inv_round
Series B    289
Series C    172
Series A    171
Pre-IPO      59
Series D     46
Pre-A        26
Series E     20
지원금          17
Seed         13
M&A          12
Series F      6
Series G      6
Name: count, dtype: int64

In [49]:
stup_sris = stup_1[stup_1['inv_round']=='Pre-IPO']
stup_sris = stup_sris.drop('inv_round',axis=1)
stup_sris

,inv_amounts,inv_dur,Patent_total,Patent_above_B_minus,Patent_Class_C,Patent_Class_B_minus,Patent_Class_B0,Patent_Class_B_plus,Patent_Class_A_minus,Patent_Class_A0,Patent_Class_A_plus,Patent_Class_AA,ROE,ROA,ratio_of_netincome_to_sales,equity_ratio,Debt_ratio,pre_2y_sales,pre_1y_sales,rate_of_sales_increase,pre_2y_opincome,pre_1y_opincome,rate_of_opincome_increase,pre_2y_netincome,pre_1y_netincome,rate_of_netincome_increase,pre_2y_assets,pre_1y_assets,rate_of_assets_increase,pre_1y_liabilities,pre_1y_equity
12,250.00,15,1,0,1,0,0,0,0,0,0,0,0.673325,-0.339694,-0.664095,-0.504503,-2.982149,92.18141,172.28661,0.868995,-40.23548,-106.48187,-1.646467,-206.67447,-114.41474,0.446401,118.30965,336.81669,1.846908,506.74172,-169.92503
67,720.00,13,51,41,10,17,11,7,1,3,1,1,0.840370,-3.331045,-58.551861,-3.963785,-1.252284,1.98403,14.30042,6.207764,-115.94851,-209.42738,-0.806210,-469.68465,-837.31620,-0.782720,384.16250,251.36744,-0.345674,1247.73402,-996.36658
68,720.00,3,51,41,10,17,11,7,1,3,1,1,0.840370,-3.331045,-58.551861,-3.963785,-1.252284,1.98403,14.30042,6.207764,-115.94851,-209.42738,-0.806210,-469.68465,-837.31620,-0.782720,384.16250,251.36744,-0.345674,1247.73402,-996.36658
80,3000.00,18,6,4,2,1,1,2,0,0,0,0,0.298790,-0.162482,-0.095269,-0.543800,-2.838911,1150.70000,1555.60000,0.351873,-58.60000,25.70000,1.438567,-470.30000,-148.20000,0.684882,628.30000,912.10000,0.451695,1408.10000,-496.00000
92,30.00,12,42,19,23,4,2,4,2,3,3,1,-0.331040,-0.317129,-1.976104,0.957979,0.043864,15.55946,30.15121,0.937806,-32.75480,-54.95506,-0.677771,-39.76740,-59.58192,-0.498260,70.64152,187.87889,1.659610,7.89478,179.98411
266,131.00,13,18,5,13,3,1,1,0,0,0,0,-3.158530,-2.870712,-107.334648,0.908876,0.100260,0.25084,5.50659,20.952599,-39.67148,-66.98086,-0.688388,-184.94244,-591.04790,-2.195848,174.58865,205.88893,0.179280,18.76140,187.12753
281,150.00,13,1,0,1,0,0,0,0,0,0,0,0.042659,0.020026,0.010834,0.469452,1.130145,612.68839,784.92005,0.281108,-22.53315,-9.58564,0.574598,-156.35689,8.50397,1.054388,314.06296,424.64195,0.352092,225.29312,199.34882
282,22.00,8,1,0,1,0,0,0,0,0,0,0,0.124106,0.055434,0.037957,0.446664,1.238799,1099.24148,1538.49841,0.399600,7.14527,15.90586,1.226068,-8.09335,58.39734,8.215472,697.43714,1053.46205,0.510476,582.90867,470.54337
317,176.00,14,55,49,6,7,17,9,9,2,5,0,0.420762,-1.244309,-8.695156,-2.957276,-1.338149,12.14503,23.36205,0.923589,-69.91325,-96.07462,-0.374198,-188.24290,-203.13668,-0.079120,33.30294,163.25260,3.902048,646.03554,-482.78294
362,145.00,31,25,14,11,11,2,1,0,0,0,0,-1.230371,-1.105296,-61.311012,0.898344,0.113160,1.00000,1.50245,0.502450,-66.57046,-73.71310,-0.107294,-133.08582,-92.11673,0.307840,148.89780,83.34122,-0.440279,8.47217,74.86905


In [50]:
# describe() 출력
desc = stup_sris.describe()

# 왜도와 첨도 추가
skewness = stup_sris.skew()
kurtosis = stup_sris.kurt()

# 기술통계량에 왜도와 첨도 추가
desc.loc['skew'] = skewness
desc.loc['kurtosis'] = kurtosis
desc

,inv_amounts,inv_dur,Patent_total,Patent_above_B_minus,Patent_Class_C,Patent_Class_B_minus,Patent_Class_B0,Patent_Class_B_plus,Patent_Class_A_minus,Patent_Class_A0,Patent_Class_A_plus,Patent_Class_AA,ROE,ROA,ratio_of_netincome_to_sales,equity_ratio,Debt_ratio,pre_2y_sales,pre_1y_sales,rate_of_sales_increase,pre_2y_opincome,pre_1y_opincome,rate_of_opincome_increase,pre_2y_netincome,pre_1y_netincome,rate_of_netincome_increase,pre_2y_assets,pre_1y_assets,rate_of_assets_increase,pre_1y_liabilities,pre_1y_equity
count,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000
mean,419.017966,14.118644,15.576271,10.389831,5.186441,3.881356,3.203390,1.542373,0.644068,0.457627,0.542373,0.118644,1.346390,-0.964755,-95.845821,-0.581591,-1.733153,475.507542,757.593749,19.400139,-124.712608,-155.524839,-0.660939,-421.958741,-287.793151,-0.924317,427.648660,596.034398,0.764865,524.838811,71.195417
std,616.967818,11.178928,19.737242,13.152969,8.939402,6.139428,4.016259,2.416338,1.471133,0.952710,1.134446,0.375304,9.557753,0.958803,340.412121,1.585870,9.941032,2152.058769,3123.991903,78.367155,319.947333,345.879142,0.993287,1691.445587,505.399477,2.582428,1010.525430,1375.582431,1.440290,1020.129290,598.886730
min,15.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-6.949700,-3.331045,-1846.829097,-3.963785,-72.612013,0.000000,0.000000,-1.000000,-2138.522470,-2289.045550,-4.325285,-12766.298670,-2408.855890,-10.357633,28.306120,40.598540,-0.616695,7.472300,-996.366580
25%,94.500000,5.500000,2.000000,0.500000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.865921,-1.328805,-22.414486,-1.250328,-1.671155,0.240000,1.198000,0.000000,-76.573450,-115.551975,-1.123527,-184.722345,-178.220400,-1.489370,84.488980,116.497240,-0.285557,71.406935,-167.435740
50%,200.000000,13.000000,8.000000,5.000000,2.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.042659,-0.630126,-4.243369,0.158731,0.043864,8.871200,15.167680,0.868995,-51.289370,-75.784440,-0.484823,-100.285930,-112.353540,-0.543469,148.897800,174.405080,0.179280,220.233850,15.255150
75%,397.500000,21.500000,20.000000,13.000000,6.000000,6.000000,6.000000,2.000000,1.000000,0.000000,1.000000,0.000000,0.762449,-0.323597,-0.969261,0.567951,0.834232,35.136100,54.029440,1.726037,-32.945585,-51.735270,-0.115193,-43.240140,-74.618385,0.137929,268.815755,366.946050,1.294567,546.590160,133.778205
max,3000.000000,62.000000,110.000000,50.000000,60.000000,38.000000,17.000000,9.000000,9.000000,3.000000,5.000000,2.000000,70.154061,0.661959,568.738672,0.984858,10.596018,15579.552190,20335.945050,354.794084,7.145270,25.700000,1.438567,163.715230,150.624750,8.215472,6648.878490,7298.140060,4.521225,5427.923220,2955.730710
skew,2.657898,1.369863,2.408576,1.596495,4.359828,3.423439,1.350014,1.691972,3.742591,1.920808,2.605233,3.373837,6.667829,-0.943504,-3.503176,-1.082852,-6.450553,6.354965,5.165835,4.183917,-5.172834,-4.974236,-0.910648,-6.985449,-3.085595,-1.203187,4.787856,3.835801,1.449320,3.683313,2.534824
kurtosis,7.151531,4.286062,8.160738,1.845845,24.460730,16.083822,1.322291,1.769358,17.818334,2.266979,7.052510,11.830501,48.267346,0.176839,14.365302,-0.299642,46.356357,43.599387,28.693250,16.086723,29.031571,26.979769,2.305044,51.154253,9.676862,7.297989,25.947408,14.423813,1.160002,13.813094,9.972584


In [51]:
desc.to_excel('C:/플랫폼 SME 연구센터/투자여부예측/스타트업기준_기술통계량_Pre-IPO_250402.xlsx')

In [14]:
# describe() 출력
desc = stup_1.describe()

# 왜도와 첨도 추가
skewness = stup_1.skew()
kurtosis = stup_1.kurt()

# 기술통계량에 왜도와 첨도 추가
desc.loc['skew'] = skewness
desc.loc['kurtosis'] = kurtosis
desc

,inv_amounts,inv_dur,Patent_total,Patent_above_B_minus,ROE,ROA,ratio_of_netincome_to_sales,equity_ratio,Debt_ratio,pre_1y_sales,rate_of_sales_increase,pre_1y_opincome,rate_of_opincome_increase,pre_1y_netincome,rate_of_netincome_increase,pre_1y_assets,rate_of_assets_increase,pre_1y_liabilities,pre_1y_equity
count,837.000000,837.000000,837.000000,837.000000,837.000000,837.000000,837.000000,837.000000,837.000000,837.000000,837.000000,837.000000,837.000000,837.000000,837.000000,837.000000,837.000000,837.000000,837.000000
mean,297.345871,13.346476,6.669056,4.593787,-1.889797,-0.677456,-82.723127,0.237638,1.683711,298.260515,23.423671,10.835355,-7.598633,-60.608348,-11.047946,1119.550437,8.316238,915.980076,203.570223
std,698.235252,13.097697,14.139400,12.192627,25.265784,1.019062,1320.659818,1.145308,24.563223,2047.665322,352.091360,1614.710680,60.057163,1091.463426,123.305292,10853.480551,98.565348,9837.943814,1717.758021
min,0.999380,0.000000,0.000000,0.000000,-665.952189,-10.199877,-37595.835000,-10.277442,-72.612013,0.000000,-1.000000,-2448.959410,-887.574790,-2644.466880,-3348.441614,0.032130,-0.869045,0.001070,-2752.322130
25%,50.000000,5.000000,0.000000,0.000000,-1.061627,-0.838722,-4.705241,0.168551,0.046215,1.620980,0.068738,-78.420220,-2.055210,-105.021850,-2.357500,36.630320,-0.069581,7.297210,6.113720
50%,130.000000,12.000000,2.000000,1.000000,-0.428621,-0.408090,-1.000000,0.567123,0.336461,15.562490,0.850867,-31.700000,-0.800777,-34.594860,-0.948793,84.102790,0.592925,28.865680,36.100270
75%,260.000000,19.000000,7.000000,4.000000,0.032144,-0.177844,-0.343627,0.828181,1.064743,82.252640,1.864552,-10.200840,-0.189565,-8.131880,-0.149996,205.918410,2.348952,146.732290,100.400460
max,12500.000000,234.000000,147.000000,146.000000,70.154061,2.335512,568.738672,0.999049,676.970277,36854.901100,9534.444674,32747.266970,72.043464,21295.759960,132.552910,233983.638880,1959.135800,224351.600810,32505.024950
skew,9.099569,6.315221,5.831984,7.877627,-22.766656,-4.053933,-27.569416,-3.756929,24.900839,15.103432,24.375994,19.973357,-12.318691,17.825163,-24.250325,16.529992,18.899048,18.235003,16.335810
kurtosis,125.506476,96.222464,46.778418,81.373144,579.712087,25.857584,781.609703,18.909701,685.614402,254.100209,643.330810,404.090459,169.140492,350.468732,646.107995,305.892999,368.609165,370.182956,299.966910


In [15]:
desc.to_excel('C:/플랫폼 SME 연구센터/투자여부예측/스타트업기준_기술통계량_전체_250418.xlsx')

In [20]:
stup_1 = stup_h3[conti_var]
stup_1

,inv_amounts,inv_dur,Patent_total,Patent_above_B_minus,Patent_Class_C,Patent_Class_B_minus,Patent_Class_B0,Patent_Class_B_plus,Patent_Class_A_minus,Patent_Class_A0,...,pre_1y_opincome,rate_of_opincome_increase,pre_2y_netincome,pre_1y_netincome,rate_of_netincome_increase,pre_2y_assets,pre_1y_assets,rate_of_assets_increase,pre_1y_liabilities,pre_1y_equity
1,140.0,6,1,1,0,1,0,0,0,0,...,-4.03019,-2.736640,-0.63027,0.52070,1.826154,7.79213,33.98093,3.360930,26.29042,7.69051
2,140.0,1,1,1,0,1,0,0,0,0,...,-38.33597,-8.512199,0.52070,-35.48415,-69.147014,33.98093,112.16258,2.300751,35.03557,77.12701
3,110.0,7,2,0,2,0,0,0,0,0,...,-27.93240,-0.882821,-11.88098,-19.68655,-0.656980,45.95514,65.26454,0.420179,36.58277,28.68177
4,250.0,16,2,0,2,0,0,0,0,0,...,-50.75008,-0.816889,-19.68655,-70.84780,-2.598792,65.26454,118.18878,0.810919,44.82411,73.36467
5,70.0,1,1,1,0,0,1,0,0,0,...,-10.20084,-0.333642,-7.84905,-10.61750,-0.352711,14.86939,5.43983,-0.634159,5.03957,0.40026
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
832,65.0,11,15,13,2,6,7,0,0,0,...,-71.62066,-0.694481,-45.98936,-107.02011,-1.327062,44.57182,48.44581,0.086916,124.43365,-75.98784
833,72.0,9,15,13,2,6,7,0,0,0,...,-71.62066,-0.694481,-45.98936,-107.02011,-1.327062,44.57182,48.44581,0.086916,124.43365,-75.98784
834,72.0,1,17,13,4,6,7,0,0,0,...,-59.00817,0.176101,-107.02011,-105.02185,0.018672,48.44581,91.04655,0.879348,220.23385,-129.18730
835,22.0,25,19,13,6,6,7,0,0,0,...,-75.78444,-0.166007,-128.91379,-69.99527,0.457038,84.48898,60.36252,-0.285557,45.10737,15.25515


In [16]:
# describe() 출력
desc = stup_1.describe()

# 왜도와 첨도 추가
skewness = stup_1.skew()
kurtosis = stup_1.kurt()

# 기술통계량에 왜도와 첨도 추가
desc.loc['skew'] = skewness
desc.loc['kurtosis'] = kurtosis
desc

,inv_amounts,inv_dur,Patent_total,Patent_above_B_minus,Patent_Class_C,Patent_Class_B_minus,Patent_Class_B0,Patent_Class_B_plus,Patent_Class_A_minus,Patent_Class_A0,...,pre_1y_opincome,rate_of_opincome_increase,pre_2y_netincome,pre_1y_netincome,rate_of_netincome_increase,pre_2y_assets,pre_1y_assets,rate_of_assets_increase,pre_1y_liabilities,pre_1y_equity
count,610.000000,610.000000,610.000000,610.000000,610.000000,610.000000,610.000000,610.000000,610.000000,610.000000,...,610.000000,610.000000,610.000000,610.000000,610.000000,610.000000,610.000000,610.000000,610.000000,610.000000
mean,378.708118,13.991803,8.004918,5.709836,2.295082,1.598361,1.868852,0.863934,0.573770,0.296721,...,21.649206,-2.130344,-96.546624,-76.820399,-8.373159,609.939901,1519.138065,2.837923,1252.671773,266.466103
std,799.879450,11.017448,16.063860,13.989067,4.655079,3.237861,5.307848,2.030840,2.364620,1.333719,...,1891.612965,21.601402,551.801686,1278.204442,136.065657,5982.955590,12693.074091,26.737220,11508.357270,2008.422248
min,1.910000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-2448.959410,-397.345868,-12766.298670,-2644.466880,-3348.441614,0.032130,2.875000,-0.869045,0.091770,-2752.322130
25%,100.000000,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-93.356252,-1.505832,-69.958978,-140.756820,-2.031596,41.195465,58.683687,-0.140092,13.810065,9.271480
50%,171.750000,13.000000,3.000000,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-48.597835,-0.666083,-28.922830,-59.575555,-0.818034,95.833900,129.368910,0.405915,55.125690,54.607520
75%,325.000000,21.000000,9.000000,6.000000,2.000000,2.000000,2.000000,1.000000,0.000000,0.000000,...,-21.117785,-0.152180,-9.319410,-19.368370,-0.080164,181.935460,319.447267,1.587228,258.067460,131.443665
max,12500.000000,62.000000,147.000000,146.000000,60.000000,38.000000,57.000000,18.000000,27.000000,15.000000,...,32747.266970,72.043464,571.928910,21295.759960,36.234977,143485.220160,233983.638880,625.196939,224351.600810,32505.024950
skew,8.017987,1.068390,5.201453,6.925625,5.113508,4.391795,7.759868,4.799977,9.430546,9.094513,...,17.052072,-12.965108,-20.241350,15.287005,-24.373969,22.599079,14.104374,21.312642,15.563568,13.948193
kurtosis,96.345703,1.935577,36.283022,61.734710,44.565927,31.794939,73.066144,31.872972,101.066044,95.517640,...,294.103442,221.123793,458.994857,256.437526,599.120132,536.870112,222.482726,487.042263,269.461396,218.366555


In [17]:
desc.to_excel('C:/플랫폼 SME 연구센터/투자여부예측/스타트업기준_기술통계량_가설3_250310.xlsx')

In [220]:
invst_1 = invst_h4_1_a_conti.drop('sequantial_inv',axis=1)
#invst_1 = invst_1[invst_1['inv_dur'].notnull()]
invst_1

,Patent_total,Patent_above_B_minus,ROE,ROA,ratio_of_netincome_to_sales,equity_ratio,Debt_ratio,pre_1y_sales,rate_of_sales_increase,pre_1y_opincome,rate_of_opincome_increase,pre_1y_netincome,rate_of_netincome_increase,pre_1y_assets,rate_of_assets_increase,pre_1y_liabilities,pre_1y_equity
3,1,1,0.067707,0.015323,0.009139,0.226318,3.418554,56.97477,8.509473,-4.03019,-2.736640,0.52070,1.826154,33.98093,3.360930,26.29042,7.69051
4,0,0,-4.559630,-2.488347,-0.473917,0.545734,0.832393,44.97305,0.153301,-21.31652,-1.779929,-21.31349,-1.801900,8.56532,-0.712304,3.89093,4.67439
5,0,0,-4.559630,-2.488347,-0.473917,0.545734,0.832393,44.97305,0.153301,-21.31652,-1.779929,-21.31349,-1.801900,8.56532,-0.712304,3.89093,4.67439
7,2,1,-0.380925,-0.367148,-1.000000,0.963833,0.037524,0.00000,-1.000000,-23.13272,-12.697240,-23.29834,-17.741978,63.45761,18.468092,2.29505,61.16256
8,2,1,-0.380925,-0.367148,-1.000000,0.963833,0.037524,0.00000,-1.000000,-23.13272,-12.697240,-23.29834,-17.741978,63.45761,18.468092,2.29505,61.16256
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3739,3,2,-0.382241,-0.373907,-9.927318,0.978197,0.022289,11.59130,-0.726749,-117.14166,-3.798941,-115.07052,-3.823727,307.75206,2.195085,6.70987,301.04219
3741,9,1,0.591756,-0.746820,-50.258491,-1.262041,-1.792367,4.15093,1.000000,-131.61266,-0.508506,-208.61948,-0.269214,279.34359,0.810998,631.88669,-352.54310
3743,14,7,0.101778,0.096268,0.567899,0.945859,0.057240,161.12110,30.279091,32.92719,1.208316,91.50052,1.038099,950.47829,-0.029982,51.45970,899.01859
3744,3,1,0.604113,-3.045641,-1104.848752,-5.041507,-1.198353,0.13620,1.270000,-107.60543,-0.575996,-150.48040,0.039251,49.40845,-0.245605,298.50152,-249.09307


In [224]:
invst_1 = invst_h4_1_bc_conti
#invst_1 = invst_1[invst_1['inv_dur'].notnull()]
invst_1

,inv_amounts,inv_dur,Patent_total,Patent_above_B_minus,ROE,ROA,ratio_of_netincome_to_sales,equity_ratio,Debt_ratio,pre_1y_sales,rate_of_sales_increase,pre_1y_opincome,rate_of_opincome_increase,pre_1y_netincome,rate_of_netincome_increase,pre_1y_assets,rate_of_assets_increase,pre_1y_liabilities,pre_1y_equity
3,55.0,16.0,1,1,0.067707,0.015323,0.009139,0.226318,3.418554,56.97477,8.509473,-4.03019,-2.736640,0.52070,1.826154,33.98093,3.360930,26.29042,7.69051
4,30.0,19.0,0,0,-4.559630,-2.488347,-0.473917,0.545734,0.832393,44.97305,0.153301,-21.31652,-1.779929,-21.31349,-1.801900,8.56532,-0.712304,3.89093,4.67439
5,30.0,19.0,0,0,-4.559630,-2.488347,-0.473917,0.545734,0.832393,44.97305,0.153301,-21.31652,-1.779929,-21.31349,-1.801900,8.56532,-0.712304,3.89093,4.67439
7,100.0,8.0,2,1,-0.380925,-0.367148,-1.000000,0.963833,0.037524,0.00000,-1.000000,-23.13272,-12.697240,-23.29834,-17.741978,63.45761,18.468092,2.29505,61.16256
23,80.0,10.0,4,4,-0.690718,-0.624422,-3.632792,0.904019,0.106172,1.85086,0.290247,-12.74299,-5.863505,-6.72379,-5.911080,10.76803,5.448424,1.03353,9.73450
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3605,247.0,50.0,4,4,-11.028401,-0.729112,-3.116183,0.066112,14.125789,108.05086,0.473225,-206.47927,-0.727271,-336.70624,-0.429617,461.80288,-0.098923,431.27205,30.53083
3606,247.0,1.0,4,4,-11.028401,-0.729112,-3.116183,0.066112,14.125789,108.05086,0.473225,-206.47927,-0.727271,-336.70624,-0.429617,461.80288,-0.098923,431.27205,30.53083
3607,247.0,13.0,4,4,-11.028401,-0.729112,-3.116183,0.066112,14.125789,108.05086,0.473225,-206.47927,-0.727271,-336.70624,-0.429617,461.80288,-0.098923,431.27205,30.53083
3608,247.0,7.0,4,4,-11.028401,-0.729112,-3.116183,0.066112,14.125789,108.05086,0.473225,-206.47927,-0.727271,-336.70624,-0.429617,461.80288,-0.098923,431.27205,30.53083


In [225]:
# describe() 출력
desc = invst_1.describe()

# 왜도와 첨도 추가
skewness = invst_1.skew()
kurtosis = invst_1.kurt()

# 기술통계량에 왜도와 첨도 추가
desc.loc['skew'] = skewness
desc.loc['kurtosis'] = kurtosis
desc

,inv_amounts,inv_dur,Patent_total,Patent_above_B_minus,ROE,ROA,ratio_of_netincome_to_sales,equity_ratio,Debt_ratio,pre_1y_sales,rate_of_sales_increase,pre_1y_opincome,rate_of_opincome_increase,pre_1y_netincome,rate_of_netincome_increase,pre_1y_assets,rate_of_assets_increase,pre_1y_liabilities,pre_1y_equity
count,1222.000000,1222.000000,1222.000000,1222.000000,1222.000000,1222.000000,1222.000000,1222.000000,1222.000000,1222.000000,1222.000000,1222.000000,1222.000000,1222.000000,1222.000000,1222.000000,1222.000000,1222.000000,1222.000000
mean,372.078723,20.253682,5.809329,3.926350,-2.643564,-0.712075,-115.940149,0.223780,2.666565,199.344999,26.637036,-89.451386,-5.797510,-145.149279,-14.723911,1320.081925,18.698211,1175.839656,144.241934
std,719.724036,12.264267,11.384594,9.673599,34.269317,1.019567,1559.389979,1.227877,34.484440,1055.723767,313.900551,188.178248,42.555444,345.878175,168.930939,11657.862045,164.126438,11270.669696,705.381929
min,2.000000,0.000000,0.000000,0.000000,-665.952189,-10.199877,-37595.835000,-10.277442,-72.612013,0.000000,-1.000000,-2289.045550,-887.574790,-2636.383460,-3348.441614,0.319220,-0.869045,0.084640,-2752.322130
25%,102.000000,12.000000,0.000000,0.000000,-1.055109,-0.841040,-5.589284,0.170681,0.041997,0.910290,0.036227,-86.376150,-2.071762,-123.002173,-2.536683,48.385720,-0.117573,7.297210,8.180515
50%,182.500000,18.000000,2.000000,1.000000,-0.460980,-0.420553,-1.024996,0.623861,0.298619,15.764470,0.949276,-39.671480,-0.887979,-48.543115,-1.119060,106.529570,0.564139,34.162280,47.251870
75%,310.000000,25.000000,7.000000,4.000000,-0.069055,-0.236397,-0.530959,0.849630,0.834232,95.105473,2.028009,-17.720780,-0.339384,-16.414813,-0.287519,221.839920,2.402489,183.985120,115.279230
max,12500.000000,113.000000,147.000000,146.000000,70.154061,0.661959,568.738672,0.999049,676.970277,20335.945050,8542.842159,927.983330,36.512675,571.928910,25.078212,143485.220160,1959.135800,139061.472590,6013.018840
skew,7.029349,1.908856,6.044796,8.805835,-18.116376,-4.042688,-22.918301,-3.424886,18.404671,13.965332,19.533814,-5.216149,-16.458445,-4.820685,-19.031010,11.942606,11.192056,12.046410,5.659533
kurtosis,81.102608,6.961932,58.791604,113.795917,345.116315,23.886212,546.748429,14.665031,358.062450,235.840677,469.082047,41.640585,314.822208,26.154686,371.933445,142.748662,127.415240,144.589134,41.561804


In [226]:
desc.to_excel('C:/플랫폼 SME 연구센터/투자여부예측/투자자기준_기술통계량_가설2_a,b_250406.xlsx')

In [30]:
invst_1['Case_1'] = invst['Case_1']

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_26816\2826603167.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  invst_1['Case_1'] = invst['Case_1']


In [37]:
invst_F = invst_1[invst_1['Case_1']==0]
invst_A = invst_1[invst_1['Case_1']==1]
invst_B = invst_1[invst_1['Case_1']==2]
invst_C = invst_1[invst_1['Case_1']==3]
invst_D = invst_1[invst_1['Case_1']==4]

In [39]:
desc_F = invst_F.describe()
skewness = invst_F.skew()
kurtosis = invst_F.kurt()
desc_F.loc['skew'] = skewness
desc_F.loc['kurtosis'] = kurtosis
desc_F

,inv_amounts,inv_dur,Patent_total,Patent_above_B_minus,Patent_Class_C,Patent_Class_B_minus,Patent_Class_B0,Patent_Class_B_plus,Patent_Class_A_minus,Patent_Class_A0,Patent_Class_A_plus,Patent_Class_AA,ROE,ROA,ratio_of_netincome_to_sales,equity_ratio,Debt_ratio,pre_2y_sales,pre_1y_sales,rate_of_sales_increase,pre_2y_opincome,pre_1y_opincome,rate_of_opincome_increase,pre_2y_netincome,pre_1y_netincome,rate_of_netincome_increase,pre_2y_assets,pre_1y_assets,rate_of_assets_increase,pre_1y_liabilities,pre_1y_equity,Case_1
count,443.000000,0.0,443.000000,443.000000,443.000000,443.000000,443.000000,443.000000,443.000000,443.000000,443.000000,443.000000,443.000000,443.000000,443.000000,443.000000,443.000000,443.000000,443.000000,443.000000,443.000000,443.000000,443.000000,443.000000,443.000000,443.000000,443.000000,443.000000,443.000000,443.000000,443.000000,443.0
mean,199.322027,NaN,3.706546,2.568849,1.137698,0.697517,0.882619,0.408578,0.261851,0.124153,0.126411,0.067720,-2.131641,-0.700717,-442.733666,0.288154,2.644268,51.146444,100.236641,10.301058,-18.818727,-37.194886,-19.449404,-24.741352,-52.662917,-13.068645,94.961093,175.299367,16.313842,115.466576,59.832786,0.0
std,378.334361,NaN,9.317411,8.580290,2.148661,1.506747,3.397463,1.382705,1.494806,0.829561,0.862650,0.566741,31.853850,1.327612,3974.941871,1.102320,33.322285,262.269480,537.568254,133.496197,84.127157,113.813762,106.762610,138.309116,184.729051,52.670708,391.819605,819.343661,160.826680,579.513694,313.292674,0.0
min,5.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-665.952189,-10.199877,-37595.835000,-10.277442,-70.240598,0.000000,0.000000,-1.000000,-1154.236530,-1154.236530,-887.574790,-2389.403760,-2389.403760,-483.596769,0.077220,0.032130,-0.835265,0.001070,-584.816420,0.0
25%,50.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.182639,-0.763842,-3.229891,0.151957,0.062831,0.000000,0.556360,0.085935,-13.017355,-41.580971,-4.309744,-12.588525,-45.599005,-6.034719,7.050140,17.099750,0.034085,3.758240,2.032270,0.0
50%,100.000000,NaN,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.465339,-0.371679,-1.000000,0.545642,0.486125,3.193200,8.742590,1.000000,-5.619650,-14.897560,-1.436505,-4.463200,-14.450260,-1.494985,23.105790,41.497970,1.190439,12.768640,16.560720,0.0
75%,170.000000,NaN,4.000000,2.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.008946,-0.147127,-0.229366,0.794837,1.269182,17.219595,45.142250,2.088315,-1.112245,-5.215020,-0.429244,-0.630270,-3.617100,-0.366243,64.722315,108.983010,3.097538,41.794175,52.767655,0.0
max,4600.000000,NaN,145.000000,145.000000,17.000000,16.000000,57.000000,18.000000,27.000000,15.000000,15.000000,11.000000,19.551353,2.335512,1.514554,0.999049,676.970277,4255.160100,9508.621450,2762.016662,538.439460,927.983330,19.344597,216.832530,571.928910,132.552910,5417.722650,13812.634480,1959.135800,10190.383820,3622.250660,0.0
skew,5.880285,NaN,9.694589,11.727762,3.573364,4.741638,12.013003,6.945624,13.920528,13.976934,12.838394,16.642764,-20.570093,-4.799360,-9.277824,-5.599575,18.767244,11.583277,13.722698,19.981328,-9.044847,-4.575993,-7.550353,-13.324390,-8.843578,-6.143126,9.167698,12.204017,12.019148,13.308621,7.868149,0.0
kurtosis,49.551206,NaN,129.059192,178.157894,17.053984,34.314634,180.201268,68.133938,235.873280,237.750883,204.932632,315.375961,429.528915,28.474994,84.498268,43.022094,381.777298,163.251248,222.395216,411.170602,122.594618,62.924704,57.822477,208.523246,98.444531,42.522002,98.649225,182.514697,143.498731,214.380198,74.755389,0.0


In [40]:
desc_A = invst_A.describe()
skewness = invst_A.skew()
kurtosis = invst_A.kurt()
desc_A.loc['skew'] = skewness
desc_A.loc['kurtosis'] = kurtosis
desc_A

,inv_amounts,inv_dur,Patent_total,Patent_above_B_minus,Patent_Class_C,Patent_Class_B_minus,Patent_Class_B0,Patent_Class_B_plus,Patent_Class_A_minus,Patent_Class_A0,Patent_Class_A_plus,Patent_Class_AA,ROE,ROA,ratio_of_netincome_to_sales,equity_ratio,Debt_ratio,pre_2y_sales,pre_1y_sales,rate_of_sales_increase,pre_2y_opincome,pre_1y_opincome,rate_of_opincome_increase,pre_2y_netincome,pre_1y_netincome,rate_of_netincome_increase,pre_2y_assets,pre_1y_assets,rate_of_assets_increase,pre_1y_liabilities,pre_1y_equity,Case_1
count,210.000000,210.000000,210.000000,210.000000,210.000000,210.000000,210.000000,210.000000,210.000000,210.000000,210.000000,210.000000,210.000000,210.000000,210.000000,210.000000,210.000000,210.000000,210.000000,210.000000,210.000000,210.000000,210.000000,210.000000,210.000000,210.000000,210.000000,210.000000,210.000000,210.000000,210.000000,210.0
mean,424.968157,22.019048,5.585714,3.852381,1.733333,1.471429,1.323810,0.452381,0.252381,0.128571,0.180952,0.042857,-6.902416,-0.780757,-89.997664,0.242741,7.353325,204.108443,340.775321,29.629004,-87.283610,-132.183575,-3.414176,-172.657737,-219.846197,-3.886493,383.269623,585.895471,1.582114,364.817486,221.077976,1.0
std,669.634082,12.893365,9.971960,7.258199,3.780501,2.840521,2.386145,1.084964,0.662111,0.486805,0.792077,0.225358,65.253380,1.100088,553.710934,1.045014,66.530842,1143.334942,1602.856538,268.729982,210.547065,243.199882,19.276094,910.540689,483.020072,16.413236,945.669869,1526.213757,3.448564,891.347116,949.641828,0.0
min,15.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-665.952189,-10.066646,-5353.353870,-5.078482,-72.612013,0.000000,0.000000,-1.000000,-2138.522470,-2289.045550,-265.534331,-12766.298670,-2636.383460,-197.398343,1.630110,2.347830,-0.869045,0.420440,-2752.322130,1.0
25%,134.750000,13.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.260422,-0.862059,-5.176529,0.208813,0.046830,0.412360,1.975900,0.296660,-79.550220,-138.876240,-2.068136,-79.544170,-164.527103,-2.190657,45.419663,64.593173,-0.231956,13.445472,10.330305,1.0
50%,210.000000,21.000000,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.518945,-0.446361,-1.047803,0.586702,0.344683,8.616685,31.328620,1.000000,-30.710030,-57.401010,-0.874256,-31.387100,-71.643115,-1.123063,108.949080,142.861620,0.384148,58.072955,69.125910,1.0
75%,350.000000,26.750000,7.000000,4.000000,2.000000,2.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.169092,-0.242138,-0.528514,0.801029,0.887976,79.991290,183.565918,2.541943,-12.516557,-28.006710,-0.394501,-11.406987,-31.506050,-0.317657,224.286000,328.260710,1.926296,237.907475,143.577020,1.0
max,4600.000000,89.000000,57.000000,41.000000,31.000000,17.000000,13.000000,7.000000,4.000000,3.000000,5.000000,2.000000,70.154061,0.661959,1.000000,0.992460,676.970277,15579.552190,20335.945050,2762.016662,17.061820,109.589750,36.512675,7.702350,100.552770,23.004978,6648.878490,9233.998520,25.262272,5427.923220,6013.018840,1.0
skew,3.778776,1.607963,3.306719,3.260243,4.998242,3.048215,2.657098,3.765983,3.266106,4.331866,4.914640,5.748189,-9.944597,-4.573165,-8.723613,-2.703889,9.842780,12.024144,10.390543,10.154810,-6.330487,-4.994802,-12.191921,-12.879096,-3.744992,-8.670727,4.305932,4.428911,3.935979,3.735508,4.366326,0.0
kurtosis,16.620323,4.291504,12.241611,11.703614,32.337624,10.807466,8.008994,17.980660,11.483885,19.650557,24.403940,36.184383,99.442097,29.662459,79.211327,7.548463,98.028292,158.859420,121.699227,102.214925,50.057696,33.780542,165.542766,177.350628,13.758170,94.890200,19.676723,20.557725,21.201374,14.537398,26.048059,0.0


In [41]:
desc_B = invst_B.describe()
skewness = invst_B.skew()
kurtosis = invst_B.kurt()
desc_B.loc['skew'] = skewness
desc_B.loc['kurtosis'] = kurtosis
desc_B

,inv_amounts,inv_dur,Patent_total,Patent_above_B_minus,Patent_Class_C,Patent_Class_B_minus,Patent_Class_B0,Patent_Class_B_plus,Patent_Class_A_minus,Patent_Class_A0,Patent_Class_A_plus,Patent_Class_AA,ROE,ROA,ratio_of_netincome_to_sales,equity_ratio,Debt_ratio,pre_2y_sales,pre_1y_sales,rate_of_sales_increase,pre_2y_opincome,pre_1y_opincome,rate_of_opincome_increase,pre_2y_netincome,pre_1y_netincome,rate_of_netincome_increase,pre_2y_assets,pre_1y_assets,rate_of_assets_increase,pre_1y_liabilities,pre_1y_equity,Case_1
count,0.0,0.0,134.000000,134.000000,134.000000,134.000000,134.000000,134.000000,134.000000,134.000000,134.000000,134.000000,134.000000,134.000000,134.000000,134.000000,134.000000,134.000000,134.000000,134.000000,134.000000,134.000000,134.000000,134.000000,134.000000,134.000000,134.000000,134.000000,134.000000,134.000000,134.000000,134.0
mean,NaN,NaN,11.529851,7.380597,4.149254,2.179104,2.462687,1.320896,0.626866,0.253731,0.380597,0.156716,-2.963436,-0.770356,-162.826424,-0.350403,4.248057,246.632904,280.237970,2.598642,-123.026351,-101.353354,-0.457440,-207.112368,-150.843000,-1.768401,451.803221,469.948271,0.484453,359.359476,110.588025,2.0
std,NaN,NaN,16.860636,12.527140,6.288201,3.894957,4.919895,2.318934,1.301657,0.690282,0.940283,0.532472,16.483176,3.155641,1191.188370,3.135545,26.263939,507.317963,576.340308,11.134268,155.410621,117.461397,2.569273,437.151980,291.796812,14.982542,699.886229,675.019205,1.640452,598.611477,388.496526,0.0
min,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-126.067640,-35.203721,-13191.489362,-31.167050,-11.289070,0.000000,0.000000,-1.000000,-860.608510,-616.201150,-26.113023,-2636.383460,-2319.419950,-171.884427,9.542080,5.860870,-0.978948,1.761590,-1132.764690,2.0
25%,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.182702,-0.746597,-3.169669,-0.117879,-1.079441,6.973205,13.452312,0.000000,-152.827715,-132.917550,-0.510928,-171.450075,-202.928895,-0.750934,102.514750,122.552910,-0.305822,48.281320,-17.515070,2.0
50%,NaN,NaN,5.000000,3.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.500542,-0.370506,-0.960853,0.462564,0.483771,43.064640,60.304005,0.428170,-70.341325,-67.287150,-0.003885,-86.346665,-84.639105,-0.005221,214.952260,234.877490,-0.029982,136.506185,83.177306,2.0
75%,NaN,NaN,16.000000,8.000000,5.750000,3.000000,3.000000,2.000000,1.000000,0.000000,0.000000,0.000000,0.001051,-0.167269,-0.351977,0.654762,1.014494,155.176930,229.311157,0.996340,-30.888785,-34.121517,0.428309,-28.528807,-33.832853,0.435163,487.501472,466.227995,0.753578,404.725940,243.537530,2.0
max,NaN,NaN,92.000000,69.000000,24.000000,19.000000,36.000000,11.000000,6.000000,4.000000,6.000000,3.000000,11.222485,2.622239,168.634387,0.983158,204.147458,2563.639440,3038.782060,117.179697,89.992730,203.206340,3.393041,405.091060,389.999060,3.409259,4531.703730,5255.388724,10.128797,4646.470662,1724.075110,2.0
skew,NaN,NaN,2.310414,2.655457,1.789860,2.539938,4.024931,2.253744,2.343765,3.517365,3.189118,3.476136,-6.469971,-9.903321,-10.150355,-7.653177,7.040563,2.806331,3.403109,8.658999,-2.387991,-1.883418,-7.872542,-3.916117,-3.683403,-11.179055,3.774920,3.848417,3.856725,3.982899,0.611507,0.0
kurtosis,NaN,NaN,6.241369,7.797337,2.320192,6.361541,20.964781,4.934664,5.107161,14.015409,11.855751,11.438396,41.841337,108.534545,109.869251,71.459570,50.846037,7.449213,12.371323,85.969329,6.753193,5.818048,75.961801,16.951155,24.027827,127.672402,16.540532,20.563135,18.510436,21.578394,3.857969,0.0


In [42]:
desc_C = invst_C.describe()
skewness = invst_C.skew()
kurtosis = invst_C.kurt()
desc_C.loc['skew'] = skewness
desc_C.loc['kurtosis'] = kurtosis
desc_C

,inv_amounts,inv_dur,Patent_total,Patent_above_B_minus,Patent_Class_C,Patent_Class_B_minus,Patent_Class_B0,Patent_Class_B_plus,Patent_Class_A_minus,Patent_Class_A0,Patent_Class_A_plus,Patent_Class_AA,ROE,ROA,ratio_of_netincome_to_sales,equity_ratio,Debt_ratio,pre_2y_sales,pre_1y_sales,rate_of_sales_increase,pre_2y_opincome,pre_1y_opincome,rate_of_opincome_increase,pre_2y_netincome,pre_1y_netincome,rate_of_netincome_increase,pre_2y_assets,pre_1y_assets,rate_of_assets_increase,pre_1y_liabilities,pre_1y_equity,Case_1
count,1012.000000,1012.000000,1012.000000,1012.000000,1012.000000,1012.000000,1012.000000,1012.000000,1012.000000,1012.000000,1012.000000,1012.000000,1012.000000,1012.000000,1012.000000,1012.000000,1012.000000,1012.000000,1012.000000,1012.000000,1012.000000,1012.000000,1012.000000,1012.000000,1012.000000,1012.000000,1012.000000,1012.000000,1012.000000,1012.000000,1012.000000,1012.0
mean,361.103643,19.887352,5.855731,3.941700,1.914032,1.216403,1.252964,0.616601,0.353755,0.196640,0.218379,0.086957,-1.759810,-0.697823,-121.323471,0.219845,1.694016,95.339328,169.996810,26.016173,-46.344805,-80.584034,-6.292075,-73.649806,-129.648930,-16.972783,199.623260,1472.432869,22.249970,1344.134770,128.297696,3.0
std,729.520461,12.104031,11.660247,10.105652,3.534977,2.460290,3.720082,1.636394,1.678206,0.965205,1.042489,0.638640,23.098364,1.002037,1695.065564,1.262935,22.707596,578.446266,899.963990,322.597793,122.634819,173.439921,45.922689,436.308431,307.978316,185.419031,939.432072,12787.442933,180.157911,12372.728533,642.654514,0.0
min,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-665.952189,-10.199877,-37595.835000,-10.277442,-70.240598,-0.356640,0.000000,-1.000000,-2138.522470,-2289.045550,-887.574790,-12766.298670,-2636.383460,-3348.441614,0.032130,0.319220,-0.869045,0.084640,-2752.322130,3.0
25%,100.000000,12.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.008992,-0.828611,-5.604933,0.170681,0.041236,0.247480,0.747718,0.000000,-46.743592,-81.023057,-2.071762,-51.733390,-105.483820,-2.696009,20.977440,45.678123,-0.098923,7.033370,7.642180,3.0
50%,175.000000,18.000000,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.445791,-0.407116,-1.024996,0.632290,0.286200,6.440180,14.041230,0.933827,-18.561070,-35.861050,-0.887979,-17.602830,-42.965910,-1.104989,61.755410,96.986040,0.596232,29.208785,43.556440,3.0
75%,300.000000,25.000000,7.000000,4.000000,2.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,-0.066410,-0.233232,-0.530959,0.860064,0.833993,32.142590,82.890500,1.836802,-5.251840,-14.897560,-0.339384,-4.501610,-13.529250,-0.273144,141.583590,206.162802,2.518420,163.634630,111.600000,3.0
max,12500.000000,113.000000,147.000000,146.000000,31.000000,21.000000,57.000000,18.000000,27.000000,15.000000,16.000000,11.000000,32.446675,0.661959,568.738672,0.999049,676.970277,15579.552190,20335.945050,8542.842159,538.439460,927.983330,36.512675,216.832530,571.928910,25.078212,25586.300000,143485.220160,1959.135800,139061.472590,6013.018840,3.0
skew,7.562398,1.987730,6.369510,9.081514,3.016851,3.815179,10.629517,5.410305,12.569370,11.369647,10.161873,15.013102,-24.675458,-3.894772,-21.486633,-3.485344,26.111476,20.363806,14.979929,20.539080,-8.692452,-5.073062,-15.611192,-24.943723,-5.063229,-17.350198,20.728681,10.895625,10.164383,10.959852,6.062933,0.0
kurtosis,90.777455,7.758444,63.162854,114.715323,11.364144,18.959919,149.543207,42.332194,189.556425,164.020421,133.816321,251.802476,686.443986,22.111273,472.579347,15.052549,775.603366,516.469699,284.421731,498.491580,111.527353,41.103005,278.639229,712.189559,30.472997,308.457178,533.873152,118.191957,104.791918,119.254166,46.340165,0.0


In [43]:
desc_D = invst_D.describe()
skewness = invst_D.skew()
kurtosis = invst_D.kurt()
desc_D.loc['skew'] = skewness
desc_D.loc['kurtosis'] = kurtosis
desc_D

,inv_amounts,inv_dur,Patent_total,Patent_above_B_minus,Patent_Class_C,Patent_Class_B_minus,Patent_Class_B0,Patent_Class_B_plus,Patent_Class_A_minus,Patent_Class_A0,Patent_Class_A_plus,Patent_Class_AA,ROE,ROA,ratio_of_netincome_to_sales,equity_ratio,Debt_ratio,pre_2y_sales,pre_1y_sales,rate_of_sales_increase,pre_2y_opincome,pre_1y_opincome,rate_of_opincome_increase,pre_2y_netincome,pre_1y_netincome,rate_of_netincome_increase,pre_2y_assets,pre_1y_assets,rate_of_assets_increase,pre_1y_liabilities,pre_1y_equity,Case_1
count,1947.000000,0.0,1947.000000,1947.000000,1947.000000,1947.000000,1947.000000,1947.000000,1947.000000,1947.000000,1947.000000,1947.000000,1947.000000,1947.000000,1947.000000,1947.000000,1947.000000,1947.000000,1947.000000,1947.000000,1947.000000,1947.000000,1947.000000,1947.000000,1947.000000,1947.000000,1947.000000,1947.000000,1947.000000,1947.000000,1947.000000,1947.0
mean,339.792119,NaN,6.547509,4.330765,2.216744,1.379045,1.335901,0.700565,0.422188,0.197227,0.221880,0.073960,-1.959209,-0.703276,-195.458144,0.188705,2.018223,82.092788,184.214877,48.355241,-34.655612,-27.011196,-5.172995,-51.608051,-77.811494,-7.013046,508.633319,997.077075,13.900523,842.754794,154.322065,4.0
std,886.483442,NaN,11.493024,9.195630,3.953126,2.914530,3.321604,1.598501,1.474163,0.805386,0.954286,0.490060,27.125335,1.047475,2425.648867,1.283535,27.209497,283.895853,1296.931942,564.529408,98.946388,1062.159495,43.126345,136.711939,737.008472,81.715676,5863.706475,10613.794304,139.983381,9881.571531,1272.614122,0.0
min,0.999380,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-665.952189,-10.199877,-37595.835000,-10.277442,-72.612013,-0.356640,0.000000,-1.000000,-1154.236530,-2448.959410,-887.574790,-2389.403760,-2644.466880,-3348.441614,0.032130,0.032130,-0.869045,0.001070,-2752.322130,4.0
25%,100.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.101980,-0.854074,-4.996485,0.165449,0.038055,0.203970,1.000000,0.000000,-44.155830,-72.824980,-1.833022,-48.930160,-101.062530,-2.687341,22.436825,40.092800,-0.132213,7.068110,8.157240,4.0
50%,170.100000,NaN,2.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.409389,-0.397146,-1.000000,0.575573,0.329444,5.901270,13.961640,0.806721,-16.897330,-33.177690,-0.784928,-17.381030,-38.357930,-0.962038,63.457610,86.400200,0.524161,29.234100,35.715040,4.0
75%,300.000000,NaN,8.000000,5.000000,3.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.011882,-0.191017,-0.414125,0.846227,1.029155,35.349570,73.628130,1.747492,-4.761280,-12.910840,-0.211340,-3.296290,-10.829810,-0.141317,132.132070,174.405080,2.172699,131.447300,96.107030,4.0
max,12500.000000,NaN,145.000000,145.000000,60.000000,38.000000,57.000000,18.000000,27.000000,15.000000,15.000000,11.000000,70.154061,2.335512,568.738672,0.999049,676.970277,4255.160100,36854.901100,8542.842159,927.983330,32747.266970,72.043464,571.928910,21295.759960,132.552910,143485.220160,233983.638880,1959.135800,224351.600810,32505.024950,4.0
skew,10.352783,NaN,4.740144,6.955793,4.050474,4.169276,8.808720,4.193061,10.503149,10.817819,7.777394,16.405636,-22.981988,-3.912057,-15.115255,-3.794487,23.560073,6.310950,23.905925,14.268750,-4.335664,30.202169,-15.541841,-6.363987,24.773700,-35.904842,22.670248,18.677933,13.008940,19.804251,18.357212,0.0
kurtosis,130.047273,NaN,38.314395,83.951432,32.207610,26.371843,122.100337,26.503364,166.959239,177.771748,89.571889,341.147623,554.735821,23.507050,230.125509,19.822919,582.607517,51.960791,659.336748,209.449752,64.403510,931.588661,287.485829,65.871724,727.118268,1443.975359,544.857096,382.009306,173.561059,424.576056,435.824259,0.0


In [47]:
desc_concat = pd.concat([desc_F,desc_A,desc_B,desc_C,desc_D])
desc_concat

,inv_amounts,inv_dur,Patent_total,Patent_above_B_minus,Patent_Class_C,Patent_Class_B_minus,Patent_Class_B0,Patent_Class_B_plus,Patent_Class_A_minus,Patent_Class_A0,Patent_Class_A_plus,Patent_Class_AA,ROE,ROA,ratio_of_netincome_to_sales,equity_ratio,Debt_ratio,pre_2y_sales,pre_1y_sales,rate_of_sales_increase,pre_2y_opincome,pre_1y_opincome,rate_of_opincome_increase,pre_2y_netincome,pre_1y_netincome,rate_of_netincome_increase,pre_2y_assets,pre_1y_assets,rate_of_assets_increase,pre_1y_liabilities,pre_1y_equity,Case_1
count,443.000000,0.000000,443.000000,443.000000,443.000000,443.000000,443.000000,443.000000,443.000000,443.000000,443.000000,443.000000,443.000000,443.000000,443.000000,443.000000,443.000000,443.000000,443.000000,443.000000,443.000000,443.000000,443.000000,443.000000,443.000000,443.000000,443.000000,443.000000,443.000000,443.000000,443.000000,443.0
mean,199.322027,NaN,3.706546,2.568849,1.137698,0.697517,0.882619,0.408578,0.261851,0.124153,0.126411,0.067720,-2.131641,-0.700717,-442.733666,0.288154,2.644268,51.146444,100.236641,10.301058,-18.818727,-37.194886,-19.449404,-24.741352,-52.662917,-13.068645,94.961093,175.299367,16.313842,115.466576,59.832786,0.0
std,378.334361,NaN,9.317411,8.580290,2.148661,1.506747,3.397463,1.382705,1.494806,0.829561,0.862650,0.566741,31.853850,1.327612,3974.941871,1.102320,33.322285,262.269480,537.568254,133.496197,84.127157,113.813762,106.762610,138.309116,184.729051,52.670708,391.819605,819.343661,160.826680,579.513694,313.292674,0.0
min,5.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-665.952189,-10.199877,-37595.835000,-10.277442,-70.240598,0.000000,0.000000,-1.000000,-1154.236530,-1154.236530,-887.574790,-2389.403760,-2389.403760,-483.596769,0.077220,0.032130,-0.835265,0.001070,-584.816420,0.0
25%,50.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.182639,-0.763842,-3.229891,0.151957,0.062831,0.000000,0.556360,0.085935,-13.017355,-41.580971,-4.309744,-12.588525,-45.599005,-6.034719,7.050140,17.099750,0.034085,3.758240,2.032270,0.0
50%,100.000000,NaN,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.465339,-0.371679,-1.000000,0.545642,0.486125,3.193200,8.742590,1.000000,-5.619650,-14.897560,-1.436505,-4.463200,-14.450260,-1.494985,23.105790,41.497970,1.190439,12.768640,16.560720,0.0
75%,170.000000,NaN,4.000000,2.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.008946,-0.147127,-0.229366,0.794837,1.269182,17.219595,45.142250,2.088315,-1.112245,-5.215020,-0.429244,-0.630270,-3.617100,-0.366243,64.722315,108.983010,3.097538,41.794175,52.767655,0.0
max,4600.000000,NaN,145.000000,145.000000,17.000000,16.000000,57.000000,18.000000,27.000000,15.000000,15.000000,11.000000,19.551353,2.335512,1.514554,0.999049,676.970277,4255.160100,9508.621450,2762.016662,538.439460,927.983330,19.344597,216.832530,571.928910,132.552910,5417.722650,13812.634480,1959.135800,10190.383820,3622.250660,0.0
skew,5.880285,NaN,9.694589,11.727762,3.573364,4.741638,12.013003,6.945624,13.920528,13.976934,12.838394,16.642764,-20.570093,-4.799360,-9.277824,-5.599575,18.767244,11.583277,13.722698,19.981328,-9.044847,-4.575993,-7.550353,-13.324390,-8.843578,-6.143126,9.167698,12.204017,12.019148,13.308621,7.868149,0.0
kurtosis,49.551206,NaN,129.059192,178.157894,17.053984,34.314634,180.201268,68.133938,235.873280,237.750883,204.932632,315.375961,429.528915,28.474994,84.498268,43.022094,381.777298,163.251248,222.395216,411.170602,122.594618,62.924704,57.822477,208.523246,98.444531,42.522002,98.649225,182.514697,143.498731,214.380198,74.755389,0.0


In [48]:
desc_concat.to_excel('C:/플랫폼 SME 연구센터/투자여부예측/투자자기준_기술통계량_케이스별_250304.xlsx')

## 정규화

In [86]:
#from sklearn.preprocessing import FunctionTransformer

#transformer = FunctionTransformer(np.log1p, validate=True) # 음수에는 적용 불가 -> null 값으로 대체됨
#df_transformed = transformer.transform(df[['A', 'B']])

#print(pd.DataFrame(df_transformed, columns=['A_log', 'B_log']))

In [16]:
from sklearn.preprocessing import FunctionTransformer

# Symlog 변환 함수 정의
def symlog(x):
    return np.sign(x) * np.log1p(np.abs(x))

# FunctionTransformer로 Symlog 변환 정의
symlog_transformer = FunctionTransformer(symlog, validate=False)

# DataFrame의 특정 열에 적용
#df_transformed = symlog_transformer.transform(df[['profit', 'revenue']])

# 변환된 DataFrame으로 저장
#df[['profit_symlog', 'revenue_symlog']] = df_transformed

### 스타트업

In [20]:
stup_1 = stup[conti_var]
stup_1

,inv_amounts,inv_dur,Patent_total,Patent_above_B_minus,ROE,ROA,ratio_of_netincome_to_sales,equity_ratio,Debt_ratio,pre_1y_sales,rate_of_sales_increase,pre_1y_opincome,rate_of_opincome_increase,pre_1y_netincome,rate_of_netincome_increase,pre_1y_assets,rate_of_assets_increase,pre_1y_liabilities,pre_1y_equity
0,55.0,16,1,1,0.067707,0.015323,0.009139,0.226318,3.418554,56.97477,8.509473,-4.03019,-2.736640,0.52070,1.826154,33.98093,3.360930,26.29042,7.69051
1,140.0,6,1,1,0.067707,0.015323,0.009139,0.226318,3.418554,56.97477,8.509473,-4.03019,-2.736640,0.52070,1.826154,33.98093,3.360930,26.29042,7.69051
2,140.0,1,1,1,-0.460074,-0.316364,-0.330277,0.687636,0.454258,107.43747,0.885703,-38.33597,-8.512199,-35.48415,-69.147014,112.16258,2.300751,35.03557,77.12701
3,110.0,7,2,0,-0.686378,-0.301642,-0.270323,0.439469,1.275471,72.82602,0.640748,-27.93240,-0.882821,-19.68655,-0.656980,65.26454,0.420179,36.58277,28.68177
4,250.0,16,2,0,-0.965694,-0.599446,-0.660491,0.620741,0.610977,107.26536,0.472899,-50.75008,-0.816889,-70.84780,-2.598792,118.18878,0.810919,44.82411,73.36467
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
832,65.0,11,15,13,1.408385,-2.209068,-1.000000,-1.568512,-1.637547,0.00000,-1.000000,-71.62066,-0.694481,-107.02011,-1.327062,48.44581,0.086916,124.43365,-75.98784
833,72.0,9,15,13,1.408385,-2.209068,-1.000000,-1.568512,-1.637547,0.00000,-1.000000,-71.62066,-0.694481,-107.02011,-1.327062,48.44581,0.086916,124.43365,-75.98784
834,72.0,1,17,13,0.812943,-1.153496,-1.000000,-1.418915,-1.704764,0.00000,0.000000,-59.00817,0.176101,-105.02185,0.018672,91.04655,0.879348,220.23385,-129.18730
835,22.0,25,19,13,-4.588304,-1.159582,-1.000000,0.252726,2.956862,0.00000,-1.000000,-75.78444,-0.166007,-69.99527,0.457038,60.36252,-0.285557,45.10737,15.25515


In [17]:
stup_transformed = symlog_transformer.transform(stup_1)
stup_transformed

,inv_amounts,inv_dur,Patent_total,Patent_above_B_minus,ROE,ROA,ratio_of_netincome_to_sales,equity_ratio,Debt_ratio,pre_1y_sales,rate_of_sales_increase,pre_1y_opincome,rate_of_opincome_increase,pre_1y_netincome,rate_of_netincome_increase,pre_1y_assets,rate_of_assets_increase,pre_1y_liabilities,pre_1y_equity
0,4.025352,2.833213,0.693147,0.693147,0.065513,0.015207,0.009098,0.204017,1.485812,4.060008,2.252288,-1.615458,-1.318187,0.419171,1.038917,3.554803,1.472685,3.306536,2.162232
1,4.948760,1.945910,0.693147,0.693147,0.065513,0.015207,0.009098,0.204017,1.485812,4.060008,2.252288,-1.615458,-1.318187,0.419171,1.038917,3.554803,1.472685,3.306536,2.162232
2,4.948760,0.693147,0.693147,0.693147,-0.378487,-0.274873,-0.285387,0.523329,0.374496,4.686174,0.634300,-3.672139,-2.252575,-3.596878,-4.250593,4.728826,1.194150,3.584507,4.358336
3,4.709530,2.079442,1.098612,0.000000,-0.522583,-0.263627,-0.239271,0.364275,0.822187,4.301711,0.495152,-3.364962,-0.632771,-3.029484,-0.504997,4.193655,0.350783,3.626546,3.390533
4,5.525453,2.833213,1.098612,0.000000,-0.675845,-0.469657,-0.507113,0.482884,0.476841,4.684585,0.387232,-3.946426,-0.597126,-4.274550,-1.280598,4.780709,0.593834,3.824810,4.308981
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
832,4.189655,2.484907,2.772589,2.639057,0.878956,-1.165981,-0.693147,-0.943327,-0.969849,0.000000,-0.693147,-4.285249,-0.527376,-4.682317,-0.844607,3.900877,0.083344,4.831777,-4.343647
833,4.290459,2.302585,2.772589,2.639057,0.878956,-1.165981,-0.693147,-0.943327,-0.969849,0.000000,-0.693147,-4.285249,-0.527376,-4.682317,-0.844607,3.900877,0.083344,4.831777,-4.343647
834,4.290459,0.693147,2.890372,2.639057,0.594951,-0.767093,-0.693147,-0.883319,-0.995015,0.000000,0.000000,-4.094481,0.162205,-4.663645,0.018500,4.522294,0.630925,5.399220,-4.868974
835,3.135494,3.258097,2.995732,2.639057,-1.720676,-0.769915,-0.693147,0.225322,1.375451,0.000000,-0.693147,-4.341002,-0.153585,-4.262613,0.376406,4.116799,-0.251192,3.830973,2.788410


In [148]:
for col in stup_3.columns[:-2]:
    print(stup_3[col].value_counts())

early_late
1    610
0    227
Name: count, dtype: int64
tech_1
1    676
0    161
Name: count, dtype: int64
cls_cd
3     379
5     148
7     130
1     105
4      18
6      17
8      14
11      6
10      5
12      5
13      5
9       3
2       2
Name: count, dtype: int64
cofounder_yn
0    751
1     86
Name: count, dtype: int64
edu_lvl
3    329
5    250
4    207
0     34
1     13
2      4
Name: count, dtype: int64
prev_exp_relev_yn
0    562
1    275
Name: count, dtype: int64
prev_job_tenure_5y
1    518
2    228
0     91
Name: count, dtype: int64
edu_bg
1    612
0    225
Name: count, dtype: int64
startup_exp
0    568
1    261
2      8
Name: count, dtype: int64


In [29]:
stup_transformed = symlog_transformer.transform(stup_1.drop(['inv_amounts','inv_dur','Patent_total','Patent_above_B_minus'],axis=1))
stup_transformed

,ROE,ROA,ratio_of_netincome_to_sales,equity_ratio,Debt_ratio,pre_1y_sales,rate_of_sales_increase,pre_1y_opincome,rate_of_opincome_increase,pre_1y_netincome,rate_of_netincome_increase,pre_1y_assets,rate_of_assets_increase,pre_1y_liabilities,pre_1y_equity
0,0.065513,0.015207,0.009098,0.204017,1.485812,4.060008,2.252288,-1.615458,-1.318187,0.419171,1.038917,3.554803,1.472685,3.306536,2.162232
1,0.065513,0.015207,0.009098,0.204017,1.485812,4.060008,2.252288,-1.615458,-1.318187,0.419171,1.038917,3.554803,1.472685,3.306536,2.162232
2,-0.378487,-0.274873,-0.285387,0.523329,0.374496,4.686174,0.634300,-3.672139,-2.252575,-3.596878,-4.250593,4.728826,1.194150,3.584507,4.358336
3,-0.522583,-0.263627,-0.239271,0.364275,0.822187,4.301711,0.495152,-3.364962,-0.632771,-3.029484,-0.504997,4.193655,0.350783,3.626546,3.390533
4,-0.675845,-0.469657,-0.507113,0.482884,0.476841,4.684585,0.387232,-3.946426,-0.597126,-4.274550,-1.280598,4.780709,0.593834,3.824810,4.308981
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
832,0.878956,-1.165981,-0.693147,-0.943327,-0.969849,0.000000,-0.693147,-4.285249,-0.527376,-4.682317,-0.844607,3.900877,0.083344,4.831777,-4.343647
833,0.878956,-1.165981,-0.693147,-0.943327,-0.969849,0.000000,-0.693147,-4.285249,-0.527376,-4.682317,-0.844607,3.900877,0.083344,4.831777,-4.343647
834,0.594951,-0.767093,-0.693147,-0.883319,-0.995015,0.000000,0.000000,-4.094481,0.162205,-4.663645,0.018500,4.522294,0.630925,5.399220,-4.868974
835,-1.720676,-0.769915,-0.693147,0.225322,1.375451,0.000000,-0.693147,-4.341002,-0.153585,-4.262613,0.376406,4.116799,-0.251192,3.830973,2.788410


In [30]:
stup_transformed['Patent_total'] = stup_1['Patent_total']
stup_transformed['Patent_above_B_minus'] = stup_1['Patent_above_B_minus']
stup_transformed['inv_amounts'] = stup_1['inv_amounts']
stup_transformed['inv_dur'] = stup_1['inv_dur']
stup_transformed

,ROE,ROA,ratio_of_netincome_to_sales,equity_ratio,Debt_ratio,pre_1y_sales,rate_of_sales_increase,pre_1y_opincome,rate_of_opincome_increase,pre_1y_netincome,rate_of_netincome_increase,pre_1y_assets,rate_of_assets_increase,pre_1y_liabilities,pre_1y_equity,Patent_total,Patent_above_B_minus,inv_amounts,inv_dur
0,0.065513,0.015207,0.009098,0.204017,1.485812,4.060008,2.252288,-1.615458,-1.318187,0.419171,1.038917,3.554803,1.472685,3.306536,2.162232,1,1,55.0,16
1,0.065513,0.015207,0.009098,0.204017,1.485812,4.060008,2.252288,-1.615458,-1.318187,0.419171,1.038917,3.554803,1.472685,3.306536,2.162232,1,1,140.0,6
2,-0.378487,-0.274873,-0.285387,0.523329,0.374496,4.686174,0.634300,-3.672139,-2.252575,-3.596878,-4.250593,4.728826,1.194150,3.584507,4.358336,1,1,140.0,1
3,-0.522583,-0.263627,-0.239271,0.364275,0.822187,4.301711,0.495152,-3.364962,-0.632771,-3.029484,-0.504997,4.193655,0.350783,3.626546,3.390533,2,0,110.0,7
4,-0.675845,-0.469657,-0.507113,0.482884,0.476841,4.684585,0.387232,-3.946426,-0.597126,-4.274550,-1.280598,4.780709,0.593834,3.824810,4.308981,2,0,250.0,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
832,0.878956,-1.165981,-0.693147,-0.943327,-0.969849,0.000000,-0.693147,-4.285249,-0.527376,-4.682317,-0.844607,3.900877,0.083344,4.831777,-4.343647,15,13,65.0,11
833,0.878956,-1.165981,-0.693147,-0.943327,-0.969849,0.000000,-0.693147,-4.285249,-0.527376,-4.682317,-0.844607,3.900877,0.083344,4.831777,-4.343647,15,13,72.0,9
834,0.594951,-0.767093,-0.693147,-0.883319,-0.995015,0.000000,0.000000,-4.094481,0.162205,-4.663645,0.018500,4.522294,0.630925,5.399220,-4.868974,17,13,72.0,1
835,-1.720676,-0.769915,-0.693147,0.225322,1.375451,0.000000,-0.693147,-4.341002,-0.153585,-4.262613,0.376406,4.116799,-0.251192,3.830973,2.788410,19,13,22.0,25


In [39]:
stup_transformed = stup_transformed[['inv_amounts', 'inv_dur', 'ROE', 'ROA', 'ratio_of_netincome_to_sales', 'equity_ratio',
       'Debt_ratio', 'pre_1y_sales', 'rate_of_sales_increase',
       'pre_1y_opincome', 'rate_of_opincome_increase', 'pre_1y_netincome',
       'rate_of_netincome_increase', 'pre_1y_assets',
       'rate_of_assets_increase', 'pre_1y_liabilities', 'pre_1y_equity',
       'Patent_total', 'Patent_above_B_minus']]

In [40]:
shapiros = []

for i in conti_var:
    _, p_value_group_1 = stats.shapiro(stup_transformed[i])
    shapiros.append([i,p_value_group_1])
    print(i)
    print(f'스타트업의 p-value(정규성 검정): {p_value_group_1}')

inv_amounts
스타트업의 p-value(정규성 검정): 0.0
inv_dur
스타트업의 p-value(정규성 검정): 5.8803692357617035e-36
Patent_total
스타트업의 p-value(정규성 검정): 2.2420775429197073e-44
Patent_above_B_minus
스타트업의 p-value(정규성 검정): 0.0
ROE
스타트업의 p-value(정규성 검정): 6.787365791844317e-22
ROA
스타트업의 p-value(정규성 검정): 4.298336261158235e-19
ratio_of_netincome_to_sales
스타트업의 p-value(정규성 검정): 9.585576771181477e-30
equity_ratio
스타트업의 p-value(정규성 검정): 2.539925024735591e-35
Debt_ratio
스타트업의 p-value(정규성 검정): 3.727611882287792e-21
pre_1y_sales
스타트업의 p-value(정규성 검정): 5.979626973559365e-16
rate_of_sales_increase
스타트업의 p-value(정규성 검정): 9.12251982989539e-31
pre_1y_opincome
스타트업의 p-value(정규성 검정): 2.493695054957565e-28
rate_of_opincome_increase
스타트업의 p-value(정규성 검정): 3.4614974932445774e-25
pre_1y_netincome
스타트업의 p-value(정규성 검정): 3.525258852238489e-24
rate_of_netincome_increase
스타트업의 p-value(정규성 검정): 4.6115416085059326e-20
pre_1y_assets
스타트업의 p-value(정규성 검정): 8.600944934565591e-13
rate_of_assets_increase
스타트업의 p-value(정규성 검정): 1.20991818672184

In [55]:
pd.DataFrame(shapiros, columns=['Variable', 'P-value']).to_excel('C:/플랫폼 SME 연구센터/투자여부예측/스타트업기준_로그변환후_shapiro정규성검정결과_250406.xlsx')

In [37]:
for i in conti_var:
    stat, p = kstest(stup_transformed[i], 'norm')
    print(i)
    print(f"Kolmogorov-Smirnov Test for profit_symlog: p-value = {p}")

inv_amounts
Kolmogorov-Smirnov Test for profit_symlog: p-value = 0.0
inv_dur
Kolmogorov-Smirnov Test for profit_symlog: p-value = 0.0
Patent_total
Kolmogorov-Smirnov Test for profit_symlog: p-value = 5.11121992235258e-194
Patent_above_B_minus
Kolmogorov-Smirnov Test for profit_symlog: p-value = 5.11121992235258e-194
ROE
Kolmogorov-Smirnov Test for profit_symlog: p-value = 7.377556378987218e-44
ROA
Kolmogorov-Smirnov Test for profit_symlog: p-value = 6.859619546645803e-136
ratio_of_netincome_to_sales
Kolmogorov-Smirnov Test for profit_symlog: p-value = 2.5901965486674915e-129
equity_ratio
Kolmogorov-Smirnov Test for profit_symlog: p-value = 1.2794986322176114e-86
Debt_ratio
Kolmogorov-Smirnov Test for profit_symlog: p-value = 6.642572334234594e-84
pre_1y_sales
Kolmogorov-Smirnov Test for profit_symlog: p-value = 6.120574e-318
rate_of_sales_increase
Kolmogorov-Smirnov Test for profit_symlog: p-value = 4.714810262253735e-88
pre_1y_opincome
Kolmogorov-Smirnov Test for profit_symlog: p-valu

### 투자자

In [335]:
invst_transformed_1 = symlog_transformer.transform(invst_h4_1_a_conti.drop(['sequantial_inv','Patent_total','Patent_above_B_minus'],axis=1))
invst_transformed_2 = symlog_transformer.transform(invst_h4_1_bc_conti.drop(['Patent_total','Patent_above_B_minus'],axis=1))

In [336]:
invst_transformed_1['Patent_total'] = invst_h4_1_a_conti['Patent_total']
invst_transformed_1['Patent_above_B_minus'] = invst_h4_1_a_conti['Patent_above_B_minus']
invst_transformed_2['Patent_total'] = invst_h4_1_bc_conti['Patent_total']
invst_transformed_2['Patent_above_B_minus'] = invst_h4_1_bc_conti['Patent_above_B_minus']
invst_transformed_1

,ROE,ROA,ratio_of_netincome_to_sales,equity_ratio,Debt_ratio,pre_1y_sales,rate_of_sales_increase,pre_1y_opincome,rate_of_opincome_increase,pre_1y_netincome,rate_of_netincome_increase,pre_1y_assets,rate_of_assets_increase,pre_1y_liabilities,pre_1y_equity,Patent_total,Patent_above_B_minus
3,0.065513,0.015207,0.009098,0.204017,1.485812,4.060008,2.252288,-1.615458,-1.318187,0.419171,1.038917,3.554803,1.472685,3.306536,2.162232,1,1
4,-1.715532,-1.249428,-0.387923,0.435499,0.605623,3.828055,0.142628,-3.105327,-1.022425,-3.105191,-1.030298,2.258144,-0.537840,1.587382,1.735963,0,0
5,-1.715532,-1.249428,-0.387923,0.435499,0.605623,3.828055,0.142628,-3.105327,-1.022425,-3.105191,-1.030298,2.258144,-0.537840,1.587382,1.735963,0,0
7,-0.322753,-0.312727,-0.693147,0.674898,0.036837,0.000000,-0.693147,-3.183569,-2.617194,-3.190408,-2.930766,4.166008,2.968777,1.192421,4.129753,2,1
8,-0.322753,-0.312727,-0.693147,0.674898,0.036837,0.000000,-0.693147,-3.183569,-2.617194,-3.190408,-2.930766,4.166008,2.968777,1.192421,4.129753,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3739,-0.323706,-0.317658,-2.391266,0.682186,0.022044,2.533006,-0.546241,-4.771884,-1.568395,-4.754198,-1.573547,5.732539,1.161614,2.042501,5.710567,3,2
3741,0.464838,-0.557797,-3.936881,-0.816268,-1.026890,1.639177,0.693147,-4.887433,-0.411120,-5.345294,-0.238398,5.636016,0.593878,6.450291,-5.868005,9,1
3743,0.096925,0.091912,0.449737,0.665704,0.055662,5.088344,3.442950,3.524217,0.792230,4.527214,0.712017,6.858017,-0.029541,3.960045,6.802415,14,7
3744,0.472571,-1.397640,-7.008368,-1.798654,-0.787709,0.127689,0.819780,-4.687721,-0.454888,-5.020456,0.038500,3.920159,-0.219621,5.702119,-5.521833,3,1


In [337]:
invst_transformed_2

,inv_amounts,inv_dur,ROE,ROA,ratio_of_netincome_to_sales,equity_ratio,Debt_ratio,pre_1y_sales,rate_of_sales_increase,pre_1y_opincome,rate_of_opincome_increase,pre_1y_netincome,rate_of_netincome_increase,pre_1y_assets,rate_of_assets_increase,pre_1y_liabilities,pre_1y_equity,Patent_total,Patent_above_B_minus
3,4.025352,2.833213,0.065513,0.015207,0.009098,0.204017,1.485812,4.060008,2.252288,-1.615458,-1.318187,0.419171,1.038917,3.554803,1.472685,3.306536,2.162232,1,1
4,3.433987,2.995732,-1.715532,-1.249428,-0.387923,0.435499,0.605623,3.828055,0.142628,-3.105327,-1.022425,-3.105191,-1.030298,2.258144,-0.537840,1.587382,1.735963,0,0
5,3.433987,2.995732,-1.715532,-1.249428,-0.387923,0.435499,0.605623,3.828055,0.142628,-3.105327,-1.022425,-3.105191,-1.030298,2.258144,-0.537840,1.587382,1.735963,0,0
7,4.615121,2.197225,-0.322753,-0.312727,-0.693147,0.674898,0.036837,0.000000,-0.693147,-3.183569,-2.617194,-3.190408,-2.930766,4.166008,2.968777,1.192421,4.129753,2,1
23,4.394449,2.397895,-0.525153,-0.485152,-1.533160,0.643967,0.100905,1.047621,0.254834,-2.620529,-1.926218,-2.044305,-1.933126,2.465387,1.863836,0.709773,2.373463,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3605,5.513429,3.931826,-2.487271,-0.547608,-1.414926,0.064019,2.716401,4.691814,0.387454,-5.335031,-0.546543,-5.822176,-0.357406,6.137301,-0.094331,6.069055,3.450966,4,4
3606,5.513429,0.693147,-2.487271,-0.547608,-1.414926,0.064019,2.716401,4.691814,0.387454,-5.335031,-0.546543,-5.822176,-0.357406,6.137301,-0.094331,6.069055,3.450966,4,4
3607,5.513429,2.639057,-2.487271,-0.547608,-1.414926,0.064019,2.716401,4.691814,0.387454,-5.335031,-0.546543,-5.822176,-0.357406,6.137301,-0.094331,6.069055,3.450966,4,4
3608,5.513429,2.079442,-2.487271,-0.547608,-1.414926,0.064019,2.716401,4.691814,0.387454,-5.335031,-0.546543,-5.822176,-0.357406,6.137301,-0.094331,6.069055,3.450966,4,4


In [214]:
invst_t_1 = []

for i in conti_var[2:]:
    _, p_value_group_1 = stats.shapiro(invst_transformed_1[i])
    invst_t_1.append([i, p_value_group_1])
    print(i)
    print(f'연속변수의 p-value(정규성 검정): {p_value_group_1}')

Patent_total
연속변수의 p-value(정규성 검정): 0.0
Patent_above_B_minus
연속변수의 p-value(정규성 검정): 0.0
ROE
연속변수의 p-value(정규성 검정): 3.769492869033758e-43
ROA
연속변수의 p-value(정규성 검정): 1.633917372519051e-38
ratio_of_netincome_to_sales
연속변수의 p-value(정규성 검정): 0.0
equity_ratio
연속변수의 p-value(정규성 검정): 0.0
Debt_ratio
연속변수의 p-value(정규성 검정): 8.210207702479103e-42
pre_1y_sales
연속변수의 p-value(정규성 검정): 3.77718771216444e-33
rate_of_sales_increase
연속변수의 p-value(정규성 검정): 0.0
pre_1y_opincome
연속변수의 p-value(정규성 검정): 0.0
rate_of_opincome_increase
연속변수의 p-value(정규성 검정): 1.5134023414708024e-43
pre_1y_netincome
연속변수의 p-value(정규성 검정): 7.006492321624085e-45
rate_of_netincome_increase
연속변수의 p-value(정규성 검정): 1.5455903958882529e-35
pre_1y_assets
연속변수의 p-value(정규성 검정): 7.763173460627093e-31
rate_of_assets_increase
연속변수의 p-value(정규성 검정): 0.0
pre_1y_liabilities
연속변수의 p-value(정규성 검정): 3.900846610244741e-26
pre_1y_equity
연속변수의 p-value(정규성 검정): 0.0


In [218]:
pd.DataFrame(invst_t_1, columns=['Variable', 'P-Value']).to_excel('C:/플랫폼 SME 연구센터/투자여부예측/투자자기준_가설2_c_로그변환후_shapiro정규성검정결과_250406.xlsx')

In [216]:
invst_t_2 = []

for i in conti_var:
    _, p_value_group_1 = stats.shapiro(invst_transformed_2[i])
    invst_t_2.append([i, p_value_group_1])
    print(i)
    print(f'투자초기단계 그룹의 p-value(정규성 검정): {p_value_group_1}')

inv_amounts
투자초기단계 그룹의 p-value(정규성 검정): 3.856488153992921e-14
inv_dur
투자초기단계 그룹의 p-value(정규성 검정): 1.1718868943323507e-25
Patent_total
투자초기단계 그룹의 p-value(정규성 검정): 0.0
Patent_above_B_minus
투자초기단계 그룹의 p-value(정규성 검정): 0.0
ROE
투자초기단계 그룹의 p-value(정규성 검정): 6.514505056663858e-31
ROA
투자초기단계 그룹의 p-value(정규성 검정): 2.9100965163736407e-27
ratio_of_netincome_to_sales
투자초기단계 그룹의 p-value(정규성 검정): 1.2777622337874841e-36
equity_ratio
투자초기단계 그룹의 p-value(정규성 검정): 2.2238606628834847e-42
Debt_ratio
투자초기단계 그룹의 p-value(정규성 검정): 5.9707548480525794e-30
pre_1y_sales
투자초기단계 그룹의 p-value(정규성 검정): 5.147542932433574e-21
rate_of_sales_increase
투자초기단계 그룹의 p-value(정규성 검정): 7.143428552804432e-37
pre_1y_opincome
투자초기단계 그룹의 p-value(정규성 검정): 2.4137376186490257e-32
rate_of_opincome_increase
투자초기단계 그룹의 p-value(정규성 검정): 9.787643986777094e-29
pre_1y_netincome
투자초기단계 그룹의 p-value(정규성 검정): 1.1769585149883836e-27
rate_of_netincome_increase
투자초기단계 그룹의 p-value(정규성 검정): 3.8968647115661215e-26
pre_1y_assets
투자초기단계 그룹의 p-value(정규성 검정): 

In [219]:
pd.DataFrame(invst_t_2, columns=['Variable', 'P-Value']).to_excel('C:/플랫폼 SME 연구센터/투자여부예측/투자자기준_가설2_a,b_로그변환후_shapiro정규성검정결과_250406.xlsx')

In [52]:
for i in conti_var[2:]:
    stat, p = kstest(invst_transformed_1[i], 'norm')
    print(i)
    print(f"Kolmogorov-Smirnov Test for profit_symlog: p-value = {p}")

Patent_total
Kolmogorov-Smirnov Test for profit_symlog: p-value = 4.71739510455071e-126
Patent_above_B_minus
Kolmogorov-Smirnov Test for profit_symlog: p-value = 4.71739510455071e-126
Patent_Class_C
Kolmogorov-Smirnov Test for profit_symlog: p-value = 4.71739510455071e-126
Patent_Class_B_minus
Kolmogorov-Smirnov Test for profit_symlog: p-value = 4.71739510455071e-126
Patent_Class_B0
Kolmogorov-Smirnov Test for profit_symlog: p-value = 4.71739510455071e-126
Patent_Class_B_plus
Kolmogorov-Smirnov Test for profit_symlog: p-value = 4.71739510455071e-126
Patent_Class_A_minus
Kolmogorov-Smirnov Test for profit_symlog: p-value = 4.71739510455071e-126
Patent_Class_A0
Kolmogorov-Smirnov Test for profit_symlog: p-value = 4.71739510455071e-126
Patent_Class_A_plus
Kolmogorov-Smirnov Test for profit_symlog: p-value = 4.71739510455071e-126
Patent_Class_AA
Kolmogorov-Smirnov Test for profit_symlog: p-value = 4.71739510455071e-126
ROE
Kolmogorov-Smirnov Test for profit_symlog: p-value = 3.716475387732

In [53]:
for i in conti_var:
    stat, p = kstest(invst_transformed_2[i], 'norm')
    print(i)
    print(f"Kolmogorov-Smirnov Test for profit_symlog: p-value = {p}")

inv_amounts
Kolmogorov-Smirnov Test for profit_symlog: p-value = 0.0
inv_dur
Kolmogorov-Smirnov Test for profit_symlog: p-value = 0.0
Patent_total
Kolmogorov-Smirnov Test for profit_symlog: p-value = 7.063417378483843e-70
Patent_above_B_minus
Kolmogorov-Smirnov Test for profit_symlog: p-value = 7.063417378483843e-70
Patent_Class_C
Kolmogorov-Smirnov Test for profit_symlog: p-value = 7.063417378483843e-70
Patent_Class_B_minus
Kolmogorov-Smirnov Test for profit_symlog: p-value = 7.063417378483843e-70
Patent_Class_B0
Kolmogorov-Smirnov Test for profit_symlog: p-value = 7.063417378483843e-70
Patent_Class_B_plus
Kolmogorov-Smirnov Test for profit_symlog: p-value = 7.063417378483843e-70
Patent_Class_A_minus
Kolmogorov-Smirnov Test for profit_symlog: p-value = 7.063417378483843e-70
Patent_Class_A0
Kolmogorov-Smirnov Test for profit_symlog: p-value = 7.063417378483843e-70
Patent_Class_A_plus
Kolmogorov-Smirnov Test for profit_symlog: p-value = 7.063417378483843e-70
Patent_Class_AA
Kolmogorov-S

In [54]:
invst_h4_1_a_conti = invst_transformed_1
invst_h4_1_bc_conti = invst_transformed_2

In [102]:
# Anderson-Darling Test(정규성 검정) - 이번에는 안 씀

#for i in conti_var:
#    result = anderson(df_transformed[i], dist='norm')
#    print(f"Anderson-Darling Test for profit_symlog: statistic = {result.statistic}, critical values = {result.critical_values}")

#statistic 값이 critical_values보다 작으면 정규성을 따름.
#값이 클수록 정규분포에서 멀어짐.
# critical_values: 특정 유의수준 (significance level, α)에 따른 임계값

Anderson-Darling Test for profit_symlog: statistic = 104.87425774577969, critical values = [0.575 0.655 0.786 0.917 1.091]
Anderson-Darling Test for profit_symlog: statistic = 194.2355500729309, critical values = [0.575 0.655 0.786 0.917 1.091]
Anderson-Darling Test for profit_symlog: statistic = 288.82369232431074, critical values = [0.575 0.655 0.786 0.917 1.091]
Anderson-Darling Test for profit_symlog: statistic = 426.94783298985203, critical values = [0.575 0.655 0.786 0.917 1.091]
Anderson-Darling Test for profit_symlog: statistic = 419.7249741699288, critical values = [0.575 0.655 0.786 0.917 1.091]
Anderson-Darling Test for profit_symlog: statistic = 656.354973462765, critical values = [0.575 0.655 0.786 0.917 1.091]
Anderson-Darling Test for profit_symlog: statistic = 896.9103005550332, critical values = [0.575 0.655 0.786 0.917 1.091]
Anderson-Darling Test for profit_symlog: statistic = 1124.4023552993585, critical values = [0.575 0.655 0.786 0.917 1.091]
Anderson-Darling Test

In [104]:
# Yeo-Jhonson 변환 -> 이번에는 사용 안 함
#yj_transformer = PowerTransformer(method='yeo-johnson')

In [106]:
#df_transformed_1 = pd.DataFrame(yj_transformer.fit_transform(df_transformed), columns=df_transformed.columns)
#df_transformed_1

,Patent_total,Patent_above_B_minus,Patent_Class_C,Patent_Class_B_minus,Patent_Class_B0,Patent_Class_B_plus,Patent_Class_A_minus,Patent_Class_A0,Patent_Class_A_plus,Patent_Class_AA,ROE,ROA,ratio_of_netincome_to_sales,equity_ratio,Debt_ratio,pre_2y_sales,pre_1y_sales,rate_of_sales_increase,pre_2y_opincome,pre_1y_opincome,rate_of_opincome_increase,pre_2y_netincome,pre_1y_netincome,rate_of_netincome_increase,pre_2y_assets,pre_1y_assets,rate_of_assets_increase,pre_1y_liabilities,pre_1y_equity
0,-0.339388,0.069790,-0.961536,0.896435,-0.811038,-0.619943,-0.464998,-0.342357,-0.320021,-0.23749,0.447881,1.137784,0.792904,-0.672193,1.048183,0.174292,0.692010,1.392905,1.137110,1.084092,-0.506209,1.113192,1.643226,1.531878,-1.094642,-0.615479,0.964861,-0.005189,-0.464649
1,-0.339388,0.069790,-0.961536,0.896435,-0.811038,-0.619943,-0.464998,-0.342357,-0.320021,-0.23749,0.447881,1.137784,0.792904,-0.672193,1.048183,0.174292,0.692010,1.392905,1.137110,1.084092,-0.506209,1.113192,1.643226,1.531878,-1.094642,-0.615479,0.964861,-0.005189,-0.464649
2,-0.339388,0.069790,-0.961536,0.896435,-0.811038,-0.619943,-0.464998,-0.342357,-0.320021,-0.23749,0.447881,1.137784,0.792904,-0.672193,1.048183,0.174292,0.692010,1.392905,1.137110,1.084092,-0.506209,1.113192,1.643226,1.531878,-1.094642,-0.615479,0.964861,-0.005189,-0.464649
3,-0.339388,0.069790,-0.961536,0.896435,-0.811038,-0.619943,-0.464998,-0.342357,-0.320021,-0.23749,0.447881,1.137784,0.792904,-0.672193,1.048183,0.174292,0.692010,1.392905,1.137110,1.084092,-0.506209,1.113192,1.643226,1.531878,-1.094642,-0.615479,0.964861,-0.005189,-0.464649
4,-1.287305,-1.094217,-0.961536,-0.806384,-0.811038,-0.619943,-0.464998,-0.342357,-0.320021,-0.23749,-1.463353,-1.791931,0.511990,0.076285,0.256113,0.873002,0.605214,-0.487122,0.369344,0.240193,-0.265262,0.317855,0.249299,-0.181655,-0.334514,-1.578917,-1.853641,-1.037329,-0.620270
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3741,0.492369,0.863185,-0.961536,-0.806384,1.460713,-0.619943,-0.464998,-0.342357,-0.320021,-0.23749,-2.276866,-0.388022,-0.131034,-0.982099,2.139413,1.065238,0.917415,-0.148039,-1.380106,-1.428234,0.130084,-1.587921,-1.431868,0.336800,1.363251,1.053422,-0.764909,1.221279,0.068105
3742,0.492369,0.863185,-0.961536,-0.806384,1.460713,-0.619943,-0.464998,-0.342357,-0.320021,-0.23749,-2.276866,-0.388022,-0.131034,-0.982099,2.139413,1.065238,0.917415,-0.148039,-1.380106,-1.428234,0.130084,-1.587921,-1.431868,0.336800,1.363251,1.053422,-0.764909,1.221279,0.068105
3743,0.492369,0.863185,-0.961536,-0.806384,1.460713,-0.619943,-0.464998,-0.342357,-0.320021,-0.23749,-2.276866,-0.388022,-0.131034,-0.982099,2.139413,1.065238,0.917415,-0.148039,-1.380106,-1.428234,0.130084,-1.587921,-1.431868,0.336800,1.363251,1.053422,-0.764909,1.221279,0.068105
3744,0.492369,0.863185,-0.961536,-0.806384,1.460713,-0.619943,-0.464998,-0.342357,-0.320021,-0.23749,-2.276866,-0.388022,-0.131034,-0.982099,2.139413,1.065238,0.917415,-0.148039,-1.380106,-1.428234,0.130084,-1.587921,-1.431868,0.336800,1.363251,1.053422,-0.764909,1.221279,0.068105


In [108]:
#for i in conti_var:
#    _, p_value_group_1 = stats.shapiro(df_transformed_1[i])
#    print(i)
#    print(f'투자초기단계 그룹의 p-value(정규성 검정): {p_value_group_1}')

Patent_total
투자초기단계 그룹의 p-value(정규성 검정): 9.80908925027372e-45
Patent_above_B_minus
투자초기단계 그룹의 p-value(정규성 검정): 0.0
Patent_Class_C
투자초기단계 그룹의 p-value(정규성 검정): 0.0
Patent_Class_B_minus
투자초기단계 그룹의 p-value(정규성 검정): 0.0
Patent_Class_B0
투자초기단계 그룹의 p-value(정규성 검정): 0.0
Patent_Class_B_plus
투자초기단계 그룹의 p-value(정규성 검정): 0.0
Patent_Class_A_minus
투자초기단계 그룹의 p-value(정규성 검정): 0.0
Patent_Class_A0
투자초기단계 그룹의 p-value(정규성 검정): 0.0
Patent_Class_A_plus
투자초기단계 그룹의 p-value(정규성 검정): 0.0
Patent_Class_AA
투자초기단계 그룹의 p-value(정규성 검정): 0.0
ROE
투자초기단계 그룹의 p-value(정규성 검정): 1.2611686178923354e-44
ROA
투자초기단계 그룹의 p-value(정규성 검정): 3.407558380829668e-35
ratio_of_netincome_to_sales
투자초기단계 그룹의 p-value(정규성 검정): 0.0
equity_ratio
투자초기단계 그룹의 p-value(정규성 검정): 1.023471964582774e-39
Debt_ratio
투자초기단계 그룹의 p-value(정규성 검정): 1.891752926838503e-43
pre_2y_sales
투자초기단계 그룹의 p-value(정규성 검정): 3.438786431453101e-42
pre_1y_sales
투자초기단계 그룹의 p-value(정규성 검정): 1.56887259544606e-36
rate_of_sales_increase
투자초기단계 그룹의 p-value(정규성 검정): 2.1244791468091

In [107]:
#for i in conti_var:
#    stat, p = kstest(df_transformed_1[i], 'norm')
#    print(i)
#    print(f"Kolmogorov-Smirnov Test for profit_symlog: p-value = {p}")

Patent_total
Kolmogorov-Smirnov Test for profit_symlog: p-value = 8.468553438338386e-137
Patent_above_B_minus
Kolmogorov-Smirnov Test for profit_symlog: p-value = 2.612462595284171e-252
Patent_Class_C
Kolmogorov-Smirnov Test for profit_symlog: p-value = 0.0
Patent_Class_B_minus
Kolmogorov-Smirnov Test for profit_symlog: p-value = 0.0
Patent_Class_B0
Kolmogorov-Smirnov Test for profit_symlog: p-value = 0.0
Patent_Class_B_plus
Kolmogorov-Smirnov Test for profit_symlog: p-value = 0.0
Patent_Class_A_minus
Kolmogorov-Smirnov Test for profit_symlog: p-value = 0.0
Patent_Class_A0
Kolmogorov-Smirnov Test for profit_symlog: p-value = 0.0
Patent_Class_A_plus
Kolmogorov-Smirnov Test for profit_symlog: p-value = 0.0
Patent_Class_AA
Kolmogorov-Smirnov Test for profit_symlog: p-value = 0.0
ROE
Kolmogorov-Smirnov Test for profit_symlog: p-value = 1.308783241340966e-41
ROA
Kolmogorov-Smirnov Test for profit_symlog: p-value = 9.254696377341557e-18
ratio_of_netincome_to_sales
Kolmogorov-Smirnov Test for

In [109]:
#for i in conti_var:
#    result = anderson(df_transformed_1[i], dist='norm')
#    print(f"Anderson-Darling Test for profit_symlog: statistic = {result.statistic}, critical values = {result.critical_values}")
#statistic 값이 critical_values보다 작으면 정규성을 따름.
#값이 클수록 정규분포에서 멀어짐.
# critical_values: 특정 유의수준 (significance level, α)에 따른 임계값

Anderson-Darling Test for profit_symlog: statistic = 135.62464627456393, critical values = [0.575 0.655 0.786 0.917 1.091]
Anderson-Darling Test for profit_symlog: statistic = 250.5143040063249, critical values = [0.575 0.655 0.786 0.917 1.091]
Anderson-Darling Test for profit_symlog: statistic = 376.80658907209545, critical values = [0.575 0.655 0.786 0.917 1.091]
Anderson-Darling Test for profit_symlog: statistic = 560.7561351048407, critical values = [0.575 0.655 0.786 0.917 1.091]
Anderson-Darling Test for profit_symlog: statistic = 556.6256491151016, critical values = [0.575 0.655 0.786 0.917 1.091]
Anderson-Darling Test for profit_symlog: statistic = 819.6143756063357, critical values = [0.575 0.655 0.786 0.917 1.091]
Anderson-Darling Test for profit_symlog: statistic = 1044.1153807185656, critical values = [0.575 0.655 0.786 0.917 1.091]
Anderson-Darling Test for profit_symlog: statistic = 1221.8197966753469, critical values = [0.575 0.655 0.786 0.917 1.091]
Anderson-Darling Tes

## 상관관계 분석(연속형-연속형)

### 스타트업

In [20]:
#stup_corr = stup[conti_var].corr(method='pearson')
#stup_corr

,inv_amounts,inv_dur,Patent_total,Patent_above_B-,Patent_Class_C,Patent_Class_B-,Patent_Class_B0,Patent_Class_B+,Patent_Class_A-,Patent_Class_A0,Patent_Class_A+,Patent_Class_AA,ROE,ROA,ratio_of_netincome_to_sales,equity_ratio,Debt_ratio,pre_2y_sales,pre_1y_sales,rate_of_sales_increase,pre_2y_opincome,pre_1y_opincome,rate_of_opincome_increase,pre_2y_netincome,pre_1y_netincome,rate_of_netincome_increase,pre_2y_assets,pre_1y_assets,rate_of_assets_increase,pre_1y_liabilities,pre_1y_equity
inv_amounts,1.000000,0.005207,0.053997,0.058232,0.012684,-0.009720,0.075289,0.083921,0.048519,0.041089,0.025709,0.076375,0.002026,0.077540,-0.003798,-0.013857,0.020721,0.274110,0.182590,-0.005251,-0.388110,0.000325,0.033894,-0.210507,-0.078697,0.013740,0.237723,0.267051,0.008864,0.257226,0.214150
inv_dur,0.005207,1.000000,0.028465,0.020260,0.036869,0.083674,0.002103,0.011045,0.001493,-0.002332,-0.005141,-0.019609,0.000940,-0.026100,-0.007852,-0.050076,0.009664,-0.026973,-0.053158,0.022102,-0.015291,-0.040418,0.051081,-0.001866,-0.024564,0.029051,-0.004899,-0.035131,-0.013810,-0.029739,-0.051653
Patent_total,0.053997,0.028465,1.000000,0.959431,0.579032,0.679280,0.898577,0.830454,0.808380,0.776962,0.770996,0.754513,0.038305,-0.050370,0.008520,-0.191462,-0.050019,-0.024458,-0.011573,-0.021924,-0.073447,-0.001073,0.046087,-0.018449,-0.014514,0.033254,-0.007724,-0.013930,-0.034539,-0.012312,-0.017501
Patent_above_B-,0.058232,0.020260,0.959431,1.000000,0.325671,0.539953,0.963305,0.881710,0.907606,0.868649,0.862400,0.853234,0.031310,-0.059727,0.009496,-0.210062,-0.040066,-0.015470,-0.012291,-0.015740,-0.092472,-0.011439,0.039752,-0.026116,-0.028856,0.026903,-0.006749,-0.014947,-0.027885,-0.011006,-0.031407
Patent_Class_C,0.012684,0.036869,0.579032,0.325671,1.000000,0.716530,0.227713,0.235217,0.086308,0.093604,0.091666,0.062897,0.037915,0.003802,0.001114,-0.034614,-0.051878,-0.037282,-0.003269,-0.028008,0.021105,0.029480,0.039596,0.013653,0.034773,0.033719,-0.006385,-0.003490,-0.035190,-0.009460,0.032131
Patent_Class_B-,-0.009720,0.083674,0.679280,0.539953,0.716530,1.000000,0.401385,0.425862,0.213154,0.206833,0.254019,0.144049,0.033451,-0.060342,0.008181,-0.147329,-0.046101,-0.012310,-0.016424,-0.021749,-0.023652,-0.006846,0.031623,-0.013558,-0.013745,0.032232,-0.013336,-0.023447,-0.033888,-0.023525,-0.013419
Patent_Class_B0,0.075289,0.002103,0.898577,0.963305,0.227713,0.401385,1.000000,0.800956,0.901377,0.846102,0.823843,0.868337,0.025557,-0.042350,0.007122,-0.194432,-0.033207,-0.003533,-0.001304,-0.018020,-0.094752,-0.006410,0.037467,-0.025378,-0.026055,0.024284,-0.000790,-0.006909,-0.024055,-0.004736,-0.016530
Patent_Class_B+,0.083921,0.011045,0.830454,0.881710,0.235217,0.425862,0.800956,1.000000,0.803594,0.757915,0.750977,0.727794,0.037584,-0.057927,0.012990,-0.210499,-0.044717,-0.017217,0.004815,-0.006096,-0.084895,0.012053,0.049849,-0.022335,-0.008525,0.025475,-0.001539,-0.002874,-0.027644,-0.001981,-0.006812
Patent_Class_A-,0.048519,0.001493,0.808380,0.907606,0.086308,0.213154,0.901377,0.803594,1.000000,0.887721,0.847855,0.904692,0.017326,-0.064264,0.008131,-0.195367,-0.022332,-0.014182,-0.017396,-0.013770,-0.090591,-0.022195,0.029426,-0.020398,-0.035448,0.017161,-0.006629,-0.015043,-0.016640,-0.008077,-0.048790
Patent_Class_A0,0.041089,-0.002332,0.776962,0.868649,0.093604,0.206833,0.846102,0.757915,0.887721,1.000000,0.858825,0.861865,0.013644,-0.069991,0.005651,-0.194654,-0.018220,-0.030545,-0.024891,0.015431,-0.098062,-0.021992,0.024930,-0.030464,-0.037047,0.013299,-0.008311,-0.015496,-0.015541,-0.007667,-0.053999


In [112]:
# Spearman 상관계수 계산
#spearman_corr = invst_transformed.corr(method='spearman') # method='pearson'으로 변경 시 pearson 상관계수 계산
#spearman_corr

,Patent_total,Patent_above_B_minus,Patent_Class_C,Patent_Class_B_minus,Patent_Class_B0,Patent_Class_B_plus,Patent_Class_A_minus,Patent_Class_A0,Patent_Class_A_plus,Patent_Class_AA,ROE,ROA,ratio_of_netincome_to_sales,equity_ratio,Debt_ratio,pre_2y_sales,pre_1y_sales,rate_of_sales_increase,pre_2y_opincome,pre_1y_opincome,rate_of_opincome_increase,pre_2y_netincome,pre_1y_netincome,rate_of_netincome_increase,pre_2y_assets,pre_1y_assets,rate_of_assets_increase,pre_1y_liabilities,pre_1y_equity
Patent_total,1.000000,0.914014,0.778396,0.785143,0.755070,0.628147,0.481008,0.432015,0.425188,0.295860,0.131889,-0.059024,-0.260155,-0.045277,-0.177144,-0.043103,-0.132428,-0.111041,-0.183753,-0.167300,0.113451,-0.178662,-0.167887,0.072203,0.243827,0.132759,-0.189735,0.144355,-0.021134
Patent_above_B_minus,0.914014,1.000000,0.527394,0.825626,0.805960,0.691842,0.559668,0.474200,0.467212,0.320497,0.102868,-0.072453,-0.204199,-0.082323,-0.131171,-0.006523,-0.082939,-0.095925,-0.190611,-0.167989,0.107469,-0.189238,-0.180331,0.066295,0.260300,0.129572,-0.220922,0.171010,-0.034699
Patent_Class_C,0.778396,0.527394,1.000000,0.539625,0.482161,0.328909,0.149182,0.176145,0.190792,0.146543,0.124621,-0.015864,-0.247823,0.016050,-0.192537,-0.060837,-0.133927,-0.073427,-0.103801,-0.107054,0.083741,-0.102483,-0.104918,0.080024,0.140682,0.096617,-0.086037,0.061610,-0.002601
Patent_Class_B_minus,0.785143,0.825626,0.539625,1.000000,0.569221,0.509741,0.361424,0.410734,0.399059,0.256759,0.082546,-0.073119,-0.155121,-0.044986,-0.132812,0.006082,-0.063994,-0.093529,-0.167709,-0.153121,0.086315,-0.170939,-0.154963,0.055375,0.204127,0.107080,-0.175295,0.124759,-0.022554
Patent_Class_B0,0.755070,0.805960,0.482161,0.569221,1.000000,0.534352,0.391772,0.371222,0.327737,0.244870,0.105375,-0.064307,-0.192157,-0.093109,-0.125194,0.008215,-0.058743,-0.110751,-0.176864,-0.156148,0.120829,-0.186366,-0.195155,0.091181,0.248256,0.150906,-0.174735,0.184863,-0.038995
Patent_Class_B_plus,0.628147,0.691842,0.328909,0.509741,0.534352,1.000000,0.476759,0.372278,0.381241,0.315421,0.139384,-0.024441,-0.112910,-0.054336,-0.132982,0.078562,0.040640,-0.007170,-0.090402,-0.064923,0.127402,-0.092439,-0.101708,0.063333,0.206484,0.091071,-0.189656,0.131058,-0.034685
Patent_Class_A_minus,0.481008,0.559668,0.149182,0.361424,0.391772,0.476759,1.000000,0.486256,0.509961,0.327152,0.017899,-0.137786,-0.119285,-0.131341,-0.047692,0.001536,-0.032440,-0.065998,-0.129921,-0.140527,0.044194,-0.132807,-0.139333,-0.005028,0.193487,0.050450,-0.238831,0.148996,-0.076059
Patent_Class_A0,0.432015,0.474200,0.176145,0.410734,0.371222,0.372278,0.486256,1.000000,0.539586,0.235305,0.078349,-0.149571,-0.105734,-0.131532,-0.093660,0.000116,-0.032375,-0.034244,-0.129303,-0.119815,0.054409,-0.146507,-0.124144,0.012891,0.155605,0.049808,-0.187167,0.146057,-0.109020
Patent_Class_A_plus,0.425188,0.467212,0.190792,0.399059,0.327737,0.381241,0.509961,0.539586,1.000000,0.323058,0.084199,-0.074683,-0.107389,-0.057210,-0.114622,-0.032980,-0.080459,-0.065952,-0.109455,-0.097923,0.065226,-0.101019,-0.067657,0.037547,0.105166,0.034169,-0.135189,0.076847,-0.062617
Patent_Class_AA,0.295860,0.320497,0.146543,0.256759,0.244870,0.315421,0.327152,0.235305,0.323058,1.000000,0.055805,0.032577,-0.046073,0.028175,-0.043011,0.034530,-0.004023,-0.047784,0.012608,-0.025590,-0.006856,0.005264,-0.040892,-0.040916,0.090316,0.051604,-0.074261,0.010028,0.013013


In [21]:
# Spearman 상관계수 & p-value를 데이터프레임 전체에 적용하는 함수
def spearman_corr_pval(df):
    cols = df.columns
    corr_matrix = pd.DataFrame(index=cols, columns=cols, dtype=float)
    pval_matrix = pd.DataFrame(index=cols, columns=cols, dtype=float)
    
    for col1 in cols:
        for col2 in cols:
            corr, pval = spearmanr(df[col1], df[col2])
            corr_matrix.loc[col1, col2] = corr
            pval_matrix.loc[col1, col2] = pval
    
    return corr_matrix, pval_matrix

In [41]:
# 실행
stup_sp_corr, stup_sp_pvals = spearman_corr_pval(stup_transformed)

In [44]:
stup_sp_corr.to_excel('C:/플랫폼 SME 연구센터/투자여부예측/스타트업기준_연속변수간선형상관_250418_종속변수변환X.xlsx')

In [42]:
stup_sp_corr[abs(stup_sp_corr)>0.5]

,inv_amounts,inv_dur,ROE,ROA,ratio_of_netincome_to_sales,equity_ratio,Debt_ratio,pre_1y_sales,rate_of_sales_increase,pre_1y_opincome,rate_of_opincome_increase,pre_1y_netincome,rate_of_netincome_increase,pre_1y_assets,rate_of_assets_increase,pre_1y_liabilities,pre_1y_equity,Patent_total,Patent_above_B_minus
inv_amounts,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.588745,NaN,NaN,NaN,NaN,NaN
inv_dur,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ROE,NaN,NaN,1.000000,NaN,NaN,NaN,-0.637718,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ROA,NaN,NaN,NaN,1.000000,0.547901,NaN,NaN,NaN,NaN,NaN,NaN,0.563183,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ratio_of_netincome_to_sales,NaN,NaN,NaN,0.547901,1.000000,NaN,NaN,0.552821,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
equity_ratio,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.653454,0.559408,NaN,NaN
Debt_ratio,NaN,NaN,-0.637718,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
pre_1y_sales,NaN,NaN,NaN,NaN,0.552821,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,0.501195,NaN,0.543909,NaN,NaN,NaN
rate_of_sales_increase,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
pre_1y_opincome,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,0.909096,NaN,-0.563174,NaN,-0.545610,NaN,NaN,NaN


In [43]:
stup_sp_pvals[abs(stup_sp_pvals)<0.05]

,inv_amounts,inv_dur,ROE,ROA,ratio_of_netincome_to_sales,equity_ratio,Debt_ratio,pre_1y_sales,rate_of_sales_increase,pre_1y_opincome,rate_of_opincome_increase,pre_1y_netincome,rate_of_netincome_increase,pre_1y_assets,rate_of_assets_increase,pre_1y_liabilities,pre_1y_equity,Patent_total,Patent_above_B_minus
inv_amounts,0.000000e+00,9.780539e-06,NaN,NaN,NaN,NaN,NaN,1.702309e-25,NaN,3.573170e-42,NaN,7.900444e-42,NaN,3.174373e-79,NaN,4.339196e-50,1.167394e-22,NaN,NaN
inv_dur,9.780539e-06,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,3.472486e-03,NaN,1.053083e-04,NaN,1.001664e-04,NaN,4.067422e-15,NaN,NaN,NaN,NaN
ROE,NaN,NaN,0.000000e+00,8.698147e-20,4.413743e-04,1.510710e-06,9.601923e-97,NaN,NaN,2.166173e-10,1.360619e-10,4.015629e-08,6.523656e-08,8.902682e-04,5.942933e-04,5.362523e-06,2.320170e-09,8.085462e-06,1.421927e-04
ROA,NaN,NaN,8.698147e-20,0.000000e+00,9.524729e-67,2.407088e-18,1.061972e-11,2.007693e-12,5.369179e-05,5.434802e-50,1.729660e-09,3.260500e-71,3.050219e-23,3.236948e-09,3.023762e-33,NaN,3.417669e-43,NaN,NaN
ratio_of_netincome_to_sales,NaN,NaN,4.413743e-04,9.524729e-67,0.000000e+00,NaN,3.666040e-25,3.684741e-68,1.239634e-02,2.474500e-21,4.837894e-10,1.715988e-33,4.252097e-21,1.954180e-02,1.524883e-05,3.430803e-03,1.273322e-07,3.241961e-14,1.677908e-09
equity_ratio,NaN,NaN,1.510710e-06,2.407088e-18,NaN,0.000000e+00,8.699997e-07,2.264061e-11,NaN,2.363744e-09,4.897166e-02,4.995094e-16,NaN,2.704830e-02,1.771275e-06,4.508080e-103,4.346849e-70,NaN,3.089337e-02
Debt_ratio,NaN,NaN,9.601923e-97,1.061972e-11,3.666040e-25,8.699997e-07,0.000000e+00,2.954975e-31,1.943842e-04,1.487098e-02,NaN,2.491976e-04,2.120210e-03,NaN,NaN,2.191019e-06,1.286299e-15,1.128719e-08,2.366023e-06
pre_1y_sales,1.702309e-25,NaN,NaN,2.007693e-12,3.684741e-68,2.264061e-11,2.954975e-31,0.000000e+00,4.650528e-14,3.297272e-07,7.870014e-09,2.672246e-06,1.539607e-09,1.941098e-54,NaN,1.281719e-65,5.785253e-24,1.205449e-03,3.721476e-02
rate_of_sales_increase,NaN,3.472486e-03,NaN,5.369179e-05,1.239634e-02,NaN,1.943842e-04,4.650528e-14,0.000000e+00,1.425320e-04,NaN,7.195477e-04,NaN,NaN,5.265175e-11,NaN,NaN,1.301950e-04,1.630404e-03
pre_1y_opincome,3.573170e-42,NaN,2.166173e-10,5.434802e-50,2.474500e-21,2.363744e-09,1.487098e-02,3.297272e-07,1.425320e-04,0.000000e+00,9.004483e-06,8.628362e-320,6.820112e-08,3.282361e-71,4.992054e-11,4.252971e-66,1.964313e-06,4.829347e-06,1.521190e-06


In [45]:
stup_sp_pvals[abs(stup_sp_pvals)<0.05].to_excel('C:/플랫폼 SME 연구센터/투자여부예측/스타트업기준_상관관계행렬_pval_250418_종속변수변환X.xlsx')

In [39]:
stup_sp_corr.to_excel('C:/플랫폼 SME 연구센터/투자여부예측/스타트업기준_상관관계행렬_가설3_250310.xlsx')

In [21]:
stup_sp_corr[abs(stup_sp_corr)>0.5]

,inv_amounts,inv_dur,Patent_total,Patent_above_B-,Patent_Class_C,Patent_Class_B-,Patent_Class_B0,Patent_Class_B+,Patent_Class_A-,Patent_Class_A0,Patent_Class_A+,Patent_Class_AA,ROE,ROA,ratio_of_netincome_to_sales,equity_ratio,Debt_ratio,pre_2y_sales,pre_1y_sales,rate_of_sales_increase,pre_2y_opincome,pre_1y_opincome,rate_of_opincome_increase,pre_2y_netincome,pre_1y_netincome,rate_of_netincome_increase,pre_2y_assets,pre_1y_assets,rate_of_assets_increase,pre_1y_liabilities,pre_1y_equity
inv_amounts,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
inv_dur,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Patent_total,NaN,NaN,1.000000,0.959431,0.579032,0.679280,0.898577,0.830454,0.808380,0.776962,0.770996,0.754513,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Patent_above_B-,NaN,NaN,0.959431,1.000000,NaN,0.539953,0.963305,0.881710,0.907606,0.868649,0.862400,0.853234,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Patent_Class_C,NaN,NaN,0.579032,NaN,1.000000,0.716530,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Patent_Class_B-,NaN,NaN,0.679280,0.539953,0.716530,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Patent_Class_B0,NaN,NaN,0.898577,0.963305,NaN,NaN,1.000000,0.800956,0.901377,0.846102,0.823843,0.868337,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Patent_Class_B+,NaN,NaN,0.830454,0.881710,NaN,NaN,0.800956,1.000000,0.803594,0.757915,0.750977,0.727794,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Patent_Class_A-,NaN,NaN,0.808380,0.907606,NaN,NaN,0.901377,0.803594,1.000000,0.887721,0.847855,0.904692,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Patent_Class_A0,NaN,NaN,0.776962,0.868649,NaN,NaN,0.846102,0.757915,0.887721,1.000000,0.858825,0.861865,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
# pearson 상관계수 p-value 계산
#for i in range(len(conti_var)):
#    j=conti_var[i]
#    for l in range(len(conti_var)):
#        k=conti_var[l]
#        pearson_corr, p_val = pearsonr(stup[j], stup[k])
#        print(j,",",k)
#        print(f'Pearson correlation: {pearson_corr}')
#        print(f'p-value: {p_val}')
#        print(" ")

inv_amounts , inv_amounts
Pearson correlation: 1.0
p-value: 0.0
 
inv_amounts , inv_dur
Pearson correlation: 0.005207317487664932
p-value: 0.8804265338015852
 
inv_amounts , Patent_total
Pearson correlation: 0.05399698917167599
p-value: 0.11852643646176678
 
inv_amounts , Patent_above_B-
Pearson correlation: 0.058232412565738974
p-value: 0.09225314483509446
 
inv_amounts , Patent_Class_C
Pearson correlation: 0.012683634220436935
p-value: 0.7140547083616121
 
inv_amounts , Patent_Class_B-
Pearson correlation: -0.00971984050154679
p-value: 0.7788709930003164
 
inv_amounts , Patent_Class_B0
Pearson correlation: 0.07528887052482022
p-value: 0.0294048881355685
 
inv_amounts , Patent_Class_B+
Pearson correlation: 0.08392124586206802
p-value: 0.015158148916464748
 
inv_amounts , Patent_Class_A-
Pearson correlation: 0.04851900186355439
p-value: 0.16078712761882022
 
inv_amounts , Patent_Class_A0
Pearson correlation: 0.041088620398970814
p-value: 0.2350470561094695
 
inv_amounts , Patent_Class_

Patent_Class_B0 , Patent_total
Pearson correlation: 0.8985774740832246
p-value: 6.197751596381888e-301
 
Patent_Class_B0 , Patent_above_B-
Pearson correlation: 0.9633053698223006
p-value: 0.0
 
Patent_Class_B0 , Patent_Class_C
Pearson correlation: 0.2277130294886685
p-value: 2.6309426417223667e-11
 
Patent_Class_B0 , Patent_Class_B-
Pearson correlation: 0.40138540304744913
p-value: 9.583776641353863e-34
 
Patent_Class_B0 , Patent_Class_B0
Pearson correlation: 1.0
p-value: 0.0
 
Patent_Class_B0 , Patent_Class_B+
Pearson correlation: 0.8009557955589734
p-value: 3.319136484062429e-188
 
Patent_Class_B0 , Patent_Class_A-
Pearson correlation: 0.9013771472699545
p-value: 9.606712286419902e-306
 
Patent_Class_B0 , Patent_Class_A0
Pearson correlation: 0.8461018574418623
p-value: 2.213153936011112e-230
 
Patent_Class_B0 , Patent_Class_A+
Pearson correlation: 0.8238434823608798
p-value: 4.465802337177525e-208
 
Patent_Class_B0 , Patent_Class_AA
Pearson correlation: 0.8683367731478591
p-value: 1.

ROE , Patent_above_B-
Pearson correlation: 0.03130958057622741
p-value: 0.3656327784675998
 
ROE , Patent_Class_C
Pearson correlation: 0.037914783918671854
p-value: 0.27322519158006175
 
ROE , Patent_Class_B-
Pearson correlation: 0.033451200305296545
p-value: 0.33374438479909174
 
ROE , Patent_Class_B0
Pearson correlation: 0.0255574941295433
p-value: 0.460260325197416
 
ROE , Patent_Class_B+
Pearson correlation: 0.03758394800482822
p-value: 0.27743556431153604
 
ROE , Patent_Class_A-
Pearson correlation: 0.017325618263323423
p-value: 0.6166983380742312
 
ROE , Patent_Class_A0
Pearson correlation: 0.01364430918015909
p-value: 0.6934545878714274
 
ROE , Patent_Class_A+
Pearson correlation: 0.026192036877853053
p-value: 0.44919488562914023
 
ROE , Patent_Class_AA
Pearson correlation: 0.020859397999185535
p-value: 0.5467442517581258
 
ROE , ROE
Pearson correlation: 1.0
p-value: 0.0
 
ROE , ROA
Pearson correlation: 0.04870843251557204
p-value: 0.1591573927293466
 
ROE , ratio_of_netincome_t

pre_2y_sales , equity_ratio
Pearson correlation: -0.0071048136426780425
p-value: 0.8373811574541298
 
pre_2y_sales , Debt_ratio
Pearson correlation: 0.016553838138668255
p-value: 0.6324827067226568
 
pre_2y_sales , pre_2y_sales
Pearson correlation: 0.9999999999999939
p-value: 0.0
 
pre_2y_sales , pre_1y_sales
Pearson correlation: 0.5553781663403117
p-value: 6.651268692780059e-69
 
pre_2y_sales , rate_of_sales_increase
Pearson correlation: -0.012143472461392009
p-value: 0.7257308033650203
 
pre_2y_sales , pre_2y_opincome
Pearson correlation: -0.5359431517740845
p-value: 2.068529991588067e-63
 
pre_2y_sales , pre_1y_opincome
Pearson correlation: 0.05949777644339108
p-value: 0.08538172958760555
 
pre_2y_sales , rate_of_opincome_increase
Pearson correlation: 0.02006705461273591
p-value: 0.5620833841766982
 
pre_2y_sales , pre_2y_netincome
Pearson correlation: -0.8772420937735816
p-value: 2.3472825791848627e-268
 
pre_2y_sales , pre_1y_netincome
Pearson correlation: -0.02544089482416189
p-v

pre_2y_netincome , pre_2y_opincome
Pearson correlation: 0.7324287383630086
p-value: 1.4803088598407087e-141
 
pre_2y_netincome , pre_1y_opincome
Pearson correlation: 0.13668701705735295
p-value: 7.271897336964971e-05
 
pre_2y_netincome , rate_of_opincome_increase
Pearson correlation: -0.016261127227393167
p-value: 0.6385137126124313
 
pre_2y_netincome , pre_2y_netincome
Pearson correlation: 0.999999999999999
p-value: 0.0
 
pre_2y_netincome , pre_1y_netincome
Pearson correlation: 0.20513011452715543
p-value: 2.1042802138770832e-09
 
pre_2y_netincome , rate_of_netincome_increase
Pearson correlation: -0.012195472770141382
p-value: 0.724603945820174
 
pre_2y_netincome , pre_2y_assets
Pearson correlation: -0.11149712911122586
p-value: 0.0012336801854141554
 
pre_2y_netincome , pre_1y_assets
Pearson correlation: -0.057198685459877954
p-value: 0.0981898210606965
 
pre_2y_netincome , rate_of_assets_increase
Pearson correlation: 0.01088933211712425
p-value: 0.7530826448247386
 
pre_2y_netincome

pre_1y_equity , Patent_Class_A+
Pearson correlation: -0.04850418117636868
p-value: 0.16091516387059532
 
pre_1y_equity , Patent_Class_AA
Pearson correlation: -0.041464650877804314
p-value: 0.23078683705435651
 
pre_1y_equity , ROE
Pearson correlation: 0.0074765425621788235
p-value: 0.8290007972381996
 
pre_1y_equity , ROA
Pearson correlation: 0.10998910699835099
p-value: 0.0014375274852910132
 
pre_1y_equity , ratio_of_netincome_to_sales
Pearson correlation: 0.013677237615694053
p-value: 0.6927523806540262
 
pre_1y_equity , equity_ratio
Pearson correlation: 0.08304200960555319
p-value: 0.016259158772977577
 
pre_1y_equity , Debt_ratio
Pearson correlation: 0.013601859250382667
p-value: 0.6943602312097087
 
pre_1y_equity , pre_2y_sales
Pearson correlation: 0.2124174591431675
p-value: 5.392797498793046e-10
 
pre_1y_equity , pre_1y_sales
Pearson correlation: 0.8735698843401383
p-value: 2.3003222132964393e-263
 
pre_1y_equity , rate_of_sales_increase
Pearson correlation: -0.0015876422527801

### 투자자

In [65]:
#invst_1_corr = invst_1_conti_var[conti_var].corr(method='pearson')
#invst_1_corr

,inv_amounts,inv_dur,Patent_total,Patent_above_B_minus,Patent_Class_C,Patent_Class_B_minus,Patent_Class_B0,Patent_Class_B_plus,Patent_Class_A_minus,Patent_Class_A0,Patent_Class_A_plus,Patent_Class_AA,ROE,ROA,ratio_of_netincome_to_sales,equity_ratio,Debt_ratio,pre_2y_sales,pre_1y_sales,rate_of_sales_increase,pre_2y_opincome,pre_1y_opincome,rate_of_opincome_increase,pre_2y_netincome,pre_1y_netincome,rate_of_netincome_increase,pre_2y_assets,pre_1y_assets,rate_of_assets_increase,pre_1y_liabilities,pre_1y_equity
inv_amounts,1.000000,0.009051,0.040302,0.033610,0.037372,0.001108,0.055690,0.071083,-0.000051,0.014887,-0.014622,0.028278,0.003461,0.099222,0.002117,-0.010668,0.026841,0.241044,0.280330,-0.008623,-0.501745,-0.470315,0.031158,-0.224694,-0.483716,0.013696,0.658548,0.385635,0.024521,0.361105,0.603618
inv_dur,0.009051,1.000000,0.168219,0.144705,0.144039,0.182932,0.107356,0.128268,0.092030,0.090279,0.095027,0.066507,0.016750,-0.094588,-0.005131,-0.129101,-0.022792,-0.022810,-0.036042,-0.034603,0.004180,0.060202,0.081026,0.006269,0.053655,-0.002923,0.020071,-0.113681,-0.055400,-0.111037,-0.104646
Patent_total,0.040302,0.168219,1.000000,0.955187,0.599453,0.709466,0.889963,0.827479,0.757546,0.795413,0.713887,0.695632,0.038236,-0.113163,0.010278,-0.248904,-0.047090,-0.033111,-0.046473,-0.029394,-0.066305,-0.044583,0.041149,-0.028813,-0.085166,0.035929,0.028842,-0.039184,-0.054572,-0.035154,-0.085890
Patent_above_B_minus,0.033610,0.144705,0.955187,1.000000,0.335667,0.615278,0.951696,0.874668,0.857107,0.871254,0.816426,0.788855,0.031208,-0.125982,0.008466,-0.268581,-0.038094,-0.022100,-0.031471,-0.020040,-0.080284,-0.064271,0.036259,-0.036054,-0.108689,0.028669,0.033464,-0.030799,-0.043953,-0.026189,-0.090557
Patent_Class_C,0.037372,0.144039,0.599453,0.335667,1.000000,0.594011,0.258717,0.268161,0.093097,0.175349,0.064163,0.080622,0.037291,-0.019460,0.009815,-0.065838,-0.046848,-0.045610,-0.062795,-0.039352,0.006089,0.031915,0.032905,0.005800,0.022873,0.036816,0.001297,-0.041415,-0.054815,-0.041056,-0.028459
Patent_Class_B_minus,0.001108,0.182932,0.709466,0.615278,0.594011,1.000000,0.448745,0.536928,0.222758,0.336870,0.253985,0.168634,0.033943,-0.164651,0.004991,-0.253253,-0.043069,-0.017482,-0.031126,-0.010231,-0.029047,-0.009945,0.037567,-0.038773,-0.099226,0.036589,0.025935,-0.039551,-0.054148,-0.036650,-0.068057
Patent_Class_B0,0.055690,0.107356,0.889963,0.951696,0.258717,0.448745,1.000000,0.773482,0.852314,0.838163,0.786141,0.804033,0.025337,-0.091696,0.003221,-0.232717,-0.031470,-0.013972,-0.021304,-0.028585,-0.098222,-0.082634,0.033377,-0.036382,-0.116565,0.025472,0.049790,-0.024073,-0.038708,-0.021560,-0.053369
Patent_Class_B_plus,0.071083,0.128268,0.827479,0.874668,0.268161,0.536928,0.773482,1.000000,0.714116,0.731415,0.677118,0.630870,0.034909,-0.112286,0.015937,-0.260106,-0.041131,-0.014478,-0.023780,-0.015376,-0.079363,-0.053889,0.037079,-0.038189,-0.128821,0.023659,0.066315,-0.023375,-0.040381,-0.020888,-0.052568
Patent_Class_A_minus,-0.000051,0.092030,0.757546,0.857107,0.093097,0.222758,0.852314,0.714116,1.000000,0.831879,0.811210,0.840154,0.017976,-0.086195,0.005035,-0.204383,-0.021221,-0.024234,-0.027095,-0.017646,-0.059435,-0.057045,0.022190,-0.010921,-0.035759,0.016330,-0.005690,-0.020287,-0.023737,-0.014544,-0.102891
Patent_Class_A0,0.014887,0.090279,0.795413,0.871254,0.175349,0.336870,0.838163,0.731415,0.831879,1.000000,0.802973,0.820987,0.014867,-0.111405,0.011031,-0.228841,-0.018371,-0.027959,-0.032741,0.008548,-0.071548,-0.068900,0.018693,-0.028539,-0.086424,0.013695,-0.003783,-0.019438,-0.022516,-0.011862,-0.131716


In [227]:
# Spearman 상관계수 & p-value를 데이터프레임 전체에 적용하는 함수
def spearman_corr_pval(df):
    cols = df.columns
    corr_matrix = pd.DataFrame(index=cols, columns=cols, dtype=float)
    pval_matrix = pd.DataFrame(index=cols, columns=cols, dtype=float)
    
    for col1 in cols:
        for col2 in cols:
            corr, pval = spearmanr(df[col1], df[col2])
            corr_matrix.loc[col1, col2] = corr
            pval_matrix.loc[col1, col2] = pval
    
    return corr_matrix, pval_matrix

In [160]:
#invst_1 = invst_transformed
#invst_1['Case_1'] = invst['Case_1']

In [162]:
#invst_1_conti_var = invst_1[invst_1['Case_1'].isin([1, 3])]

In [228]:
# 실행
invst_sp_corr, invst_sp_pvals = spearman_corr_pval(invst_h4_1_bc_conti)

In [229]:
invst_sp_corr

,inv_amounts,inv_dur,Patent_total,Patent_above_B_minus,ROE,ROA,ratio_of_netincome_to_sales,equity_ratio,Debt_ratio,pre_1y_sales,rate_of_sales_increase,pre_1y_opincome,rate_of_opincome_increase,pre_1y_netincome,rate_of_netincome_increase,pre_1y_assets,rate_of_assets_increase,pre_1y_liabilities,pre_1y_equity
inv_amounts,1.000000,0.070834,0.006524,0.022716,0.025553,0.054453,0.019086,-0.058742,0.028579,0.290400,0.089117,-0.505655,-0.032662,-0.505795,-0.057753,0.592954,0.090037,0.421337,0.330840
inv_dur,0.070834,1.000000,0.194741,0.199504,0.045980,-0.105233,-0.039874,-0.031515,-0.115470,-0.047251,-0.123170,-0.071504,0.256587,-0.061330,0.262030,-0.028572,-0.428891,0.029616,-0.084714
Patent_total,0.006524,0.194741,1.000000,0.923859,0.149478,-0.070203,-0.289579,-0.012250,-0.215942,-0.205421,-0.150506,-0.125324,0.109295,-0.133551,0.070742,0.069910,-0.167069,0.078469,-0.046696
Patent_above_B_minus,0.022716,0.199504,0.923859,1.000000,0.125187,-0.087446,-0.240529,-0.046434,-0.175304,-0.162033,-0.126697,-0.124448,0.115807,-0.151453,0.062959,0.074833,-0.202054,0.110364,-0.057639
ROE,0.025553,0.045980,0.149478,0.125187,1.000000,0.286776,0.035537,-0.125695,-0.692197,-0.131277,-0.058175,0.179743,0.214590,0.111985,0.142196,0.121213,0.071276,0.137937,-0.186514
ROA,0.054453,-0.105233,-0.070203,-0.087446,0.286776,1.000000,0.461877,0.332894,0.192158,0.205891,0.146893,0.378597,0.167044,0.478275,0.302912,0.230491,0.375769,-0.095933,0.503654
ratio_of_netincome_to_sales,0.019086,-0.039874,-0.289579,-0.240529,0.035537,0.461877,1.000000,-0.036053,0.320912,0.535431,0.089506,0.189911,0.139288,0.274389,0.244022,0.098787,0.134414,0.094238,0.194672
equity_ratio,-0.058742,-0.031515,-0.012250,-0.046434,-0.125695,0.332894,-0.036053,1.000000,-0.165564,-0.284304,-0.027400,0.240945,-0.006001,0.365762,0.059865,-0.123369,0.149917,-0.742700,0.545457
Debt_ratio,0.028579,-0.115470,-0.215942,-0.175304,-0.692197,0.192158,0.320912,-0.165564,1.000000,0.478518,0.212369,-0.016736,-0.086118,0.040672,0.037161,0.081751,0.081224,0.190375,0.289007
pre_1y_sales,0.290400,-0.047251,-0.205421,-0.162033,-0.131277,0.205891,0.535431,-0.284304,0.478518,1.000000,0.374518,-0.239697,0.048059,-0.232253,0.118159,0.456419,0.083338,0.532470,0.307761


In [230]:
invst_sp_corr.to_excel('C:/플랫폼 SME 연구센터/투자여부예측/투자자기준_가설2_a,b_상관관계행렬_250406.xlsx')

In [231]:
invst_sp_pvals[abs(invst_sp_pvals)<0.05].to_excel('C:/플랫폼 SME 연구센터/투자여부예측/투자자기준_가설2_a,b_상관관계행렬_pval_250406.xlsx')

In [232]:
invst_sp_corr[abs(invst_sp_corr)>0.5]

,inv_amounts,inv_dur,Patent_total,Patent_above_B_minus,ROE,ROA,ratio_of_netincome_to_sales,equity_ratio,Debt_ratio,pre_1y_sales,rate_of_sales_increase,pre_1y_opincome,rate_of_opincome_increase,pre_1y_netincome,rate_of_netincome_increase,pre_1y_assets,rate_of_assets_increase,pre_1y_liabilities,pre_1y_equity
inv_amounts,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.505655,NaN,-0.505795,NaN,0.592954,NaN,NaN,NaN
inv_dur,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Patent_total,NaN,NaN,1.000000,0.923859,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Patent_above_B_minus,NaN,NaN,0.923859,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ROE,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,-0.692197,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ROA,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.503654
ratio_of_netincome_to_sales,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,0.535431,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
equity_ratio,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.742700,0.545457
Debt_ratio,NaN,NaN,NaN,NaN,-0.692197,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
pre_1y_sales,NaN,NaN,NaN,NaN,NaN,NaN,0.535431,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.532470,NaN


In [233]:
invst_sp_pvals[abs(invst_sp_pvals)<0.05]

,inv_amounts,inv_dur,Patent_total,Patent_above_B_minus,ROE,ROA,ratio_of_netincome_to_sales,equity_ratio,Debt_ratio,pre_1y_sales,rate_of_sales_increase,pre_1y_opincome,rate_of_opincome_increase,pre_1y_netincome,rate_of_netincome_increase,pre_1y_assets,rate_of_assets_increase,pre_1y_liabilities,pre_1y_equity
inv_amounts,0.000000e+00,1.325892e-02,NaN,NaN,NaN,NaN,NaN,4.005960e-02,NaN,3.565466e-25,1.819077e-03,2.658723e-80,NaN,2.366634e-80,4.354113e-02,6.478590e-117,1.629230e-03,9.063687e-54,1.337362e-32
inv_dur,1.325892e-02,0.000000e+00,6.582118e-12,1.952758e-12,NaN,2.286068e-04,NaN,NaN,5.211754e-05,NaN,1.576607e-05,1.241144e-02,7.986138e-20,3.205425e-02,1.235351e-20,NaN,7.460449e-56,NaN,3.040118e-03
Patent_total,NaN,6.582118e-12,0.000000e+00,0.000000e+00,1.525234e-07,1.410421e-02,4.909102e-25,NaN,2.325621e-14,4.135798e-13,1.250229e-07,1.113798e-05,1.290854e-04,2.802620e-06,1.337993e-02,1.451128e-02,4.212337e-09,6.060757e-03,NaN
Patent_above_B_minus,NaN,1.952758e-12,0.000000e+00,0.000000e+00,1.138938e-05,2.216335e-03,1.527490e-17,NaN,6.842772e-10,1.225924e-08,8.898002e-06,1.283626e-05,4.953671e-05,1.039626e-07,2.775021e-02,8.871924e-03,1.006132e-12,1.106913e-04,4.395470e-02
ROE,NaN,NaN,1.525234e-07,1.138938e-05,0.000000e+00,1.451945e-24,NaN,1.048426e-05,5.238765e-175,4.138599e-06,4.202650e-02,2.476230e-10,3.395218e-14,8.744949e-05,6.004636e-07,2.151224e-05,1.269460e-02,1.297672e-06,4.995843e-11
ROA,NaN,2.286068e-04,1.410421e-02,2.216335e-03,1.451945e-24,0.000000e+00,1.337350e-65,5.221115e-33,1.255990e-11,3.648692e-13,2.500129e-07,6.208061e-43,4.235100e-09,7.375795e-71,2.385711e-27,3.373279e-16,2.852006e-42,7.854599e-04,1.393931e-79
ratio_of_netincome_to_sales,NaN,NaN,4.909102e-25,1.527490e-17,NaN,1.337350e-65,0.000000e+00,NaN,1.140548e-30,1.349101e-91,1.736406e-03,2.187288e-11,1.018839e-06,1.512250e-22,5.029296e-18,5.437533e-04,2.413175e-06,9.728799e-04,6.697666e-12
equity_ratio,4.005960e-02,NaN,NaN,NaN,1.048426e-05,5.221115e-33,NaN,0.000000e+00,5.817093e-09,3.739678e-24,NaN,1.339527e-17,NaN,5.568799e-40,3.639838e-02,1.526991e-05,1.401336e-07,1.003524e-214,1.165418e-95
Debt_ratio,NaN,5.211754e-05,2.325621e-14,6.842772e-10,5.238765e-175,1.255990e-11,1.140548e-30,5.817093e-09,0.000000e+00,6.136898e-71,6.287617e-14,NaN,2.587084e-03,NaN,NaN,4.241346e-03,4.494928e-03,1.951319e-11,6.130478e-25
pre_1y_sales,3.565466e-25,NaN,4.135798e-13,1.225924e-08,4.138599e-06,3.648692e-13,1.349101e-91,3.739678e-24,6.136898e-71,0.000000e+00,5.569142e-42,1.985077e-17,NaN,1.979984e-16,3.461099e-05,6.515684e-64,3.552947e-03,2.016286e-90,3.203089e-28


In [67]:
# pearson 상관계수 p-value 계산
#for i in range(len(conti_var)):
#    j=conti_var[i]
#    for l in range(len(conti_var)):
#        k=conti_var[l]
#        pearson_corr, p_val = pearsonr(invst_1_conti_var[conti_var][j], invst_1_conti_var[conti_var][k])
#        if p_val < 0.05:
#            print(j,",",k)
#            print(f'Pearson correlation: {pearson_corr}')
#            print(f'p-value: {p_val}')
#            print(" ")
#        else:
#            pass

inv_amounts , inv_amounts
Pearson correlation: 1.0
p-value: 0.0
 
inv_amounts , Patent_Class_B_plus
Pearson correlation: 0.07108276262143445
p-value: 0.012938780249612225
 
inv_amounts , ROA
Pearson correlation: 0.09922215374681456
p-value: 0.0005136306715875166
 
inv_amounts , pre_2y_sales
Pearson correlation: 0.24104364939856265
p-value: 1.2982115912395964e-17
 
inv_amounts , pre_1y_sales
Pearson correlation: 0.2803296858843983
p-value: 1.6777480697468854e-23
 
inv_amounts , pre_2y_opincome
Pearson correlation: -0.5017454449195693
p-value: 6.70153643078238e-79
 
inv_amounts , pre_1y_opincome
Pearson correlation: -0.4703147746718488
p-value: 2.8591056951194983e-68
 
inv_amounts , pre_2y_netincome
Pearson correlation: -0.22469360078104306
p-value: 1.8885743588585382e-15
 
inv_amounts , pre_1y_netincome
Pearson correlation: -0.48371638563867775
p-value: 1.145137593130086e-72
 
inv_amounts , pre_2y_assets
Pearson correlation: 0.6585484118901657
p-value: 8.027260593135153e-153
 
inv_amoun

Patent_Class_B0 , ROA
Pearson correlation: -0.09169623007078799
p-value: 0.0013323559509349414
 
Patent_Class_B0 , equity_ratio
Pearson correlation: -0.232717444737364
p-value: 1.7192970111335213e-16
 
Patent_Class_B0 , pre_2y_opincome
Pearson correlation: -0.09822235917114372
p-value: 0.0005852196191305695
 
Patent_Class_B0 , pre_1y_opincome
Pearson correlation: -0.08263379974866725
p-value: 0.0038447304175800996
 
Patent_Class_B0 , pre_1y_netincome
Pearson correlation: -0.11656478487363153
p-value: 4.4166215844454516e-05
 
Patent_Class_B_plus , inv_amounts
Pearson correlation: 0.07108276262143445
p-value: 0.012938780249612225
 
Patent_Class_B_plus , inv_dur
Pearson correlation: 0.1282679635339219
p-value: 6.863417241752405e-06
 
Patent_Class_B_plus , Patent_total
Pearson correlation: 0.8274788139257561
p-value: 4.411337608780117e-308
 
Patent_Class_B_plus , Patent_above_B_minus
Pearson correlation: 0.8746677431914816
p-value: 0.0
 
Patent_Class_B_plus , Patent_Class_C
Pearson correla

ROA , Patent_Class_B_minus
Pearson correlation: -0.16465114809611003
p-value: 7.0640753269865926e-09
 
ROA , Patent_Class_B0
Pearson correlation: -0.09169623007078799
p-value: 0.0013323559509349414
 
ROA , Patent_Class_B_plus
Pearson correlation: -0.11228593042487109
p-value: 8.367568741921911e-05
 
ROA , Patent_Class_A_minus
Pearson correlation: -0.08619547056761816
p-value: 0.002564075851609886
 
ROA , Patent_Class_A0
Pearson correlation: -0.11140529208663993
p-value: 9.517430986290654e-05
 
ROA , ROA
Pearson correlation: 1.0
p-value: 0.0
 
ROA , equity_ratio
Pearson correlation: 0.6631718126031754
p-value: 1.0650635177485023e-155
 
ROA , pre_1y_netincome
Pearson correlation: 0.1640971134212164
p-value: 7.943887969319686e-09
 
ROA , rate_of_netincome_increase
Pearson correlation: 0.36120652328070196
p-value: 5.782556250211555e-39
 
ROA , pre_2y_assets
Pearson correlation: 0.058664926954180324
p-value: 0.04032307091104634
 
ROA , pre_1y_assets
Pearson correlation: 0.06692753656197561


pre_2y_netincome , pre_1y_sales
Pearson correlation: -0.9003813342032316
p-value: 0.0
 
pre_2y_netincome , pre_2y_opincome
Pearson correlation: 0.8013950129627617
p-value: 1.4128017381089308e-274
 
pre_2y_netincome , pre_1y_opincome
Pearson correlation: 0.6501350378413426
p-value: 1.0264677262005035e-147
 
pre_2y_netincome , pre_2y_netincome
Pearson correlation: 0.9999999999999984
p-value: 0.0
 
pre_2y_netincome , pre_1y_netincome
Pearson correlation: 0.5085634323267236
p-value: 2.3455830533990755e-81
 
pre_2y_netincome , pre_2y_assets
Pearson correlation: -0.43550559054167093
p-value: 1.0081783964262092e-57
 
pre_2y_netincome , pre_1y_assets
Pearson correlation: -0.07276619435410446
p-value: 0.010944472569996398
 
pre_2y_netincome , pre_1y_liabilities
Pearson correlation: -0.06041156741218852
p-value: 0.03472233871004984
 
pre_2y_netincome , pre_1y_equity
Pearson correlation: -0.23734583129238274
p-value: 4.1396484129056956e-17
 
pre_1y_netincome , inv_amounts
Pearson correlation: -0.

pre_1y_equity , Patent_Class_A0
Pearson correlation: -0.1317157401120289
p-value: 3.84051835576907e-06
 
pre_1y_equity , Patent_Class_A_plus
Pearson correlation: -0.10157508236047344
p-value: 0.00037605780202428965
 
pre_1y_equity , Patent_Class_AA
Pearson correlation: -0.09074657547436149
p-value: 0.0014955481129479025
 
pre_1y_equity , ROA
Pearson correlation: 0.22152109816211257
p-value: 4.751079084061008e-15
 
pre_1y_equity , equity_ratio
Pearson correlation: 0.23655437616403485
p-value: 5.2923519705095314e-17
 
pre_1y_equity , pre_2y_sales
Pearson correlation: 0.3028819152373659
p-value: 2.414931530696399e-27
 
pre_1y_equity , pre_1y_sales
Pearson correlation: 0.3044412337023594
p-value: 1.271590341511872e-27
 
pre_1y_equity , pre_2y_opincome
Pearson correlation: -0.35137217334234677
p-value: 7.951528460878019e-37
 
pre_1y_equity , pre_1y_opincome
Pearson correlation: -0.37168631891248805
p-value: 2.508489138047263e-41
 
pre_1y_equity , pre_2y_netincome
Pearson correlation: -0.237

## T-test(명목형-연속형)

### 스타트업

In [52]:
stup_1

,inv_amounts,inv_dur,Patent_total,Patent_above_B_minus,ROE,ROA,ratio_of_netincome_to_sales,equity_ratio,Debt_ratio,pre_1y_sales,rate_of_sales_increase,pre_1y_opincome,rate_of_opincome_increase,pre_1y_netincome,rate_of_netincome_increase,pre_1y_assets,rate_of_assets_increase,pre_1y_liabilities,pre_1y_equity
0,55.0,16,1,1,0.067707,0.015323,0.009139,0.226318,3.418554,56.97477,8.509473,-4.03019,-2.736640,0.52070,1.826154,33.98093,3.360930,26.29042,7.69051
1,140.0,6,1,1,0.067707,0.015323,0.009139,0.226318,3.418554,56.97477,8.509473,-4.03019,-2.736640,0.52070,1.826154,33.98093,3.360930,26.29042,7.69051
2,140.0,1,1,1,-0.460074,-0.316364,-0.330277,0.687636,0.454258,107.43747,0.885703,-38.33597,-8.512199,-35.48415,-69.147014,112.16258,2.300751,35.03557,77.12701
3,110.0,7,2,0,-0.686378,-0.301642,-0.270323,0.439469,1.275471,72.82602,0.640748,-27.93240,-0.882821,-19.68655,-0.656980,65.26454,0.420179,36.58277,28.68177
4,250.0,16,2,0,-0.965694,-0.599446,-0.660491,0.620741,0.610977,107.26536,0.472899,-50.75008,-0.816889,-70.84780,-2.598792,118.18878,0.810919,44.82411,73.36467
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
832,65.0,11,15,13,1.408385,-2.209068,-1.000000,-1.568512,-1.637547,0.00000,-1.000000,-71.62066,-0.694481,-107.02011,-1.327062,48.44581,0.086916,124.43365,-75.98784
833,72.0,9,15,13,1.408385,-2.209068,-1.000000,-1.568512,-1.637547,0.00000,-1.000000,-71.62066,-0.694481,-107.02011,-1.327062,48.44581,0.086916,124.43365,-75.98784
834,72.0,1,17,13,0.812943,-1.153496,-1.000000,-1.418915,-1.704764,0.00000,0.000000,-59.00817,0.176101,-105.02185,0.018672,91.04655,0.879348,220.23385,-129.18730
835,22.0,25,19,13,-4.588304,-1.159582,-1.000000,0.252726,2.956862,0.00000,-1.000000,-75.78444,-0.166007,-69.99527,0.457038,60.36252,-0.285557,45.10737,15.25515


In [53]:
stup_3 = stup[stup_nmnl_var]
stup_3['inv_amounts'] = stup['inv_amounts']
stup_3['inv_dur'] = stup['inv_dur']

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_23408\1567415764.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stup_3['inv_amounts'] = stup['inv_amounts']
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_23408\1567415764.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stup_3['inv_dur'] = stup['inv_dur']


In [54]:
stup_3

,early_late,tech_1,cls_cd,cofounder_yn,edu_lvl,prev_exp_relev_yn,prev_job_tenure_5y,edu_bg,startup_exp,inv_amounts,inv_dur
0,0,0,1,1,3,0,2,1,1,55.0,16
1,1,0,1,1,3,0,2,1,1,140.0,6
2,1,0,1,1,3,0,2,1,1,140.0,1
3,1,0,2,0,4,0,2,1,0,110.0,7
4,1,0,2,0,4,0,2,1,0,250.0,16
...,...,...,...,...,...,...,...,...,...,...,...
832,1,1,5,1,5,1,2,1,0,65.0,11
833,1,1,5,1,5,1,2,1,0,72.0,9
834,1,1,5,1,5,1,2,1,0,72.0,1
835,1,1,5,1,5,1,2,1,0,22.0,25


In [23]:
stup_3 = stup_transformed
stup_3['early_late'] = stup['early_late']
stup_3

,inv_amounts,inv_dur,ROE,ROA,ratio_of_netincome_to_sales,equity_ratio,Debt_ratio,pre_1y_sales,rate_of_sales_increase,pre_1y_opincome,rate_of_opincome_increase,pre_1y_netincome,rate_of_netincome_increase,pre_1y_assets,rate_of_assets_increase,pre_1y_liabilities,pre_1y_equity,Patent_total,Patent_above_B_minus,early_late
0,4.025352,2.833213,0.065513,0.015207,0.009098,0.204017,1.485812,4.060008,2.252288,-1.615458,-1.318187,0.419171,1.038917,3.554803,1.472685,3.306536,2.162232,1,1,0
1,4.948760,1.945910,0.065513,0.015207,0.009098,0.204017,1.485812,4.060008,2.252288,-1.615458,-1.318187,0.419171,1.038917,3.554803,1.472685,3.306536,2.162232,1,1,1
2,4.948760,0.693147,-0.378487,-0.274873,-0.285387,0.523329,0.374496,4.686174,0.634300,-3.672139,-2.252575,-3.596878,-4.250593,4.728826,1.194150,3.584507,4.358336,1,1,1
3,4.709530,2.079442,-0.522583,-0.263627,-0.239271,0.364275,0.822187,4.301711,0.495152,-3.364962,-0.632771,-3.029484,-0.504997,4.193655,0.350783,3.626546,3.390533,2,0,1
4,5.525453,2.833213,-0.675845,-0.469657,-0.507113,0.482884,0.476841,4.684585,0.387232,-3.946426,-0.597126,-4.274550,-1.280598,4.780709,0.593834,3.824810,4.308981,2,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
832,4.189655,2.484907,0.878956,-1.165981,-0.693147,-0.943327,-0.969849,0.000000,-0.693147,-4.285249,-0.527376,-4.682317,-0.844607,3.900877,0.083344,4.831777,-4.343647,15,13,1
833,4.290459,2.302585,0.878956,-1.165981,-0.693147,-0.943327,-0.969849,0.000000,-0.693147,-4.285249,-0.527376,-4.682317,-0.844607,3.900877,0.083344,4.831777,-4.343647,15,13,1
834,4.290459,0.693147,0.594951,-0.767093,-0.693147,-0.883319,-0.995015,0.000000,0.000000,-4.094481,0.162205,-4.663645,0.018500,4.522294,0.630925,5.399220,-4.868974,17,13,1
835,3.135494,3.258097,-1.720676,-0.769915,-0.693147,0.225322,1.375451,0.000000,-0.693147,-4.341002,-0.153585,-4.262613,0.376406,4.116799,-0.251192,3.830973,2.788410,19,13,1


In [24]:
stup_3.columns

Index(['inv_amounts', 'inv_dur', 'ROE', 'ROA', 'ratio_of_netincome_to_sales',
       'equity_ratio', 'Debt_ratio', 'pre_1y_sales', 'rate_of_sales_increase',
       'pre_1y_opincome', 'rate_of_opincome_increase', 'pre_1y_netincome',
       'rate_of_netincome_increase', 'pre_1y_assets',
       'rate_of_assets_increase', 'pre_1y_liabilities', 'pre_1y_equity',
       'Patent_total', 'Patent_above_B_minus', 'early_late'],
      dtype='object')

In [87]:
stup_3 = stup[stup_nmnl_var]
stup_3

,early_late,tech_1,cls_cd,cofounder_yn,edu_lvl,prev_exp_relev_yn,prev_job_tenure_5y,edu_bg,startup_exp
0,0,0,1,1,3,0,2,1,1
1,1,0,1,1,3,0,2,1,1
2,1,0,1,1,3,0,2,1,1
3,1,0,2,0,4,0,2,1,0
4,1,0,2,0,4,0,2,1,0
...,...,...,...,...,...,...,...,...,...
832,1,1,5,1,5,1,2,1,0
833,1,1,5,1,5,1,2,1,0
834,1,1,5,1,5,1,2,1,0
835,1,1,5,1,5,1,2,1,0


In [89]:
stup_3['inv_amounts'] = stup_transformed['inv_amounts']
stup_3['inv_dur'] = stup_transformed['inv_dur']

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_35456\1137470801.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stup_3['inv_amounts'] = stup_transformed['inv_amounts']
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_35456\1137470801.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stup_3['inv_dur'] = stup_transformed['inv_dur']


In [144]:
for i in conti_var:
    group_1 = stup_transformed[stup_transformed['inv_round_2']==1][i]
    group_2 = stup_transformed[stup_transformed['inv_round_2']==2][i]
    # 정규성 검정(Shapiro-Wilk Test) - 귀무가설: 데이터가 정규분포를 따른다
    _, p_value_group_1 = stats.shapiro(group_1)
    _, p_value_group_2 = stats.shapiro(group_2)
    print(i)
    print(f'투자초기단계 그룹의 p-value(정규성 검정): {p_value_group_1}')
    print(f'투자후기단계 그룹의 p-value(정규성 검정): {p_value_group_2}')
    # 분산 동질성 검정(Levene's Test) - 귀무가설: 두 그룹의 분산이 같다
    stat, p_value_var = stats.levene(group_1, group_2)
    print(f"Levene's Test p-value: {p_value_var}")
    if p_value_group_1 < 0.05 or p_value_group_2 < 0.05:
        stat, p_value = stats.mannwhitneyu(group_1, group_2)
        print(f'U-통계량: {stat}')
        print(f'p-value: {p_value}')
        print(" ")
    else:
        t_stat, p_value = stats.ttest_ind(group_1, group_2, equal_var=False) # equal_var=False이면 Welch's t-test, True이면 독립 t-test
        print(f't-통계량: {t_stat}')
        print(f'p-value: {p_value}')
        print(" ")

inv_amounts
투자초기단계 그룹의 p-value(정규성 검정): 0.11646144837141037
투자후기단계 그룹의 p-value(정규성 검정): 1.852487366704736e-05
Levene's Test p-value: 0.7280636648902623
U-통계량: 25713.5
p-value: 1.6040620565904232e-44
 
inv_dur
투자초기단계 그룹의 p-value(정규성 검정): 6.43839427993953e-07
투자후기단계 그룹의 p-value(정규성 검정): 1.4695195746719265e-19
Levene's Test p-value: 0.7982530606876456
U-통계량: 56931.0
p-value: 7.503319908192527e-05
 
Patent_total
투자초기단계 그룹의 p-value(정규성 검정): 2.7264711213552395e-13
투자후기단계 그룹의 p-value(정규성 검정): 4.507572517341955e-17
Levene's Test p-value: 3.1355033311292203e-07
U-통계량: 53084.5
p-value: 1.3264306175985983e-07
 
Patent_above_B_minus
투자초기단계 그룹의 p-value(정규성 검정): 8.743483535962102e-18
투자후기단계 그룹의 p-value(정규성 검정): 1.016899413582823e-21
Levene's Test p-value: 4.523487054301776e-12
U-통계량: 50502.0
p-value: 3.55004357299516e-10
 
Patent_Class_C
투자초기단계 그룹의 p-value(정규성 검정): 2.9452172682811926e-17
투자후기단계 그룹의 p-value(정규성 검정): 6.668456319689402e-26
Levene's Test p-value: 0.0186758769788115
U-통계량: 63389.5
p-valu

In [90]:
stup_3.columns

Index(['early_late', 'tech_1', 'cls_cd', 'cofounder_yn', 'edu_lvl',
       'prev_exp_relev_yn', 'prev_job_tenure_5y', 'edu_bg', 'startup_exp',
       'inv_amounts', 'inv_dur'],
      dtype='object')

In [55]:
for i in stup_3.columns[:-2]:
    print(i, stup_3[i].unique())
    print(" ")

early_late [0 1]
 
tech_1 [0 1]
 
cls_cd [ 1  2  3  4  5  6  7  8  9 10 11 12 13]
 
cofounder_yn [1 0]
 
edu_lvl [3 4 0 5 1 2]
 
prev_exp_relev_yn [0 1]
 
prev_job_tenure_5y [2 1 0]
 
edu_bg [1 0]
 
startup_exp [1 0 2]
 


In [56]:
dep_var = ['inv_amounts','inv_dur']

In [25]:
dep_var = ['ROE', 'ROA', 'ratio_of_netincome_to_sales',
       'equity_ratio', 'Debt_ratio', 'pre_1y_sales', 'rate_of_sales_increase',
       'pre_1y_opincome', 'rate_of_opincome_increase', 'pre_1y_netincome',
       'rate_of_netincome_increase', 'pre_1y_assets',
       'rate_of_assets_increase', 'pre_1y_liabilities', 'pre_1y_equity',
       'Patent_total', 'Patent_above_B_minus']

In [57]:
group_cols = ['early_late','tech_1','cofounder_yn', 'prev_exp_relev_yn', 'edu_bg']

In [26]:
group_cols = ['early_late']

In [58]:
t_tests = []

for col in group_cols:
    for i in dep_var:
        group_1 = stup_3[stup_3[col]==0][i]
        group_2 = stup_3[stup_3[col]==1][i]
    # 정규성 검정(Shapiro-Wilk Test) - 귀무가설: 데이터가 정규분포를 따른다
        _, p_value_group_1 = stats.shapiro(group_1)
        _, p_value_group_2 = stats.shapiro(group_2)
        print(col)
        print(i)
        print(f'값이 0인 그룹의 p-value(정규성 검정): {p_value_group_1}')
        print(f'값이 1인 그룹의 p-value(정규성 검정): {p_value_group_2}')
    # 분산 동질성 검정(Levene's Test) - 귀무가설: 두 그룹의 분산이 같다
        stat, p_value_var = stats.levene(group_1, group_2)
        print(f"Levene's Test p-value: {p_value_var}")
        if p_value_group_1 < 0.05 or p_value_group_2 < 0.05:
            stat, p_value = stats.mannwhitneyu(group_1, group_2)
            t_tests.append([i, col, round(stat,3), round(p_value,3)])
            print(f'U-통계량: {stat}')
            print(f'p-value: {p_value}')
            print(" ")
        elif p_value_var > 0.05:
            t_stat, p_value = stats.ttest_ind(group_1, group_2, equal_var=True) # equal_var=False이면 Welch's t-test, True이면 독립 t-test
            t_tests.append([i, col, round(t_stat,3), round(p_value,3)])
            print(f't-통계량: {t_stat}')
            print(f'p-value: {p_value}')
            print(" ")
        else:
            t_stat, p_value = stats.ttest_ind(group_1, group_2, equal_var=False) # equal_var=False이면 Welch's t-test, True이면 독립 t-test
            t_tests.append([i, col, round(t_stat,3), round(p_value,3)])
            print(f't-통계량: {t_stat}')
            print(f'p-value: {p_value}')
            print(" ")

early_late
inv_amounts
값이 0인 그룹의 p-value(정규성 검정): 9.987260683441173e-24
값이 1인 그룹의 p-value(정규성 검정): 3.5646230335494697e-41
Levene's Test p-value: 8.886233225345557e-06
U-통계량: 25713.5
p-value: 1.6040620565904232e-44
 
early_late
inv_dur
값이 0인 그룹의 p-value(정규성 검정): 1.978346670051261e-26
값이 1인 그룹의 p-value(정규성 검정): 5.312815638305586e-17
Levene's Test p-value: 0.34143357412008435
U-통계량: 56931.0
p-value: 7.503319908192527e-05
 
tech_1
inv_amounts
값이 0인 그룹의 p-value(정규성 검정): 1.0963381390343549e-23
값이 1인 그룹의 p-value(정규성 검정): 1.1524278570607296e-41
Levene's Test p-value: 0.0004353554277846005
U-통계량: 57570.0
p-value: 0.2528981088827019
 
tech_1
inv_dur
값이 0인 그룹의 p-value(정규성 검정): 1.264314819644661e-22
값이 1인 그룹의 p-value(정규성 검정): 1.4694530641807984e-18
Levene's Test p-value: 0.791690920741707
U-통계량: 51413.5
p-value: 0.2754894268817598
 
cofounder_yn
inv_amounts
값이 0인 그룹의 p-value(정규성 검정): 1.401298464324817e-45
값이 1인 그룹의 p-value(정규성 검정): 4.1453440390556756e-16
Levene's Test p-value: 0.6861981441031322
U

In [59]:
pd.DataFrame(t_tests, columns=['dep_var','variable', 'U-statistic', 'p-value'])

,dep_var,variable,U-statistic,p-value
0,inv_amounts,early_late,25713.5,0.000
1,inv_dur,early_late,56931.0,0.000
2,inv_amounts,tech_1,57570.0,0.253
3,inv_dur,tech_1,51413.5,0.275
4,inv_amounts,cofounder_yn,25700.0,0.002
5,inv_dur,cofounder_yn,31502.5,0.710
6,inv_amounts,prev_exp_relev_yn,77660.5,0.907
7,inv_dur,prev_exp_relev_yn,77448.0,0.958
8,inv_amounts,edu_bg,55952.5,0.000
9,inv_dur,edu_bg,67577.0,0.681


In [30]:
pd.DataFrame(t_tests, columns=['variable', 'dep_var', 'U-statistic', 'p-value'])[['dep_var', 'variable', 'U-statistic', 'p-value']].to_excel('C:/플랫폼 SME 연구센터/투자여부예측/스타트업기준_초기후기변수_M-WUtest_250407.xlsx')

In [60]:
pd.DataFrame(t_tests, columns=['dep_var','variable', 'U-statistic', 'p-value']).to_excel('C:/플랫폼 SME 연구센터/투자여부예측/스타트업기준_M-WUtest_250418.xlsx')

### 투자자

In [248]:
invst_2 = invst_transformed_1
invst_2['sequantial_inv'] = invst_h4_1_a_conti['sequantial_inv']
invst_2

,ROE,ROA,ratio_of_netincome_to_sales,equity_ratio,Debt_ratio,pre_1y_sales,rate_of_sales_increase,pre_1y_opincome,rate_of_opincome_increase,pre_1y_netincome,rate_of_netincome_increase,pre_1y_assets,rate_of_assets_increase,pre_1y_liabilities,pre_1y_equity,Patent_total,Patent_above_B_minus,sequantial_inv
3,0.065513,0.015207,0.009098,0.204017,1.485812,4.060008,2.252288,-1.615458,-1.318187,0.419171,1.038917,3.554803,1.472685,3.306536,2.162232,1,1,1
4,-1.715532,-1.249428,-0.387923,0.435499,0.605623,3.828055,0.142628,-3.105327,-1.022425,-3.105191,-1.030298,2.258144,-0.537840,1.587382,1.735963,0,0,1
5,-1.715532,-1.249428,-0.387923,0.435499,0.605623,3.828055,0.142628,-3.105327,-1.022425,-3.105191,-1.030298,2.258144,-0.537840,1.587382,1.735963,0,0,1
7,-0.322753,-0.312727,-0.693147,0.674898,0.036837,0.000000,-0.693147,-3.183569,-2.617194,-3.190408,-2.930766,4.166008,2.968777,1.192421,4.129753,2,1,1
8,-0.322753,-0.312727,-0.693147,0.674898,0.036837,0.000000,-0.693147,-3.183569,-2.617194,-3.190408,-2.930766,4.166008,2.968777,1.192421,4.129753,2,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3739,-0.323706,-0.317658,-2.391266,0.682186,0.022044,2.533006,-0.546241,-4.771884,-1.568395,-4.754198,-1.573547,5.732539,1.161614,2.042501,5.710567,3,2,0
3741,0.464838,-0.557797,-3.936881,-0.816268,-1.026890,1.639177,0.693147,-4.887433,-0.411120,-5.345294,-0.238398,5.636016,0.593878,6.450291,-5.868005,9,1,0
3743,0.096925,0.091912,0.449737,0.665704,0.055662,5.088344,3.442950,3.524217,0.792230,4.527214,0.712017,6.858017,-0.029541,3.960045,6.802415,14,7,0
3744,0.472571,-1.397640,-7.008368,-1.798654,-0.787709,0.127689,0.819780,-4.687721,-0.454888,-5.020456,0.038500,3.920159,-0.219621,5.702119,-5.521833,3,1,0


In [340]:
invst_2 = invst_h4_1_bc_nmnl
invst_2

,early_late,tech_1,BoD_yn,cls_cd,cofounder_yn,edu_lvl,prev_exp_relev_yn,prev_job_tenure_5y,edu_bg,startup_exp,inv_amounts,inv_dur
3,0,0,1,1,1,3,0,2,1,1,55.0,16.0
4,0,0,0,1,0,3,1,1,1,1,30.0,19.0
5,0,0,0,1,0,3,1,1,1,1,30.0,19.0
7,0,0,0,1,0,5,0,2,0,0,100.0,8.0
23,0,1,1,3,0,5,0,1,1,0,80.0,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...
3605,1,0,0,4,0,3,0,1,1,0,247.0,50.0
3606,1,0,0,4,0,3,0,1,1,0,247.0,1.0
3607,1,0,0,4,0,3,0,1,1,0,247.0,13.0
3608,1,0,0,4,0,3,0,1,1,0,247.0,7.0


In [341]:
invst_2['inv_amounts'] = invst_transformed_2['inv_amounts']
invst_2['inv_dur'] = invst_transformed_2['inv_dur']
invst_2

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_35456\3314644850.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  invst_2['inv_amounts'] = invst_transformed_2['inv_amounts']
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_35456\3314644850.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  invst_2['inv_dur'] = invst_transformed_2['inv_dur']


,early_late,tech_1,BoD_yn,cls_cd,cofounder_yn,edu_lvl,prev_exp_relev_yn,prev_job_tenure_5y,edu_bg,startup_exp,inv_amounts,inv_dur
3,0,0,1,1,1,3,0,2,1,1,4.025352,2.833213
4,0,0,0,1,0,3,1,1,1,1,3.433987,2.995732
5,0,0,0,1,0,3,1,1,1,1,3.433987,2.995732
7,0,0,0,1,0,5,0,2,0,0,4.615121,2.197225
23,0,1,1,3,0,5,0,1,1,0,4.394449,2.397895
...,...,...,...,...,...,...,...,...,...,...,...,...
3605,1,0,0,4,0,3,0,1,1,0,5.513429,3.931826
3606,1,0,0,4,0,3,0,1,1,0,5.513429,0.693147
3607,1,0,0,4,0,3,0,1,1,0,5.513429,2.639057
3608,1,0,0,4,0,3,0,1,1,0,5.513429,2.079442


In [294]:
invst_2.isnull().sum()

early_late            0
tech_1                0
BoD_yn                0
cofounder_yn          0
edu_lvl               0
prev_exp_relev_yn     0
prev_job_tenure_5y    0
edu_bg                0
startup_exp           0
inv_amounts           0
inv_dur               0
dtype: int64

In [342]:
invst_2.columns

Index(['early_late', 'tech_1', 'BoD_yn', 'cls_cd', 'cofounder_yn', 'edu_lvl',
       'prev_exp_relev_yn', 'prev_job_tenure_5y', 'edu_bg', 'startup_exp',
       'inv_amounts', 'inv_dur'],
      dtype='object')

In [343]:
for i in invst_2.columns[:-2]:
    print(i, invst_2[i].unique())
    print(" ")

early_late [0 1]
 
tech_1 [0 1]
 
BoD_yn [1 0]
 
cls_cd [ 1  3  5  7  4  6 10  8 13  2  9 11 12]
 
cofounder_yn [1 0]
 
edu_lvl [3 5 4 0 1 2]
 
prev_exp_relev_yn [0 1]
 
prev_job_tenure_5y [2 1 0]
 
edu_bg [1 0]
 
startup_exp [1 0 2]
 


In [235]:
invst_2.columns

Index(['Patent_total', 'Patent_above_B_minus', 'ROE', 'ROA',
       'ratio_of_netincome_to_sales', 'equity_ratio', 'Debt_ratio',
       'pre_1y_sales', 'rate_of_sales_increase', 'pre_1y_opincome',
       'rate_of_opincome_increase', 'pre_1y_netincome',
       'rate_of_netincome_increase', 'pre_1y_assets',
       'rate_of_assets_increase', 'pre_1y_liabilities', 'pre_1y_equity',
       'sequantial_inv'],
      dtype='object')

In [250]:
invst_2['sequantial_inv'].value_counts()

sequantial_inv
0    2035
1    1222
Name: count, dtype: int64

In [243]:
dep_var = ['Patent_total', 'Patent_above_B_minus', 'ROE', 'ROA',
       'ratio_of_netincome_to_sales', 'equity_ratio', 'Debt_ratio',
       'pre_1y_sales', 'rate_of_sales_increase', 'pre_1y_opincome',
       'rate_of_opincome_increase', 'pre_1y_netincome',
       'rate_of_netincome_increase', 'pre_1y_assets',
       'rate_of_assets_increase', 'pre_1y_liabilities', 'pre_1y_equity']

In [238]:
group_cols=['sequantial_inv']

In [339]:
dep_var = ['inv_amounts','inv_dur']

In [338]:
group_cols = ['early_late','tech_1', 'BoD_yn', 'cofounder_yn', 'prev_exp_relev_yn', 'edu_bg']

In [300]:
t_tests=[]

for col in group_cols:
    for i in dep_var:
        group_1 = invst_2[invst_2[col]==0][i]
        group_2 = invst_2[invst_2[col]==1][i]
    # 정규성 검정(Shapiro-Wilk Test) - 귀무가설: 데이터가 정규분포를 따른다
        _, p_value_group_1 = stats.shapiro(group_1)
        _, p_value_group_2 = stats.shapiro(group_2)
        print(col)
        print(i)
        print(f'값이 0인 그룹의 p-value(정규성 검정): {p_value_group_1}')
        print(f'값이 1인 그룹의 p-value(정규성 검정): {p_value_group_2}')
    # 분산 동질성 검정(Levene's Test) - 귀무가설: 두 그룹의 분산이 같다
        stat, p_value_var = stats.levene(group_1, group_2)
        print(f"Levene's Test p-value: {p_value_var}")
        if p_value_group_1 < 0.05 or p_value_group_2 < 0.05:
            stat, p_value = stats.mannwhitneyu(group_1, group_2)
            t_tests.append([i, col, round(stat,3), round(p_value,3)])
            print(f'U-통계량: {stat}')
            print(f'p-value: {p_value}')
            print(" ")
        elif p_value_var > 0.05:
            t_stat, p_value = stats.ttest_ind(group_1, group_2, equal_var=True) # equal_var=False이면 Welch's t-test, True이면 독립 t-test
            t_tests.append([i, col, round(t_stat,3), round(p_value,3)])
            print(f't-통계량: {t_stat}')
            print(f'p-value: {p_value}')
            print(" ")
        else:
            t_stat, p_value = stats.ttest_ind(group_1, group_2, equal_var=False) # equal_var=False이면 Welch's t-test, True이면 독립 t-test
            t_tests.append([i, col, round(t_stat,3), round(p_value,3)])
            print(f't-통계량: {t_stat}')
            print(f'p-value: {p_value}')
            print(" ")

early_late
inv_amounts
값이 0인 그룹의 p-value(정규성 검정): 0.03009147383272648
값이 1인 그룹의 p-value(정규성 검정): 2.7067473842548102e-17
Levene's Test p-value: 0.6633728056068278
U-통계량: 40313.0
p-value: 2.8602130286143993e-42
 
early_late
inv_dur
값이 0인 그룹의 p-value(정규성 검정): 8.230771442185869e-08
값이 1인 그룹의 p-value(정규성 검정): 6.863373865348682e-25
Levene's Test p-value: 0.18299557847039416
U-통계량: 73138.5
p-value: 1.1330888345764678e-10
 
tech_1
inv_amounts
값이 0인 그룹의 p-value(정규성 검정): 4.3487222711746654e-08
값이 1인 그룹의 p-value(정규성 검정): 2.4543503340801465e-10
Levene's Test p-value: 0.0012407096032324766
U-통계량: 132294.5
p-value: 0.008484650378260185
 
tech_1
inv_dur
값이 0인 그룹의 p-value(정규성 검정): 4.960141506375878e-15
값이 1인 그룹의 p-value(정규성 검정): 8.721689788976122e-21
Levene's Test p-value: 0.9809036466039892
U-통계량: 95881.0
p-value: 1.986945638095079e-06
 
BoD_yn
inv_amounts
값이 0인 그룹의 p-value(정규성 검정): 3.9749726947480024e-12
값이 1인 그룹의 p-value(정규성 검정): 4.015319063910283e-06
Levene's Test p-value: 0.12205591039271282
U-통계

In [301]:
t_tests

[['inv_amounts', 'early_late', 40313.0, 0.0],
 ['inv_dur', 'early_late', 73138.5, 0.0],
 ['inv_amounts', 'tech_1', 132294.5, 0.008],
 ['inv_dur', 'tech_1', 95881.0, 0.0],
 ['inv_amounts', 'BoD_yn', 89936.0, 0.0],
 ['inv_dur', 'BoD_yn', 94912.0, 0.015],
 ['inv_amounts', 'cofounder_yn', 57174.5, 0.0],
 ['inv_dur', 'cofounder_yn', 72204.0, 0.672],
 ['inv_amounts', 'prev_exp_relev_yn', 185419.5, 0.0],
 ['inv_dur', 'prev_exp_relev_yn', 153101.5, 0.07],
 ['inv_amounts', 'edu_bg', 109388.5, 0.0],
 ['inv_dur', 'edu_bg', 131954.0, 0.397]]

In [302]:
pd.DataFrame(t_tests, columns=['dep_var', 'Variable', 'U-statistic', 'P-value']).to_excel('C:/플랫폼 SME 연구센터/투자여부예측/투자자기준_가설2_a,b_M-WUtest_250406.xlsx')

In [81]:
for i in dep_var:
    group_1 = invst_2[invst_2['Case_1']==1][i]
    group_2 = invst_2[invst_2['Case_1']==3][i]
    # 정규성 검정(Shapiro-Wilk Test) - 귀무가설: 데이터가 정규분포를 따른다
    _, p_value_group_1 = stats.shapiro(group_1)
    _, p_value_group_2 = stats.shapiro(group_2)
    print(col)
    print(i)
    print(f'투자초기단계 그룹의 p-value(정규성 검정): {p_value_group_1}')
    print(f'투자후기단계 그룹의 p-value(정규성 검정): {p_value_group_2}')
    # 분산 동질성 검정(Levene's Test) - 귀무가설: 두 그룹의 분산이 같다
    stat, p_value_var = stats.levene(group_1, group_2)
    print(f"Levene's Test p-value: {p_value_var}")
    if p_value_group_1 < 0.05 or p_value_group_2 < 0.05:
        stat, p_value = stats.mannwhitneyu(group_1, group_2)
        print(f'U-통계량: {stat}')
        print(f'p-value: {p_value}')
        print(" ")
    else:
        t_stat, p_value = stats.ttest_ind(group_1, group_2, equal_var=False) # equal_var=False이면 Welch's t-test, True이면 독립 t-test
        print(f't-통계량: {t_stat}')
        print(f'p-value: {p_value}')
        print(" ")

Case_1
inv_amounts
투자초기단계 그룹의 p-value(정규성 검정): 4.015319063910283e-06
투자후기단계 그룹의 p-value(정규성 검정): 0.8268194198608398
Levene's Test p-value: 0.3476769651632957
U-통계량: 10852.0
p-value: 0.04174793799111432
 
Case_1
inv_dur
투자초기단계 그룹의 p-value(정규성 검정): 1.5910119444129123e-08
투자후기단계 그룹의 p-value(정규성 검정): 2.562831014074618e-07
Levene's Test p-value: 0.7348278385420195
U-통계량: 10483.5
p-value: 0.1332366540233262
 


## ANOVA

In [83]:
#group_col = 'inv_round_1'
# 열별로 ANOVA 수행
#for col in conti_var:
    # 결측치 제거
#    df_temp = stup_1[[group_col, col]].dropna()

    # 그룹마다 데이터가 최소 2개 이상 있는지 체크
    # (원웨이 ANOVA에서는 반드시 필요하지는 않지만,
    #  한 그룹에 표본수가 1 이하이면 문제가 될 수 있으므로 예방 차원에서 권장)
#    group_counts = df_temp.groupby(group_col).size()
#    if any(group_counts < 2):
#        print(f"[경고] {col} 분석 불가: 어떤 그룹의 표본 수가 너무 적습니다. (group_counts={group_counts.to_dict()})")
#        continue

#    formula = f"{col} ~ C({group_col})"
#    try:
#        model = smf.ols(formula, data=df_temp).fit()
#        anova_result = anova_lm(model, typ=2)
    
        # p-value 추출 (C(A5) 행의 "PR(>F)" 열)
#        p_value = anova_result.loc[f"C({group_col})", "PR(>F)"]
    
        # p-value < 0.05만 출력
#        if p_value < 0.05:
#            print(f"=== ANOVA 결과: {col} ===")
#            print(anova_result)
#            print()
#    except Exception as e:
#        print(f"[에러] {col} 분석 중 예외 발생: {e}")
#        print()    

=== ANOVA 결과: inv_amounts ===
                      sum_sq     df          F        PR(>F)
C(inv_round_1)  8.487751e+07   11.0  19.726744  1.524554e-35
Residual        3.226996e+08  825.0        NaN           NaN

=== ANOVA 결과: Patent_total ===
                       sum_sq     df         F    PR(>F)
C(inv_round_1)    9320.384572   11.0  4.429421  0.000002
Residual        157814.943983  825.0       NaN       NaN

=== ANOVA 결과: Patent_above_B_minus ===
                       sum_sq     df         F    PR(>F)
C(inv_round_1)    5538.677854   11.0  3.498371  0.000084
Residual        118741.209840  825.0       NaN       NaN

=== ANOVA 결과: Patent_Class_C ===
                      sum_sq     df         F    PR(>F)
C(inv_round_1)    713.959475   11.0  3.784693  0.000026
Residual        14148.298590  825.0       NaN       NaN

=== ANOVA 결과: Patent_Class_B_minus ===
                     sum_sq     df         F        PR(>F)
C(inv_round_1)   574.896273   11.0  6.893885  3.362059e-11
Residual     

In [140]:
#group_cols = ['cofounder_yn', 'edu_lvl', 'prev_exp_relev_yn', 'prev_job_tenure_5y', 'edu_bg', 'startup_exp', 'prev_mgr_yn', 'prev_c_lvl_yn']
# 열별로 ANOVA 수행
#for group_col in group_cols:
#    for col in dep_var:
        # 결측치 제거
#        df_temp = stup_3[[group_col, col]].dropna()

    # 그룹마다 데이터가 최소 2개 이상 있는지 체크
    # (원웨이 ANOVA에서는 반드시 필요하지는 않지만,
    #  한 그룹에 표본수가 1 이하이면 문제가 될 수 있으므로 예방 차원에서 권장)
#        group_counts = df_temp.groupby(group_col).size()
#        if any(group_counts < 2):
#            print(f"[경고] {col} 분석 불가: 어떤 그룹의 표본 수가 너무 적습니다. (group_counts={group_counts.to_dict()})")
#            continue

#        formula = f"{col} ~ C({group_col})"
#        try:
#            model = smf.ols(formula, data=df_temp).fit()
#            anova_result = anova_lm(model, typ=2)
    
        # p-value 추출 (C(A5) 행의 "PR(>F)" 열)
#            p_value = anova_result.loc[f"C({group_col})", "PR(>F)"]
    
        # p-value < 0.05만 출력
#            if p_value < 0.05:
#                print(f"=== ANOVA 결과: {col} ===")
#                print(anova_result)
#                print()
#        except Exception as e:
#            print(f"[에러] {col} 분석 중 예외 발생: {e}")
#            print() 

=== ANOVA 결과: inv_dur ===
                              sum_sq     df         F    PR(>F)
C(prev_job_tenure_5y)    1658.685751    2.0  4.879285  0.007821
Residual               141756.836352  834.0       NaN       NaN

=== ANOVA 결과: inv_amounts ===
                      sum_sq     df         F    PR(>F)
C(startup_exp)  4.698318e+06    2.0  4.862997  0.007948
Residual        4.028788e+08  834.0       NaN       NaN

=== ANOVA 결과: inv_amounts ===
                      sum_sq     df        F    PR(>F)
C(prev_mgr_yn)  4.524581e+06    1.0  9.37353  0.002272
Residual        4.030526e+08  835.0      NaN       NaN



## Kruskal-Wallis 검정

### 스타트업

In [112]:
dep_var = ['inv_amounts','inv_dur']

In [111]:
stup_3

,early_late,tech_1,cls_cd,cofounder_yn,edu_lvl,prev_exp_relev_yn,prev_job_tenure_5y,edu_bg,startup_exp,inv_amounts,inv_dur
0,0,0,1,1,3,0,2,1,1,4.025352,2.833213
1,1,0,1,1,3,0,2,1,1,4.948760,1.945910
2,1,0,1,1,3,0,2,1,1,4.948760,0.693147
3,1,0,2,0,4,0,2,1,0,4.709530,2.079442
4,1,0,2,0,4,0,2,1,0,5.525453,2.833213
...,...,...,...,...,...,...,...,...,...,...,...
832,1,1,5,1,5,1,2,1,0,4.189655,2.484907
833,1,1,5,1,5,1,2,1,0,4.290459,2.302585
834,1,1,5,1,5,1,2,1,0,4.290459,0.693147
835,1,1,5,1,5,1,2,1,0,3.135494,3.258097


In [123]:
for i in stup_3.columns[:-2]:
    print(i, stup_3[i].unique())

early_late [0 1]
tech_1 [0 1]
cls_cd [ 1  2  3  4  5  6  7  8  9 10 11 12 13]
cofounder_yn [1 0]
edu_lvl [3 4 0 5 1 2]
prev_exp_relev_yn [0 1]
prev_job_tenure_5y [2 1 0]
edu_bg [1 0]
startup_exp [1 0 2]


In [141]:
groups

[0      2.833213
 1      1.945910
 2      0.693147
 6      2.995732
 7      2.995732
          ...   
 783    3.465736
 784    2.890372
 821    2.564949
 822    3.218876
 831    3.433987
 Name: inv_dur, Length: 261, dtype: float64,
 3      2.079442
 4      2.833213
 5      0.693147
 9      2.833213
 10     1.945910
          ...   
 832    2.484907
 833    2.302585
 834    0.693147
 835    3.258097
 836    1.791759
 Name: inv_dur, Length: 568, dtype: float64,
 20     3.218876
 42     1.098612
 43     2.890372
 61     2.944439
 62     0.693147
 384    3.433987
 385    0.693147
 386    1.791759
 Name: inv_dur, dtype: float64]

In [136]:
group_cols = ['cls_cd', 'edu_lvl', 'prev_job_tenure_5y', 'startup_exp']
# Kruskal-Wallis 검정 결과 저장을 위한 딕셔너리
k_w_results = []
kruskal_results = {}

for col in group_cols:
# 각 연속형 변수에 대해 Kruskal-Wallis 검정 수행
    for variable in dep_var:
        # inv_round_1별로 그룹화된 데이터를 리스트로 생성
        groups = [stup_3[stup_3[col] == round_val][variable] for round_val in stup_3[col].unique()]
    
    # Kruskal-Wallis 검정 수행
        h_stat, p_value = kruskal(*groups)
    
    # 결과 저장
        kruskal_results[variable] = {'H-statistic': h_stat, 'p-value': p_value}
        k_w_results.append([variable, col, h_stat, p_value])
        print(col)
        print(f'{variable}: H-statistic = {h_stat}, p-value = {p_value}')
        print(' ')

# 결과 출력: p-value가 0.05 이하인 변수는 유의미한 변수로 간주
    significant_results = {var: result for var, result in kruskal_results.items() if result['p-value'] <= 0.05}

    print("Significant Variables based on Kruskal-Wallis Test:")
    for var, result in significant_results.items():
        print(col)
        print(f"{var}: H-statistic = {result['H-statistic']}, p-value = {result['p-value']}")
        print("------------------------------------------------------------")
        print(' ')

cls_cd
inv_amounts: H-statistic = 38.52488656497807, p-value = 0.00012597738082218237
 
cls_cd
inv_dur: H-statistic = 19.418415814429867, p-value = 0.0789188169328414
 
Significant Variables based on Kruskal-Wallis Test:
cls_cd
inv_amounts: H-statistic = 38.52488656497807, p-value = 0.00012597738082218237
------------------------------------------------------------
 
edu_lvl
inv_amounts: H-statistic = 18.955723504141694, p-value = 0.001958979295438725
 
edu_lvl
inv_dur: H-statistic = 12.352845856506827, p-value = 0.03026012062282995
 
Significant Variables based on Kruskal-Wallis Test:
edu_lvl
inv_amounts: H-statistic = 18.955723504141694, p-value = 0.001958979295438725
------------------------------------------------------------
 
edu_lvl
inv_dur: H-statistic = 12.352845856506827, p-value = 0.03026012062282995
------------------------------------------------------------
 
prev_job_tenure_5y
inv_amounts: H-statistic = 1.3399365413887645, p-value = 0.5117248142019779
 
prev_job_tenure_5

In [140]:
pd.DataFrame(k_w_results, columns=['dep_var','variable','H-statistic','p-value']).to_excel('C:/플랫폼 SME 연구센터/투자여부예측/스타트업기준_K-W_250406.xlsx')

In [166]:
#group_cols = ['cofounder_yn', 'edu_lvl', 'prev_exp_relev_yn', 'prev_job_tenure_5y', 'edu_bg', 'startup_exp', 'prev_mgr_yn', 'prev_c_lvl_yn']
# Kruskal-Wallis 검정 결과 저장을 위한 딕셔너리
#kruskal_results = {}

#for col in group_cols:
# 각 연속형 변수에 대해 Kruskal-Wallis 검정 수행
#    for variable in dep_var:
        # inv_round_1별로 그룹화된 데이터를 리스트로 생성
#        groups = [stup_3[stup_3[col] == round_val][variable] for round_val in stup_3[col].unique()]
    
    # Kruskal-Wallis 검정 수행
#        h_stat, p_value = kruskal(*groups)
    
    # 결과 저장
#        kruskal_results[variable] = {'H-statistic': h_stat, 'p-value': p_value}

# 결과 출력: 모든 변수에 대해 H-통계량과 p-value 출력
#    significant_results = {var: result for var, result in kruskal_results.items()}

#    print("Significant Variables based on Kruskal-Wallis Test:")
#    for var, result in significant_results.items():
#        print(col)
#        print(f"{var}: H-statistic = {result['H-statistic']}, p-value = {result['p-value']}")
#        print(" ")

Significant Variables based on Kruskal-Wallis Test:
cofounder_yn
inv_amounts: H-statistic = 9.64084960050099, p-value = 0.0019029719546143117
 
cofounder_yn
inv_dur: H-statistic = 0.1387700661543757, p-value = 0.7095068072571789
 
Significant Variables based on Kruskal-Wallis Test:
edu_lvl
inv_amounts: H-statistic = 18.955723504141694, p-value = 0.001958979295438725
 
edu_lvl
inv_dur: H-statistic = 12.352845856506827, p-value = 0.03026012062282995
 
Significant Variables based on Kruskal-Wallis Test:
prev_exp_relev_yn
inv_amounts: H-statistic = 0.6665970491911675, p-value = 0.7165562525583653
 
prev_exp_relev_yn
inv_dur: H-statistic = 0.2566852123842253, p-value = 0.8795519875610571
 
Significant Variables based on Kruskal-Wallis Test:
prev_job_tenure_5y
inv_amounts: H-statistic = 1.3399365413887645, p-value = 0.5117248142019779
 
prev_job_tenure_5y
inv_dur: H-statistic = 3.653936526488796, p-value = 0.16090063744513244
 
Significant Variables based on Kruskal-Wallis Test:
edu_bg
inv_a

In [160]:
var, result = kruskal_results['inv_dur'].items()

In [162]:
result

('p-value', 0.7095068072571789)

### 투자자

In [111]:
invst_1 = invst_1.drop(['Case_1'],axis=1)

In [305]:
for i in invst_2.columns[:-2]:
    print(i, invst_2[i].unique())

early_late [0 1]
tech_1 [0 1]
BoD_yn [1 0]
cofounder_yn [1 0]
edu_lvl [3 5 4 0 1 2]
prev_exp_relev_yn [0 1]
prev_job_tenure_5y [2 1 0]
edu_bg [1 0]
startup_exp [1 0 2]


In [344]:
invst_1 = invst_2[['cls_cd','edu_lvl','prev_job_tenure_5y','startup_exp','inv_amounts','inv_dur']]
invst_1

,cls_cd,edu_lvl,prev_job_tenure_5y,startup_exp,inv_amounts,inv_dur
3,1,3,2,1,4.025352,2.833213
4,1,3,1,1,3.433987,2.995732
5,1,3,1,1,3.433987,2.995732
7,1,5,2,0,4.615121,2.197225
23,3,5,1,0,4.394449,2.397895
...,...,...,...,...,...,...
3605,4,3,1,0,5.513429,3.931826
3606,4,3,1,0,5.513429,0.693147
3607,4,3,1,0,5.513429,2.639057
3608,4,3,1,0,5.513429,2.079442


In [83]:
invst_1 = invst_h4_1_a_conti
invst_1 = invst_1.drop(['Case_1'],axis=1)
invst_1['Case_1'] = invst_h4_1_a_nmnl['Case_1']
invst_1

,Patent_total,Patent_above_B_minus,Patent_Class_C,Patent_Class_B_minus,Patent_Class_B0,Patent_Class_B_plus,Patent_Class_A_minus,Patent_Class_A0,Patent_Class_A_plus,Patent_Class_AA,ROE,ROA,ratio_of_netincome_to_sales,equity_ratio,Debt_ratio,pre_2y_sales,pre_1y_sales,rate_of_sales_increase,pre_2y_opincome,pre_1y_opincome,rate_of_opincome_increase,pre_2y_netincome,pre_1y_netincome,rate_of_netincome_increase,pre_2y_assets,pre_1y_assets,rate_of_assets_increase,pre_1y_liabilities,pre_1y_equity,Case_1
3,0.693147,0.693147,0.000000,0.693147,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.065513,0.015207,0.009098,0.204017,1.485812,1.944677,4.060008,2.252288,-0.731675,-1.615458,-1.318187,-0.488746,0.419171,1.038917,2.173857,3.554803,1.472685,3.306536,2.162232,1
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,-1.715532,-1.249428,-0.387923,0.435499,0.605623,3.688756,3.828055,0.142628,-2.159639,-3.105327,-1.022425,-2.152553,-3.105191,-1.030298,3.426610,2.258144,-0.537840,1.587382,1.735963,0
23,1.609438,1.609438,0.000000,1.386294,0.000000,0.693147,0.000000,0.0,0.000000,0.000000,-0.525153,-0.485152,-1.533160,0.643967,0.100905,0.889741,1.047621,0.254834,-1.049643,-2.620529,-1.926218,-0.679505,-2.044305,-1.933126,0.982030,2.465387,1.863836,0.709773,2.373463,1
24,1.609438,1.609438,0.000000,1.386294,0.000000,0.693147,0.000000,0.0,0.000000,0.000000,-0.525153,-0.485152,-1.533160,0.643967,0.100905,0.889741,1.047621,0.254834,-1.049643,-2.620529,-1.926218,-0.679505,-2.044305,-1.933126,0.982030,2.465387,1.863836,0.709773,2.373463,0
26,1.609438,1.609438,0.000000,0.693147,0.693147,0.000000,0.000000,0.0,1.098612,0.000000,-0.214150,-0.198546,-3.454585,0.652160,0.083727,0.329469,0.370618,0.139462,-1.421968,-2.685732,-1.469226,0.106151,-2.691131,-4.826424,2.942056,4.152586,1.248881,1.796370,4.070232,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3741,0.693147,0.693147,0.000000,0.693147,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,-1.313739,-0.892463,-0.935099,0.425148,0.635223,7.673008,7.313120,-0.264198,-6.088889,-6.425195,-0.336955,-7.533757,-7.749503,-0.215851,8.305400,7.384263,-0.471304,6.630329,6.749591,2
3742,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,-1.567081,-0.455764,-0.423390,0.141705,1.882318,6.569175,7.148635,0.580078,-5.010389,-6.134619,-1.128752,-5.551590,-6.509032,-0.959840,6.878386,7.057659,0.179443,6.892660,5.180123,2
3743,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,-0.537131,-0.375570,-0.198518,0.495316,0.444700,4.772378,5.752384,0.985322,-3.943522,-4.240155,-0.301693,-3.901973,-4.247632,-0.351667,4.725616,5.025475,0.302173,4.012656,4.584447,2
3744,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,-0.743088,-0.342171,-0.443151,0.314881,0.993993,6.763108,6.249033,-0.338193,-5.044173,-5.185810,-0.142494,-5.042508,-5.666466,-0.626970,5.498582,6.560903,1.065006,6.099546,5.569314,2


In [84]:
# Kruskal-Wallis 검정 결과 저장을 위한 딕셔너리
kruskal_results = {}

# 각 연속형 변수에 대해 Kruskal-Wallis 검정 수행
for variable in conti_var[2:]:
    # inv_round_1별로 그룹화된 데이터를 리스트로 생성
    groups = [invst_1[invst_1['Case_1'] == round_val][variable] for round_val in invst_1['Case_1'].unique()]
    
    # Kruskal-Wallis 검정 수행
    h_stat, p_value = kruskal(*groups)
    
    # 결과 저장
    kruskal_results[variable] = {'H-statistic': h_stat, 'p-value': p_value}

# 결과 출력: p-value가 0.05 이하인 변수는 유의미한 변수로 간주
significant_results = {var: result for var, result in kruskal_results.items() if result['p-value'] <= 0.05}

print("Significant Variables based on Kruskal-Wallis Test:")
for var, result in significant_results.items():
    print(f"{var}: H-statistic = {result['H-statistic']}, p-value = {result['p-value']}")
    print(" ")

Significant Variables based on Kruskal-Wallis Test:
Patent_total: H-statistic = 22.231316047524896, p-value = 0.0001802511582229873
 
Patent_above_B_minus: H-statistic = 16.753137054021277, p-value = 0.0021584721196292365
 
Patent_Class_C: H-statistic = 20.769809476929794, p-value = 0.0003517406631667408
 
Patent_Class_B_minus: H-statistic = 13.388709719355653, p-value = 0.009524689710743097
 
Patent_Class_B0: H-statistic = 22.407171367523894, p-value = 0.00016627693663174764
 
Patent_Class_B_plus: H-statistic = 19.878505348588224, p-value = 0.0005277461370396004
 
Patent_Class_A_minus: H-statistic = 15.305885964158756, p-value = 0.004107077845429493
 
Patent_Class_A0: H-statistic = 12.41621094730304, p-value = 0.014510259607886272
 
Patent_Class_A_plus: H-statistic = 20.301120663157985, p-value = 0.0004354761268446109
 
ROA: H-statistic = 12.120236069713478, p-value = 0.016479200571750854
 
ratio_of_netincome_to_sales: H-statistic = 16.468335655408506, p-value = 0.0024510045236308784


In [85]:
invst_h4_1_bc_nmnl

,inv_round_1,tech_1,BoD_yn,cofounder_yn,edu_lvl,prev_exp_relev_yn,prev_job_tenure_5y,edu_bg,startup_exp,prev_mgr_yn,prev_c_lvl_yn,Case_1,inv_amounts,inv_dur
3,4,0,1,1,3,0,2,1,1,1,1,1,4.025352,2.833213
23,4,1,1,0,5,0,1,1,0,0,0,1,4.394449,2.397895
54,4,1,1,0,5,0,1,1,0,1,0,1,5.111988,2.564949
56,4,1,1,0,5,0,1,1,0,1,0,1,5.111988,2.772589
104,4,1,0,0,5,0,0,1,0,0,0,3,4.510860,2.944439
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3585,10,0,1,0,3,0,1,1,0,0,0,1,7.992841,3.135494
3590,10,0,1,0,3,0,1,1,0,0,0,1,7.992841,2.639057
3592,10,0,1,0,3,0,1,1,0,0,0,1,7.992841,2.639057
3596,10,0,1,0,3,0,1,1,0,0,0,1,7.738319,0.000000


In [86]:
invst_3 = invst_h4_1_bc_nmnl
#invst_3 = pd.concat([invst_3, invst_h4_1_bc_conti[['inv_amounts', 'inv_dur']]], axis=1)
invst_3

,inv_round_1,tech_1,BoD_yn,cofounder_yn,edu_lvl,prev_exp_relev_yn,prev_job_tenure_5y,edu_bg,startup_exp,prev_mgr_yn,prev_c_lvl_yn,Case_1,inv_amounts,inv_dur
3,4,0,1,1,3,0,2,1,1,1,1,1,4.025352,2.833213
23,4,1,1,0,5,0,1,1,0,0,0,1,4.394449,2.397895
54,4,1,1,0,5,0,1,1,0,1,0,1,5.111988,2.564949
56,4,1,1,0,5,0,1,1,0,1,0,1,5.111988,2.772589
104,4,1,0,0,5,0,0,1,0,0,0,3,4.510860,2.944439
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3585,10,0,1,0,3,0,1,1,0,0,0,1,7.992841,3.135494
3590,10,0,1,0,3,0,1,1,0,0,0,1,7.992841,2.639057
3592,10,0,1,0,3,0,1,1,0,0,0,1,7.992841,2.639057
3596,10,0,1,0,3,0,1,1,0,0,0,1,7.738319,0.000000


In [87]:
for i in invst_3.columns[:-2]:
    print(i, invst_3[i].unique())
    print(" ")

inv_round_1 [ 4  5  6  8 12  9  3  7 11 10]
 
tech_1 [0 1]
 
BoD_yn [1 0]
 
cofounder_yn [1 0]
 
edu_lvl [3 5 4 0 1]
 
prev_exp_relev_yn [0 1 2]
 
prev_job_tenure_5y [2 1 0]
 
edu_bg [1 0 2]
 
startup_exp [1 0 2]
 
prev_mgr_yn [1 0]
 
prev_c_lvl_yn [1 0]
 
Case_1 [1 3]
 


In [308]:
invst_1.columns

Index(['edu_lvl', 'prev_job_tenure_5y', 'startup_exp', 'inv_amounts',
       'inv_dur'],
      dtype='object')

In [345]:
dep_var = ['inv_amounts','inv_dur']

In [346]:
group_cols = ['cls_cd', 'edu_lvl', 'prev_job_tenure_5y', 'startup_exp']

In [347]:
k_w_results = []
# Kruskal-Wallis 검정 결과 저장을 위한 딕셔너리
kruskal_results = {}

for col in group_cols:
# 각 연속형 변수에 대해 Kruskal-Wallis 검정 수행
    for variable in dep_var:
        # inv_round_1별로 그룹화된 데이터를 리스트로 생성
        groups = [invst_1[invst_1[col] == round_val][variable] for round_val in invst_1[col].unique()]
    
    # Kruskal-Wallis 검정 수행
        h_stat, p_value = kruskal(*groups)
    
    # 결과 저장
        kruskal_results[variable] = {'H-statistic': h_stat, 'p-value': p_value}
        k_w_results.append([variable, col, h_stat, p_value])
        print(col)
        print(f'{variable}: H-statistic = {h_stat}, p-value = {p_value}')
        print(' ')

# 결과 출력: p-value가 0.05 이하인 변수는 유의미한 변수로 간주
    significant_results = {var: result for var, result in kruskal_results.items() if result['p-value'] <= 0.05}

    print("Significant Variables based on Kruskal-Wallis Test:")
    for var, result in significant_results.items():
        print(col)
        print(f"{var}: H-statistic = {result['H-statistic']}, p-value = {result['p-value']}")
        print('-------------------------------------------------------------------')
        print(" ")

cls_cd
inv_amounts: H-statistic = 86.17886726855524, p-value = 2.699744165256307e-13
 
cls_cd
inv_dur: H-statistic = 47.71049867757532, p-value = 3.509848875746093e-06
 
Significant Variables based on Kruskal-Wallis Test:
cls_cd
inv_amounts: H-statistic = 86.17886726855524, p-value = 2.699744165256307e-13
-------------------------------------------------------------------
 
cls_cd
inv_dur: H-statistic = 47.71049867757532, p-value = 3.509848875746093e-06
-------------------------------------------------------------------
 
edu_lvl
inv_amounts: H-statistic = 23.210384457626766, p-value = 0.000307692465267522
 
edu_lvl
inv_dur: H-statistic = 36.58597965454337, p-value = 7.25018218027791e-07
 
Significant Variables based on Kruskal-Wallis Test:
edu_lvl
inv_amounts: H-statistic = 23.210384457626766, p-value = 0.000307692465267522
-------------------------------------------------------------------
 
edu_lvl
inv_dur: H-statistic = 36.58597965454337, p-value = 7.25018218027791e-07
------------

In [348]:
pd.DataFrame(k_w_results, columns=['dep_var','Variable','H-statistic','P-value']).to_excel('C:/플랫폼 SME 연구센터/투자여부예측/투자자기준_가설2_a,b_K-W_250406_re.xlsx')

## 카이제곱검정

### 스타트업

In [31]:
stup_2 = stup[stup_nmnl_var]

In [32]:
stup_2

,early_late,tech_1,cls_cd,cofounder_yn,edu_lvl,prev_exp_relev_yn,prev_job_tenure_5y,edu_bg,startup_exp
0,0,0,1,1,3,0,2,1,1
1,1,0,1,1,3,0,2,1,1
2,1,0,1,1,3,0,2,1,1
3,1,0,2,0,4,0,2,1,0
4,1,0,2,0,4,0,2,1,0
...,...,...,...,...,...,...,...,...,...
832,1,1,5,1,5,1,2,1,0
833,1,1,5,1,5,1,2,1,0
834,1,1,5,1,5,1,2,1,0
835,1,1,5,1,5,1,2,1,0


In [33]:
stup_2.columns

Index(['early_late', 'tech_1', 'cls_cd', 'cofounder_yn', 'edu_lvl',
       'prev_exp_relev_yn', 'prev_job_tenure_5y', 'edu_bg', 'startup_exp'],
      dtype='object')

In [34]:
dependent_variable = 'early_late'
independent_variables = ['tech_1', 'cls_cd', 'cofounder_yn', 'edu_lvl',
       'prev_exp_relev_yn', 'prev_job_tenure_5y', 'edu_bg', 'startup_exp']

# 결과를 저장할 리스트
results = []

# 카이제곱 검정 수행
for var in independent_variables:
    # 종속 변수와 독립 변수에 대한 교차 테이블 생성
    contingency_table = pd.crosstab(stup_2[dependent_variable], stup_2[var])
    
    # 카이제곱 검정 수행
    chi2, p_value, _, _ = chi2_contingency(contingency_table)
    
    # 결과 저장
    results.append({
        'Variable': var,
        'Chi2 Statistic': chi2,
        'P-Value': p_value
    })

# 결과를 데이터프레임으로 출력
results_df = pd.DataFrame(results)

# p-value가 0.05 이하인 변수들만 필터링하여 유의미한 변수들 출력
significant_vars = results_df[results_df['P-Value'] <= 0.05]
print(significant_vars)

       Variable  Chi2 Statistic   P-Value
2  cofounder_yn        6.327602  0.011887
3       edu_lvl       23.196372  0.000310


In [38]:
pd.crosstab(stup_2['early_late'],stup_2['cofounder_yn'])

cofounder_yn,0,1
early_late,,
0,214,13
1,537,73


In [36]:
pd.DataFrame(results).to_excel('C:/플랫폼 SME 연구센터/투자여부예측/스타트업기준_초기후기변수_카이제곱검정_250407.xlsx')

In [25]:
results = []

# 카이제곱 검정 수행
for dependent_variable in stup_2.columns:
    for var in independent_variables:
    # 종속 변수와 독립 변수에 대한 교차 테이블 생성
        contingency_table = pd.crosstab(stup_2[dependent_variable], stup_2[var])
    
    # 카이제곱 검정 수행
        chi2, p_value, _, _ = chi2_contingency(contingency_table)
    
    # 결과 저장
        results.append({
            'Dep_Variable': dependent_variable,
            'Variable': var,
            'Chi2 Statistic': chi2,
            'P-Value': p_value
        })

# 결과를 데이터프레임으로 출력
results_df = pd.DataFrame(results)

# p-value가 0.05 이하인 변수들만 필터링하여 유의미한 변수들 출력
significant_vars = results_df[results_df['P-Value'] <= 0.05]
print(significant_vars)

     Dep_Variable            Variable  Chi2 Statistic        P-Value
0     inv_round_1              cls_cd      443.708212   2.384992e-35
1     inv_round_1        cofounder_yn       31.445820   9.354930e-04
2     inv_round_1             edu_lvl      117.233646   2.126363e-06
3     inv_round_1   prev_exp_relev_yn       80.829180   1.182842e-08
6     inv_round_1         startup_exp       64.549355   4.628538e-06
..            ...                 ...             ...            ...
84  prev_c_lvl_yn   prev_exp_relev_yn       28.302132   7.149408e-07
85  prev_c_lvl_yn  prev_job_tenure_5y       69.593046   7.727911e-16
87  prev_c_lvl_yn         startup_exp      336.457594   8.692840e-74
88  prev_c_lvl_yn         prev_mgr_yn      219.195702   1.354637e-49
89  prev_c_lvl_yn       prev_c_lvl_yn      832.870541  3.851159e-183

[72 rows x 4 columns]


In [27]:
results_df.to_excel('C:/플랫폼 SME 연구센터/투자여부예측/범주형변수간상관관계_stup.xlsx')

In [91]:
dependent_variable = 'inv_round_2'
independent_variables = ['cofounder_yn', 'edu_lvl', 'prev_exp_relev_yn', 'prev_job_tenure_5y', 
                          'edu_bg', 'startup_exp', 'prev_mgr_yn', 'prev_c_lvl_yn']

# 결과를 저장할 리스트
results = []

# 카이제곱 검정 수행
for var in independent_variables:
    # 종속 변수와 독립 변수에 대한 교차 테이블 생성
    contingency_table = pd.crosstab(stup_2[dependent_variable], stup_2[var])
    
    # 카이제곱 검정 수행
    chi2, p_value, _, _ = chi2_contingency(contingency_table)
    
    # 결과 저장
    results.append({
        'Variable': var,
        'Chi2 Statistic': chi2,
        'P-Value': p_value
    })

# 결과를 데이터프레임으로 출력
results_df = pd.DataFrame(results)

# p-value가 0.05 이하인 변수들만 필터링하여 유의미한 변수들 출력
significant_vars = results_df[results_df['P-Value'] <= 0.05]
print(significant_vars)

       Variable  Chi2 Statistic   P-Value
0  cofounder_yn        6.327602  0.011887
1       edu_lvl       23.196372  0.000310
4        edu_bg        6.950944  0.030947


### 투자자

In [349]:
invst_2 = invst_h4_1_a_nmnl
invst_2

,early_late,tech_1,BoD_yn,cls_cd,cofounder_yn,edu_lvl,prev_exp_relev_yn,prev_job_tenure_5y,edu_bg,startup_exp,sequantial_inv
3,0,0,1,1,1,3,0,2,1,1,1
4,0,0,0,1,0,3,1,1,1,1,1
5,0,0,0,1,0,3,1,1,1,1,1
7,0,0,0,1,0,5,0,2,0,0,1
8,0,0,0,1,0,5,0,2,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
3739,1,1,1,5,0,4,0,0,0,0,0
3741,1,1,1,5,0,4,1,1,1,0,0
3743,1,1,1,5,0,5,0,0,1,0,0
3744,1,1,1,7,0,5,0,0,1,0,0


In [285]:
invst_2.columns

Index(['early_late', 'tech_1', 'BoD_yn', 'cofounder_yn', 'edu_lvl',
       'prev_exp_relev_yn', 'prev_job_tenure_5y', 'edu_bg', 'startup_exp',
       'sequantial_inv'],
      dtype='object')

In [350]:
dependent_variable = 'sequantial_inv'

In [351]:
independent_variables = ['early_late', 'tech_1', 'BoD_yn', 'cls_cd','cofounder_yn', 'edu_lvl',
       'prev_exp_relev_yn', 'prev_job_tenure_5y', 'edu_bg', 'startup_exp']

In [ ]:
independent_variables = ['inv_round_1', 'BoD_yn', 'cofounder_yn', 'edu_lvl', 'prev_exp_relev_yn', 'prev_job_tenure_5y', 
                         'edu_bg', 'startup_exp', 'prev_mgr_yn', 'prev_c_lvl_yn']

In [352]:
# 결과를 저장할 리스트
results = []

# 카이제곱 검정 수행
for var in independent_variables:
    # 종속 변수와 독립 변수에 대한 교차 테이블 생성
    contingency_table = pd.crosstab(invst_2[dependent_variable], invst_2[var])
    
    # 카이제곱 검정 수행
    chi2, p_value, _, _ = chi2_contingency(contingency_table)
    
    # 결과 저장
    results.append({
        'Variable': var,
        'Chi2 Statistic': chi2,
        'P-Value': round(p_value,3)
    })

# 결과를 데이터프레임으로 출력
results_df = pd.DataFrame(results)

# p-value가 0.05 이하인 변수들만 필터링하여 유의미한 변수들 출력
significant_vars = results_df[results_df['P-Value'] <= 0.05]
print(significant_vars)

     Variable  Chi2 Statistic  P-Value
0  early_late       16.727961      0.0
2      BoD_yn      150.377091      0.0
8      edu_bg       12.207707      0.0


In [354]:
results_df

,Variable,Chi2 Statistic,P-Value
0,early_late,16.727961,0.000
1,tech_1,3.600865,0.058
2,BoD_yn,150.377091,0.000
3,cls_cd,20.500892,0.058
4,cofounder_yn,0.032238,0.858
5,edu_lvl,3.624793,0.605
6,prev_exp_relev_yn,0.538622,0.463
7,prev_job_tenure_5y,2.574830,0.276
8,edu_bg,12.207707,0.000
9,startup_exp,3.659581,0.160


In [353]:
results_df.to_excel('C:/플랫폼 SME 연구센터/투자여부예측/투자자기준_가설2_c_카이제곱검정_250406_re.xlsx')